In [34]:
from bs4 import BeautifulSoup
from bs4.element import NavigableString

def build_document_tree(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # 根节点
    root = {"tag": "root", "children": []}
    stack = [root]  # 栈，用于追踪当前层级的节点

    # 已解析节点集合，存储已处理的节点引用
    parsed_nodes = set()

    # 遍历所有标签
    for tag in soup.descendants:
        # 跳过已解析节点
        if tag in parsed_nodes:
            continue

        if tag.name in ["h1", "h2", "h3", "h4", "h5", "h6", "title"] or (tag.name and "html" in tag.name):  # 标题节点: h1-h6, title, html
            if "html" in tag.name:
                node = {"tag": "html", "text": None, "children": []}
                stack = [root]
                stack[-1]["children"].append(node)
                stack.append(node)
            elif tag.name == "title":
                # 找到之前的html节点，修改其text
                for i in range(len(stack)-1, -1, -1):
                    if stack[i]["tag"] == "html":
                        stack[i]["text"] = tag.get_text()
                        break
                # 如果没有找到html节点，创建一个
                if stack[-1]["tag"] != "html":
                    node = {"tag": "html", "text": tag.get_text(), "children": []}
                    stack[-1]["children"].append(node)
                    stack.append(node)
            else:
              # 先找到最近的层级节点
              if tag.name == stack[-1]["tag"]:
                  # 如果当前标题与前一个标题同级，将当前标题添加到前一个标题的父节点下
                  stack.pop()
              else:
                  # 如果当前标题与前一个标题不同级，那要找到最近的同级标题
                  for i in range(len(stack)-1, -1, -1):
                      if (stack[i]["tag"] != "html" and stack[i]["tag"] != "root" and int(stack[i]["tag"][1]) < int(tag.name[1])) or stack[i]["tag"] == "html":
                          stack = stack[:i+1]
                          break
                  if i == 0:
                      stack = [root]
              
              node = {"tag": tag.name, "text": tag.get_text(), "children": []}

              # 插入到栈顶节点的 children 中
              stack[-1]["children"].append(node)

              # 将当前节点推入栈
              stack.append(node)

            # 标记为已解析
            parsed_nodes.add(tag)

        elif tag.name in ["ul", "ol"]:  # 列表节点
            # 找到最近的层级节点
            parent_node = stack[-1]

            # 创建列表节点
            list_node = {"tag": tag.name, "text": None, "children": []}
            parent_node["children"].append(list_node)

            # 遍历所有直接子节点（不限制为 li）
            for child in tag.find_all(recursive=False):
                if child not in parsed_nodes:
                    if child.name == "li":  # 如果是 li，递归解析
                        list_item = build_list_item_node(child, parsed_nodes)
                        if list_item:
                            list_node["children"].append(list_item)
                    else:  # 其他标签，直接添加为子节点
                        text = child.get_text(strip=True)
                        list_node["children"].append({
                            "tag": child.name,
                            "text": text,
                            "children": []
                        })
                    # 标记为已解析
                    parsed_nodes.add(child)

            # 标记当前列表节点为已解析
            parsed_nodes.add(tag)

        elif tag.name == "table":
            # 表格节点
            node = {"tag": "table", "text": str(tag), "children": []}
            stack[-1]["children"].append(node)

            # 标记当前表格及其所有子节点为已解析
            mark_all_children_as_parsed(tag, parsed_nodes)
        
        elif isinstance(tag, NavigableString):
            continue
        
        elif len(tag.find_all(recursive=False)) == 0:
            # 叶子节点
            text = tag.get_text().strip()
            if text:
                node = {"tag": tag.name, "text": text, "children": []}
                stack[-1]["children"].append(node)

    return root

def build_list_item_node(list_item, parsed_nodes):
    """解析列表项及其嵌套内容"""
    direct_text = ''.join(list_item.find_all(text=True, recursive=False)).strip()
    node = {
        "tag": "li",
        "text": direct_text,
        "children": []
    }

    # 遍历子节点，包括可能存在的非列表项
    for child in list_item.find_all(recursive=False):
        if child in parsed_nodes:
            continue

        if child.name in ["ul", "ol"]:  # 嵌套列表
            nested_list = {"tag": child.name, "text": None, "children": []}
            for nested_item in child.find_all("li", recursive=False):
                nested_list_item = build_list_item_node(nested_item, parsed_nodes)
                if nested_list_item:
                    nested_list["children"].append(nested_list_item)
            node["children"].append(nested_list)
            parsed_nodes.add(child)
        elif child.name and child.get_text(strip=True):  # 非 `ul`, `ol` 标签
            child_node = {
                "tag": child.name,
                "text": child.get_text(strip=True),
                "children": []
            }
            node["children"].append(child_node)
            parsed_nodes.add(child)

    parsed_nodes.add(list_item)  # 标记列表项为已解析
    return node

  
def mark_all_children_as_parsed(element, parsed_nodes):
    """递归标记节点及其所有子节点为已解析"""
    parsed_nodes.add(element)
    for child in element.descendants:
        parsed_nodes.add(child)

# 示例 HTML
html = """
<html>
<header>
  <h1>Document Title</h1>
  <nav>
    <ul>
      <li>Home</li>
      <li>About</li>
      <li>Contact</li>
    </ul>
  </nav>
</header>
<main>
  <section>
    <h2>Introduction</h2>
    <p>Welcome to the document. This is the introduction section.</p>
  </section>
  <section>
    <ul>
      <li>Item 1</li>
      <li>Item 2</li>
    </ul>
  </section>
  <article>
    <h3>Article Title</h3>
    <p>This is an article with a title and some text content.</p>
  </article>
  <div>
    <h4>Div Container Title</h4>
    <p>Content inside a div container with a title.</p>
  </div>
  <table>
    <tr>
      <td>Cell 1</td>
      <td>Cell 2</td>
    </tr>
    <tr>
      <td>Cell 3</td>
      <td>Cell 4</td>
    </tr>
  </table>
</main>
<footer>
  <p>Footer content goes here.</p>
</footer>
</html>
"""

# 解析并生成文档树
document_tree = build_document_tree(html)

def print_tree(node, level=0):
    """递归打印文档树，格式为 Tag, Text, Level"""
    tag = node.get("tag", "unknown")
    text = node.get("text", "")
    print(f"Tag: {tag}, Text: {text}, Level: {level}")

    for child in node.get("children", []):
        print_tree(child, level + 1)

# 打印文档树
print_tree(document_tree)

# # 打印结构化文档树
# import json
# print(json.dumps(document_tree, indent=4, ensure_ascii=False))

Tag: root, Text: , Level: 0
Tag: html, Text: None, Level: 1
Tag: h1, Text: Document Title, Level: 2
Tag: ul, Text: None, Level: 3
Tag: li, Text: Home, Level: 4
Tag: li, Text: About, Level: 4
Tag: li, Text: Contact, Level: 4
Tag: h2, Text: Introduction, Level: 3
Tag: p, Text: Welcome to the document. This is the introduction section., Level: 4
Tag: ul, Text: None, Level: 4
Tag: li, Text: Item 1, Level: 5
Tag: li, Text: Item 2, Level: 5
Tag: h3, Text: Article Title, Level: 4
Tag: p, Text: This is an article with a title and some text content., Level: 5
Tag: h4, Text: Div Container Title, Level: 5
Tag: p, Text: Content inside a div container with a title., Level: 6
Tag: table, Text: <table>
<tr>
<td>Cell 1</td>
<td>Cell 2</td>
</tr>
<tr>
<td>Cell 3</td>
<td>Cell 4</td>
</tr>
</table>, Level: 6
Tag: p, Text: Footer content goes here., Level: 6


/tmp/ipykernel_2399415/3164862471.py:112: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  direct_text = ''.join(list_item.find_all(text=True, recursive=False)).strip()


In [12]:
HTML_test = """
<html0><title>The Greatest Russian Male Figure Skaters of All Time</title><div><div1><div><div><span>COLLECTION</span><span>22 LISTS</span></div><a>Figure Skating</a><span>Lists about the beautiful, technically grueling sports of figure skating and ice dancing and the talented athletes who compete in them all around the world.</span></div><ul><span>The Best Movies About Figure Skating</span><span>The Greatest Male Figure Skaters</span><span>The Greatest Ice Dancers Ever</span><span>The Top Russian Female Figure Skaters</span><span>The Greatest Olympic Figure Skaters</span><span>The Top American Figure Skaters</span><span>Top Female Skaters Working Today</span><span>The Top American Women in History</span><span>The Top Russian Male Figure Skaters</span><span>The Top Russian Figure Skaters</span><span>The Top Skaters Working Today</span><span>The Greatest Women of All Time</span><span>The Top American Male Figure Skaters</span><span>The Top Current Figure Skating Pairs</span><span>The Greatest Ice Dancing Pairs Ever</span><span>The Best Retired Figure Skaters</span><span>The Greatest Men Working Today</span><span>Top Ice Dancing Pairs Working Today</span><span>The Greatest Figure Skating Pairs Ever</span><span>The Top American Ice Dancers</span><span>The Greatest Ice Dancers in the World</span><span>The Top Russian Ice Dancers</span></ul></div1><article><div1><div><p>Russian figure skaters have been making their mark in the sport since the early days of competition. The first time figure skating was held as an Olympic competition was during the 1908 Summer <a>Olympic Games</a> (London), which was also the first and only time a special figures contest was held. With his gold-medal win in this special figures category, Russian figure skater Nikolai Panin became the first Russian to win an Olympic gold medal.</p>\n<p>Russian figures skaters in men\'s\xa0singles competitions have continued to bring glory to Russia up to the present day. The World Figure Skating Championships, for example, were dominated by Russian male skaters in the late \'90s and early 2000s. From 1998 to 2004, it was a Russian who took the gold -\xa0<a>Alexei Yagudin</a>, 1998 to 2000 and 2002; <a>Evgeni Plushenko</a>, 2001, 2003, and 2004.</p>\n<p>This list contains the best skaters in Russian history. These are the men who have inspired newcomers to the sport and drawn audiences to figure skating competitions. They may all have different styles, but they are each worthy of being called the greatest Russian male figure skaters.</p></div><div>Most divisive: <span>Alexei Urmanov</span></div><div>Over 400 Ranker voters have come together to rank this list of The Top Russian Male Figure Skaters</div></div1><ul><li0><div><div><div>1</div><div>225 VOTES</div></div><a>Alexei Yagudin</a></div><li>Photo: <a>YouTube</a></li><div><span>Alexei Yagudin medaled many times in his career at many of figure skating\'s most esteemed events, including the Winter Olympic Games -\xa0<a>he took the gold in 2002</a>.</span>\n<span>225 votes</span><div><div>More\xa0<a>Alexei Yagudin</a></div><ul><li><span>#5<span> of 51 on</span></span><a>The Greatest Male Figure Skaters of All Time</a></li><li><span>#11<span> of 41 on</span></span><a>The Greatest Retired Figure Skaters of All Time</a></li><li><span>#4<span> of 37 on</span></span><a>The Greatest Russian Figure Skaters of All Time</a></li></ul></div></div></li0><li1><div><div><div>2</div><div>216 VOTES</div></div><a>Evgeni Plushenko</a></div><li>Photo: <a>YouTube</a></li><div><span>Evgeni Plushenko counts among his career accomplishments winning the gold at the Russian Figure Skating Championships <a>10 times</a>. </span>\n<span>216 votes</span><div><div>More\xa0<a>Evgeni Plushenko</a></div><ul><li><span>#359<span> of 1,269 on</span></span><a>The 1000+ Best Athletes Of All Time, Ranked</a></li><li><span>#2<span> of 51 on</span></span><a>The Greatest Male Figure Skaters of All Time</a></li><li><span>#2<span> of 37 on</span></span><a>The Greatest Russian Figure Skaters of All Time</a></li></ul></div></div></li1><li3><div><div><div>4</div><div>32 VOTES</div></div><a>Alexandre Fadeev</a></div><li>Photo: <a>YouTube</a></li><div><span>In 1984, <a>1987</a>, 1988, and <a>1999</a>, Alexandre Fadeev (representing the USSR) earned the gold medal at the European Figure Skating Championships.</span>\n<span>32 votes</span></div></li3></ul></article></div></html0>\n<html1><title>Who are the Russian female figure skaters who filled the winners podium-2021? (PHOTOS + VIDEO) - Russia Beyond</title><div1><div0><h1>Who are the Russian female figure skaters who filled the winners podium-2021? (PHOTOS + VIDEO)</h1><a>\n            Lifestyle\n        </a><time>\n             March 30 2021\n        </time><a>\n                    Yekaterina Sinelschikova\n                </a></div0><div2>\n            For the first time in history, Russian figure skaters occupied the entire winners podium at the 2021 World Figure Skating Championships in the ladies’ singles in Stockholm. Previously, only American women skaters had managed the same feat, back in 1991!\n        </div2><div3><h30>Anna Shcherbakova - Gold medalist</h30><div0><p>Anna Shcherbakova of Russia competes in Women\'s Free Skating during the ISU World Figure Skating Championships on March 26, 2021 in Stockholm </p>\nJoel Marklund/Keystone Press Agency/Global Look Press\n            </div0><h31>Elizaveta Tuktamysheva - Silver medalist</h31><p7>A young Tuktamysheva had prevailed at Russian Championships at the end of the noughties, particularly in the ladies singles. She was labelled the child prodigy of figure skating, since she had come to the sport relatively late, at the age of nine, and by the time she was 12, she was already performing extremely difficult elements, having mastered the whole arsenal of triple jumps. It was thanks to these that she earned a reputation as the most “leaping” female figure skater.</p7><p8>“I trained in the old school. I started skating and winning in major league sports very early. This was 2009-2010, when there had been a decline in Russian ladies’ singles skating and you just needed to know how to jump. If you could jump, you would win. That’s the way it turned out ever since I was young. I jumped and won. It was normal. It was how the system worked,” she <a>recounted</a>.</p8><p9>In 2018-2019, she was arguably the most talked about figure skater in the world, after her bewitching performance in a flight attendant’s costume to the accompaniment of Britney Spears’ Toxic in the Grand Prix Series. Accusations that she had “crossed the line” rained down on Tuktamysheva then, to which she just retorted: “I’m just putting the sexiness back into figure skating.”</p9><p13>She is talked about as the main record-setter in Russian figure skating. Trusova was the first female figure skater in history to perform three quads in a single program. In terms of technique, she has already beaten all the adult champions. Recently, a video of her demonstration program was reposted by Hollywood actress Sharon Stone.</p13><p14>The young skater herself admits that only two things in life leave her exhilarated - skating and her dogs (Tina the chihuahua, Lana the poodle and Jack the husky). She has been managed by Evgeni Plushenko since 2020 - and lives in a cottage attached to his academy, so as not to waste time on travel.</p14></div3><div4><a>\n                    sport\n                </a><a>\n                    figure skating\n                </a></div4></div1></html1>\n<div0>\n<span>\n<a>Home</a>\n<a>/ Lifestyle</a>\n<a>/ People</a>\n</span>\n<h1>Figure skating legends inspire youth at Olympic Day event</h1>\n<span>\n          By China Daily | \n          chinadaily.com.cn |\n          Updated: 2024-06-24 15:22\n</span>\n<span>Share</span>\n<span>Share - WeChat</span>\n<span>CLOSE</span>\n<div>\n<div>\n<figcaption>\n   Former Team China figure skater Tong Jian guides students to grasp skating skills on Sunday.\xa0[Photo provided to China Daily] \n </figcaption>\n<p>A figure skating public welfare class, held at the National Speed Skating Oval on Sunday to commemorate Olympic Day, became a platform for inspiration and skill-building for young enthusiasts.</p>\n<p>Former Team China figure skater Tong Jian, a two-time world champion in figure skating pairs alongside his partner Pang Qing, and Russian international coach Andrei Lazukin, took the lead in guiding the eager students through ice skating exercises. Their hands-on approach not only showcased the intricacies of the sport but also nurtured a relaxed and enjoyable learning environment for all participants.</p>\n</div>\n<div>\n<span>1</span>\n<a>2</a>\n<a>3</a>\n<a>4</a>\n<a>Next</a>\xa0\xa0 <a>&gt;&gt;|</a>\n</div>\n<div>\n<a>\n                1/4\n              </a>\n<a>Next</a>\n</div>\n</div>\n</div0>\n<html3>\n<title>Katya Alexandrovskaya: How Did the Figure Skater Die?</title>\n<div0>\n<div0>\n<h1>Katya Alexandrovskaya: How Did the Figure Skater Die?</h1> <div>\n<a> Kriti Mehrotra</a>\n<span>September 16, 2023</span>\n<div>\n<span>Facebook</span><span>Twitter</span><span>Pinterest</span><span>Reddit</span><span>Email</span> </div>\n</div>\n</div0>\n<div>\n<p2>“[Katya] was so tiny, small even for her young age,” her first local coach stated. “I took a closer look at her. This girl was amazing right from the start. She was so swift… and absolutely fearless.” It thus comes as no surprise she began competing at 11 itself, only to then find herself pairing up with Vladislav Lysoy and Aleksandr Epifanov before finding a long-term partner in Harley Windsor. The truth is they’d met in December 2015 for a tryout, but it was evident there was magic between the duo the moment they stepped foot into the rink together thanks to their similar techniques.</p2>\n<p4>Hence came Katya’s release by Russia upon the Australian Association’s request, a move down under, expedited citizenship, and countless training sessions to ensure the pair could debut in 2016. Their first Junior Grand Prix (JGP) in the Czech Republic wasn’t really great, yet they returned with a bang for the Estonia event less than a month later, meaning they earned gold medals. Then came the duo’s 2017 Junior World Championship, 2018 Junior Grand Prix Win, and 2018 Winter Olympics participation, just for them to end up ranking 18 before deciding to change coaches.</p4>\n<p7>Katya was thus medically ordered to step away from her love of skating for good, as a result of which her partnership with Harley also came to a mutual end and she found herself isolated, per the movie. Alas, it all got too much for the 20-year-old and she jumped from the 6th-floor window of her Moscow home on July 18, 2020 — she died by suicide, leaving behind a mere note reading “Lyublyu” (Rough Translation: I love).</p7>\n</div>\n</div0>\n</html3>\n<html4>\n<title>List of Russian champions in figure skating – Pairs - FamousFix List</title>\n<div>\n<div>\n<div>\n<h1>Russian champions in figure skating – Pairs</h1>\r\nThis list has <i>28 members</i>.\r\n</div>\n<div>\r\nFLAG\r\n</div>\n<span>\r\nLike \n</span>\n</div>\n<ul>\n<div0>\n<div>\n<a>Ekaterina Gordeeva</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#1</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>36 </span>\n<span>3 </span>\n<span>2 </span>\n</div>\n<div>Ekaterina "Katia" Alexandrovna Gordeeva (Russian: Екатерина Александровна Гордеева; born 28 May 1971) is a Russian former figure skater. Together with her husband, the late Sergei Grinkov, she was the 1988 and 1994 Olympic Champion and four-time World Champion (1986, 1987, 1989, 1990) in pair skating. After Grinkov\'s death, Gordeeva continued performing as a singles skater.</div>\n<ul>\n<li>\n<a>Figure skaters from Moscow</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>232T</span>\n</li>\n<li>\n<a>Competitors at the 1990 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>351T</span>\n</li>\n<li>\n<a>Russian State University of Physical Education, Sport, Youth and Tourism alumni</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>49T</span>\n</li>\n</ul>\n</div0>\n<div1>\n<div>\n<a>Yelena Berezhnaya</a>\n<span>Pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#2</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n</div>\n<div>Elena Viktorovna Berezhnaya (Russian: Елена Викторовна Бережная, born 11 October 1977) is a Russian former pair skater. With partner Anton Sikharulidze, she is the 1998 and 1999 World champion, 1998 Olympic silver medalist and 2002 Olympic champion.</div>\n<ul>\n<li>\n<a>Competitors at the 1998 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>314T</span>\n</li>\n<li>\n<a>Competitors at the 2001 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>345T</span>\n</li>\n<li>\n<a>Sportspeople from Stavropol Krai</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>47T</span>\n</li>\n</ul>\n</div1>\n<div2>\n<div>\n<a>Sergei Grinkov</a>\n<span>Russian pair skater (1967–1995)</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#3</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n</div>\n<div>Sergei Mikhailovich Grinkov (Russian: Серге́й Миха́йлович Гринько́в; 4 February 1967 — 20 November 1995) was a Russian pair skater. Together with partner and wife Ekaterina Gordeeva, he was the 1988 and 1994 Olympic Champion and a four-time World Champion (1986, 1987, 1989, 1990).</div>\n<ul>\n<li>\n<a>Figure skaters from Moscow</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>232T</span>\n</li>\n<li>\n<a>Competitors at the 1990 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>351T</span>\n</li>\n<li>\n<a>Russian State University of Physical Education, Sport, Youth and Tourism alumni</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>49T</span>\n</li>\n</ul>\n</div2>\n<div3>\n<div>\n<a>Ksenia Stolbova</a>\n<span>Figure skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#4</b> ·\r\n\t\t\t\t\t\t\t</div>\n<div>Ksenia Andreyevna Stolbova (Russian: Ксе́ния Андре́евна Столбо́ва; born 7 February 1992) is a retired Russian pair skater who skated with Andrei Novoselov and Fedor Klimov. She is the 2014 Olympic silver medalist, the 2014 Olympic champion in the team event, the 2014 World silver medalist, a three-time European medalist (2012 bronze, 2014, 2015 silver), the 2015–16 Grand Prix Final champion, the 2013 Winter Universiade champion, a two-time World Junior medalist (2010 bronze, 2011 silver), and a three-time Russian national champion (2014, 2015, 2017). Stolbova announced her retirement from competitions on 12 February 2020.</div>\n<ul>\n<li>\n<a>Figure skaters from Saint Petersburg</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>99T</span>\n</li>\n<li>\n<a>Competitors at the 2013 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>254T</span>\n</li>\n<li>\n<a>Universiade gold medalists for Russia</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>495T</span>\n</li>\n</ul>\n</div3>\n<div4>\n<div>\n<a>Tatiana Volosozhar</a>\n<span>Russian-Ukrainian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#5</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>1 </span>\n</div>\n<div>Tatiana Andreyеvna Volosozhar (Russian: Татья́на Андре́евна Волосожа́р, Ukrainian: Тетяна Андріївна Волосожар; born 22 May 1986) is a Ukrainian-born Russian pair skater. With Maxim Trankov, she is the two-time 2014 Olympic champion in the pairs and in team events, the 2013 World champion, a four-time (2012, 2013, 2014 and 2016) European champion, the 2012 Grand Prix Final champion, and a three-time (2011, 2013 and 2016) Russian national champion. They have also won six events on the Grand Prix series.</div>\n<ul>\n<li>\n<a>Universiade silver medalists for Ukraine</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>146T</span>\n</li>\n<li>\n<a>Season\'s world number one figure skaters</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>32T</span>\n</li>\n<li>\n<a>Season-end world number one figure skaters</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>57T</span>\n</li>\n</ul>\n</div4>\n<div5>\n<div>\n<a>Maxim Trankov</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#6</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>1 </span>\n</div>\n<div>Maxim Leonidovich Trankov (Russian: Макси́м Леони́дович Транько́в, born 7 October 1983) is a Russian pair skater. With Tatiana Volosozhar, he is a two-time  2014 Olympic champion in the pairs and in team events, the 2013 World champion, a four-time (2012, 2013, 2014, 2016) European champion, the 2012 Grand Prix Final champion, and a three-time (2011, 2013, 2016) Russian national champion. They have also won six events on the Grand Prix series.</div>\n<ul>\n<li>\n<a>Competitors at the 2005 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>131T</span>\n</li>\n<li>\n<a>Universiade bronze medalists for Russia</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>263T</span>\n</li>\n<li>\n<a>Season\'s world number one figure skaters</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>31T</span>\n</li>\n</ul>\n</div5>\n<div6>\n<div>\n<a>Tatiana Totmianina</a>\n<span>Pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#7</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>1 </span>\n</div>\n<div>Tatiana Ivanovna Totmianina (Russian: Татьяна Ивановна Тотьмянина; born 2 November 1981) is a Russian former competitive pair skater. With partner Maxim Marinin, she is the 2006 Olympic champion, two-time World champion, and five-time European champion. The pair began skating together in 1996 and retired from competition in 2006.</div>\n<ul>\n<li>\n<a>Sportspeople from Perm, Russia</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>113T</span>\n</li>\n<li>\n<a>Russian female pair skaters</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>107T</span>\n</li>\n<li>\n<a>Medalists at the 2006 Winter Olympics</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>454T</span>\n</li>\n</ul>\n</div6>\n<div7>\n<div>\n<a>Vera Bazarova</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#8</b> ·\r\n\t\t\t\t\t\t\t</div>\n<div>Vera Yevgenyevna Bazarova (Russian: Ве́ра Евге́ньевна База́рова, born 28 January 1993) is a Russian pair skater. With former partner Yuri Larionov, she is the 2012–13 Grand Prix Final silver medalist, a three-time European medalist (silver in 2012; bronze in 2011 and 2014), the 2007 World Junior silver medalist, and the 2012 Russian national champion. They won six senior Grand Prix series medals outside the final, including gold at the 2012 NHK Trophy. After their partnership ended, Bazarova competed with Andrei Deputat.</div>\n<ul>\n<li>\n<a>Competitors at the 2015 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>323T</span>\n</li>\n<li>\n<a>Figure skaters at the 2014 Winter Olympics</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>149T</span>\n</li>\n<li>\n<a>Sportspeople from Yekaterinburg</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>187T</span>\n</li>\n</ul>\n</div7>\n<div8>\n<div>\n<a>Anton Sikharulidze</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#9</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n</div>\n<div>Anton Tarielyevich Sikharulidze (Russian: Антон Тариэльевич Сихарулидзе, born 25 October 1976) is a Russian former pair skater. With Elena Berezhnaya, he is the 1998 and 1999 World champion, 1998 Olympic silver medalist and 2002 Olympic champion.</div>\n<ul>\n<li>\n<a>Competitors at the 1998 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>314T</span>\n</li>\n<li>\n<a>Competitors at the 2001 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>345T</span>\n</li>\n<li>\n<a>Figure skaters from Saint Petersburg</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>92T</span>\n</li>\n</ul>\n</div8>\n<div9>\n<div>\n<a>Maria Petrova</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#10</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>1 </span>\n</div>\n<div>Maria Igorevna Petrova (Russian: Мария Игоревна Петрова; born 29 November 1977) is a Russian pair skater. With partner Alexei Tikhonov, she is the 2000 World champion and a two-time (1999, 2000) European champion</div>\n<ul>\n<li>\n<a>Competitors at the 2001 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>345T</span>\n</li>\n<li>\n<a>Figure skaters from Saint Petersburg</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>99T</span>\n</li>\n<li>\n<a>Competitors at the 1997 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>67T</span>\n</li>\n</ul>\n</div9>\n</ul>\n</div>\n</html4>\n<div>\n<div0>\n<div>\n<h1>Skating colleagues remember Sergei Grinkov, 25 years after his death</h1>\n<bsp-page-byline><div>\n<ul>\n<li>By</li>\n<li><a>Lynn Rutherford</a><span>,</span></li>\n</ul>\n<ul>\n<li>By</li>\n<a>Lynn Rutherford</a>\n</ul>\n</div>\n<div>Published\xa0November 20, 2020 11:54 AM</div>\n</bsp-page-byline>\n</div>\n<ul>\n<span>Facebook</span>\n<span>Twitter</span>\n<span>\n        Email\n    </span>\n</ul>\n</div0>\n<div0>\n<div0>\n<div>ZbUp0WDqkYB1</div>\n<div>November 20, 2020 04:04 PM</div>\n<div>Two-time Olympic pairs figure skating champion Ekaterina "Katia" Gordeeva recalls the death of her husband and skating partner, Sergei Grinkov.</div>\n</div0>\n<div1>\n<p0>On Nov. 20, 1995, two-time Olympic pairs champion <b>Sergei Grinkov</b>, aged 28, died of a heart attack in Lake Placid, New York, while practicing with his wife and partner, <b>Ekaterina “Katia” Gordeeva</b>, for the opening of the Stars on Ice tour just five days away.</p0>\n<p8>Gordeeva and Grinkov, who represented the Soviet Union, have an illustrious competitive legacy, winning four world titles (1986, 1987, 1989, 1990) and 1988 Olympic gold before turning professional in late 1990. The couple regained their Olympic eligibility for the 1993-1994 season and won a second Olympic title, that time competing for Russia.</p8><p13>“It was the quality they brought,” Hamilton added. “They were Olympic champions, and Katia was sort of the <b>Nadia [Comăneci]</b> of the 1988 Games. She was so young (16) and beautiful, or adorable rather than beautiful. She captivated a lot of people. It was really fascinating [because] at that time the Soviet Union was a mystery to a lot of people.”</p13><p15>“Sergei had the look, the line, the quality, and he had great feet in lifts,” Hamilton said. “He was a very strong and big man (about 6 feet, 175 pounds), but he skated with this effortless power. It didn’t seem like it was making any noise over the ice. With Katia, she was always sort of floating above the ice, and they sort of meshed [with] each other. There was no heaviness to their skating, it was beautiful and light.”</p15><p17><b>Elena Bechke</b>, the 1992 Olympic silver medalist with <b>Denis Petrov</b>, grew up in the Soviet Union and competed against Gordeeva and Grinkov while skating with an earlier partner. She and Petrov performed with Stars on Ice for many years, and were present in Lake Placid when Grinkov died.</p17><p23>“They were very light, like (three-time world champion) <b>Patrick Chan</b>,” she said. “That is the difference. (Chan) skates like a feather, flowing on the ice. It’s the same talent Katia and Sergei had. They had natural rhythm; that’s why their pair elements were perfect. They had natural harmony… I saw this, and I emphasized this in the choreography I did for them: the lightness, the connection, how they looked at each other, the bodylines.”</p23><p27>“Celebration of a Life” marked Gordeeva’s debut as a singles skater, in programs choreographed by Zoueva to Tchaikovsky’s “Serenade for Strings” and Mahler’s “Adagietto.” Just 24 when Grinkov died, the skater wrote a bestseller (with E.M. Swift) in 1996 about her life with her first husband, “My Sergei,” and enjoyed a decades-long performing career. But in 1995 her professional future, and earning power, was uncertain.</p27><p31>“All of Katia and Sergei was a love story,” Zoueva said. “They started skating (together) when they were around 11 and 14. First, they were very friendly like brother and sister, then they fell in love, then they were married and had a baby. It was always about their love… Sergei was very, very happy. Always laughing, had lots of friends, loved to skate, loved to perform, loved Katia and loved his daughter. There was a lot of love.”</p31><p33>“There is a memorial plaque with several pictures of Katia and Sergei that sits on the wall steps from my office, which is a daily reminder of Sergei,” Wylie said. “Whenever I go to the USA rink, which is where Sergei passed away, I go to the blue line to the right of entrance and I think about that moment. And whenever I stand where the Zamboni comes on, I remember when Elena Bechke ran in and said, ‘Oh my gosh, Sergei fell down.’ It was one of those moments where your life just changes.”</p33>\n</div1>\n</div0>\n</div>\n<html6><title>Meet Kamila Valieva, Olympic Champion 2022 in figure skating in team classification (PHOTOS) - Russia Beyond</title><div1><div0><h1>Meet Kamila Valieva, Olympic Champion 2022 in figure skating in team classification (PHOTOS)</h1><a>\n            Lifestyle\n        </a><time>\n             Feb 08 2022\n        </time><a>\n                    Victoria Ryabikova\n                </a></div0><div3><p0>After the Russian figure skating team won a gold medal, the awarding ceremony scheduled for February 7 was postponed because of unclear reasons. Soon, <a>Inside The Games</a> and other media quoting anonymous sources reported that the ceremony delay was reportedly caused by a positive doping test of Kamila Valieva.</p0><div1><p>Russia\'s Kamila Valiyeva performs a short program in the women\'s single skating at the 6th stage of the Figure Skating World Cup in Sochi</p>\nAlexander Vilf/Sputnik\n            </div1><div3><p>Kamila Valiyeva after performing her free skating program at the Russian Figure Skating Championship in St. Petersburg. December 25, 2021</p>\nAlexei Danichev/Sputnik\n            </div3><p14>In 2019/2020, her first season together with Tutberidze, Kamila’s program was ‘Girl on a Ball’ (short program), based on the painting of the same name by the Spanish artist Pablo Picasso. Then, the program was noticed by Picasso’s granddaughter Diana and personally <a>congratulated </a>the figure skater. She also <a>won</a> with the program at the Junior Grand Prix in France (where she <a>became</a> the fourth junior skater in history to score over 200 points) and <a>then</a> the World Junior Championships. That same season, Kamila performed a free routine to the music of rock band Muse, ‘Exogenesis: Symphony Pt.3 (Redemption)’.</p14><div4><p>Kamila Valiyeva (Russia) at the exhibition performance of the Grand Prix of Figure Skating in Moscow. 22.11.2020 </p>\nAlexander Wilf/Sputnik\n            </div4><p15>The following season, Kamila tried her hand at adult figure skating with the even more difficult ‘Bolero’ program (free skating), but only in Russia.\xa0 Because of the pandemic, all junior championships were canceled that year, but the Russian Figure Skating Federation made an exception for Kamila, who was allowed to compete as an adult. Her main achievement of the season was gold at the Russian Cup. In the finals, she fell while attempting a quadruple salchow and also missed a jump, but still <a>took</a> first place.\xa0</p15><div5><p>Kamila Valieva of Russia prepares in the Junior Ladies Free Skating during the ISU Grand Prix of Figure Skating Final (Senior &amp; Junior) at Palavela Arena on December 06, 2019 in Turin, Italy</p>\nJoosep Martinson/ISU/Getty Images\n            </div5><p18>With the program ‘In Memoriam’ and with ‘Bolero’, the athlete <a>won</a> gold at the international tournament Skate Canada, won gold at the Russian Championship, becoming the main contender to participate in the Olympic Games in Beijing. In January 2022, Kamila <a>won</a> the European Championship, setting a world record in the short program (90.45). By the way, the record for the free program (185.29) and total points (272.71) are also hers and were set at the Rostelecom Cup in November 2021.\xa0</p18><div6><p>Kamila Valieva of Russia performs during Junior Ladies Free Skating during the ISU Junior Grand Prix of Figure Skating Grand Prix de Courchevel at on August 23, 2019 in Courchevel, France</p>\nMaja Hitij/ISU/Getty Images\n            </div6><p21>Valiyeva is <a>called</a> a unique figure skater, due to her elasticity and jumping skills - she is also strong in ligatures, rotations and component scores. In an interview, Kamila said that the secret to her success is repeating her program every day, as well as her coach.\xa0</p21><div7><p>Figure skater Kamila Valiyeva of Russia performs during an exhibition gala at the 2022 ISU European Figure Skating Championships at the Tondiraba Ice Hall</p>\nSergei Bobylev/TASS\n            </div7></div3><div4><a>\n                    sport\n                </a><a>\n                    olympic games\n                </a><a>\n                    figure skating\n                </a><a>\n                    olympic\n                </a></div4></div1></html6>\n<html7><title>Meet the Russian figure skaters of the 2022 Winter Olympics (PHOTOS+VIDEOS) - Russia Beyond</title><div1><div0><h1>Meet the Russian figure skaters of the 2022 Winter Olympics (PHOTOS+VIDEOS)</h1><a>\n            Lifestyle\n        </a><time>\n             Dec 28 2021\n        </time><a>\n                    Victoria Ryabikova\n                </a></div0><div1>\nAlexander Demyanchuk / TASS; Getty Images; Natalia Fedosenko / TASS\n    </div1><div3><div0><p>Saint-Petersburg. The Russian Figure Skating Championship 2022. Sports complex "Jubilee". Kamila Valieva during the demonstration performances.</p>\nAlexander Demyanchuk / TASS\n            </div0><div1><p>Kamila Valieva performs a short program in women\'s single skating at the final of the Russian Figure Skating Cup in Moscow</p>\nVladimir Song / Sputnik\n            </div1><p3>On December 26, 2021, Kamila won the title of Russian Champion in figure skating among adults and became the first participant for the 2022 Winter Olympics in Beijing.\xa0</p3><div2><p> Alexandra Trusova of Russia skates in the Women\'s Free Skate during the ISU Grand Prix of Figure Skating - Skate America at Orleans Arena on October 24, 2021 in Las Vegas, Nevada</p>\nGetty Images\n            </div2><p5>Alexandra is only 17 and the media is already calling her the <a>‘Quad Queen’.</a> And for good reason – Trusova is the first woman in history to have landed a quadruple Lutz in (ISU JGP 2018) and a quadruple flip (ISU GP Final 2019) in competition. She is also the first woman to land a quadruple toe loop, as well as two quadruple jumps (toe loop and Salchow) in one program in competition (at the ISU World Junior Championships 2018). She has also performed three quadruple jumps (quad Lutz, quad toe, quad toe-triple toe) at the Russian test skates in September 2019 and four quadruple jumps (Lutz, Salchow, two toe loops) at the Japan Open in October 2019. In 2018 and 2019, Alexandra became the world junior champion twice.</p5><p6>Born in Ryazan, the skater has two younger brothers and her father is a master in martial arts. From the age of four, Sasha has been interested in skating.\xa0 At first, her parents gave her roller skates, but when they noticed their daughter almost skating on the ceiling, they decided to train her in figure skating. For five years, she skated in Ryazan and then went to Moscow to skate under Eteri Tutberidze.</p6><div3><p>Alexandra Trusova during demonstration performances at the Russian Figure Skating Championships in St. Petersburg</p>\nAlexander Wilf / Sputnik\n            </div3><div4><p>Stockholm. Anna Shcherbakova (Russia) during a performance in the free program of women\'s single skating at the World Figure Skating Championships</p>\nNatalia Fedosenko / TASS\n            </div4></div3><div6><div>\n                        Read more\n                    </div><ul><a>Meet Anna Shcherbakova, Russia’s newest (and most relentless) Figure Skating World Champion (PHOTOS)</a><a>Who are the Russian female figure skaters who filled the winners podium-2021? (PHOTOS + VIDEO)</a><a>Top 5 YOUNG Russian figure skaters off to a promising start (PHOTOS, VIDEO)</a></ul></div6></div1></html7>\n<html8>\n<title>Famous Russian Figure Skaters</title>\n<div0>\n<h1>Famous Russian Figure Skaters</h1>\n<div1>\n<article0><div><div>\xa01\xa0</div><div>Ekaterina Gordeeva<div>(Russian Figure Skater and Olympic Gold Medalist)</div></div></div><div><span>241</span><span>161</span></div><div><div><b>Birthdate: </b>May 28, 1971</div><div><b>Sun Sign: </b>Gemini</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Height: </b>5\'1" (155 cm)</div></div></article0><article1><div><div>\xa02\xa0</div><div>Sergei Grinkov<div>(Russian Figure Skater and Olympic Gold Medalist)</div></div></div><div><span>706</span><span>164</span></div><div><div><b>Birthdate: </b>February 4, 1967</div><div><b>Sun Sign: </b>Aquarius</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Died: </b>November 20, 1995</div><div><b>Height: </b>6\'0" (183 cm)</div></div></article1><article2><div><div>\xa03\xa0</div><div>Alina Zagitova<div>(Russian Figure Skater and Olympic Gold Medalist)</div></div></div><div><span>188</span><span>4972</span></div><div><div><b>Birthdate: </b>May 18, 2002</div><div><b>Sun Sign: </b>Taurus</div><div><b>Birthplace: </b>Izhevsk, Russia</div><p>Alina Zagitova is a Russian figure skater who has won one gold medal each in <em>Olympic Games, World Figure Skating Championships, European Figure Skating Championships</em>, and <em>Grand Prix of Figure Skating Final</em>; she is the only female figure skater from Russia to have achieved this feat. At the 2019 <em>Glamour Russia Awards</em>, Alina Zagitova was named <em>Sportswoman of the Year.</em></p></div></article2><div><span>Recommended Lists: </span> <a>Russia</a> <a>Russian Female Figure Skaters</a> </div><article3><div><div>\xa04\xa0</div><div><a>Yulia Lipnitskaya</a><div>(Skater)</div></div></div><div><span>99</span><span>151</span></div><div><div><b>Birthdate: </b>June 5, 1998</div><div><b>Sun Sign: </b>Gemini</div><div><b>Birthplace: </b>Yekaterinburg, Russia</div><p>Yulia Lipnitskaya stunned her fans with the news of her retirement at the tender age of 19. Apparently, she had been struggling with injuries and anorexia. The Russian professional figure skater won an <strong>Olympic </strong>gold at 15, thus becoming the youngest Russian gold medalist in history.</p></div></article3><article4><div><div>\xa05\xa0</div><div>Evgenia Medvedeva<div>(Former Competitive Russian Figure Skater)</div></div></div><div><span>40</span><span>8437</span></div><div><div><b>Birthdate: </b>November 19, 1999</div><div><b>Sun Sign: </b>Scorpio</div><div><b>Birthplace: </b>Moscow, Russia</div><p>Evgenia Medvedeva is a Russian figure skater who has won two silver <em>Olympic </em>medals, two <em>World Championship </em>gold medals, and two <em>European Championship</em> gold medals. At the 2017 <em>World Championships</em>, Medvedeva became the first 16-year-old female skater after Michelle Kwan to win back-to-back world titles. Evgenia Medvedeva is also the first skater to achieve two <em>Grand Slams</em> in a row.\xa0</p></div></article4><article5><div><div>\xa06\xa0</div><div>Evgeni Plushenko<div>(Russian Former\xa0Figure Skater\xa0and Olympic Gold Medalist)</div></div></div><div><span>464</span><span>591</span></div><div><div><b>Birthdate: </b>November 3, 1982</div><div><b>Sun Sign: </b>Scorpio</div><div><b>Birthplace: </b>Dzhamku, Russia</div><div><b>Height: </b>5\'10" (178 cm)</div></div></article5><article7><div><div>\xa08\xa0</div><div>Kamila Valieva<div>(Russian Figure Skater and Olympic Gold Medalist)</div></div></div><div><span>22</span><span>444</span></div><div><div><b>Birthdate: </b>April 26, 2006</div><div><b>Sun Sign: </b>Taurus</div><div><b>Birthplace: </b>Kazan, Russia</div><div><b>Height: </b>5\'1" (155 cm)</div></div></article7><article8><div><div>\xa09\xa0</div><div>Elizaveta Tuktamysheva<div>(Russian Figure Skater)</div></div></div><div><span>22</span><span>148</span></div><div><div><b>Birthdate: </b>December 17, 1996</div><div><b>Sun Sign: </b>Sagittarius</div><div><b>Birthplace: </b>Glazov, Russia</div></div></article8><article9><div><div>\xa010\xa0</div><div>Anna Shcherbakova<div>(Russian\xa0Figure Skater, Television Presenter, and Olympic Gold Medalist)</div></div></div><div><span>25</span><span>350</span></div><div><div><b>Birthdate: </b>March 28, 2004</div><div><b>Sun Sign: </b>Aries</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Height: </b>5\'2" (157 cm)</div></div></article9><article12><div><div>\xa013\xa0</div><div>Ilia Kulik<div>(Russian Figure Skater)</div></div></div><div><span>19</span><span>142</span></div><div><div><b>Birthdate: </b>May 23, 1977</div><div><b>Sun Sign: </b>Gemini</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Height: </b>5\'11" (180 cm)</div></div></article12><article13><div><div>\xa014\xa0</div><div>Sergei Ponomarenko<div>(Russian Former Competitive Ice Dancer and Figure Skater)</div></div></div><div><span>13</span><span>137</span></div><div><div><b>Birthdate: </b>October 6, 1960</div><div><b>Sun Sign: </b>Libra</div><div><b>Birthplace: </b>Moscow, Russia</div><p>Sergei Ponomarenko is a Russian retired ice dancer who represented the Unified team of the Soviet Union at the 1992 <em>Olympic Games</em>, <em>World Figure Skating Championships</em>, and <em>European Championships</em>, winning gold medals in all three tournaments. He has also represented the Soviet Union in many <em>World Championships </em>and <em>European Championships,</em> winning two and three gold medals respectively.</p></div></article13><article14><div><div>\xa015\xa0</div><div>Adelina Sotnikova<div>(Russian Figure skater)</div></div></div><div><span>22</span><span>137</span></div><div><div><b>Birthdate: </b>July 1, 1996</div><div><b>Sun Sign: </b>Cancer</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Height: </b>5\'5" (165 cm)</div></div></article14><article15><div><div>\xa016\xa0</div><div>Irina Slutskaya<div>(Russian Former Figure Skater)</div></div></div><div><span>17</span><span>141</span></div><div><div><b>Birthdate: </b>February 9, 1979</div><div><b>Sun Sign: </b>Aquarius</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Height: </b>5\'3" (160 cm)</div></div></article15><article17><div><div>\xa018\xa0</div><div>Alexei Yagudin<div>(Former Russian Competitive\xa0Figure Skater\xa0and Olympic Gold Medalist)</div></div></div><div><span>25</span><span>1400</span></div><div><div><b>Birthdate: </b>March 18, 1980</div><div><b>Sun Sign: </b>Pisces</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><div><b>Height: </b>5\'9" (175 cm)</div></div></article17><article18><div><div>\xa019\xa0</div><div>Elena Ilinykh<div>(Russian Former Competitive\xa0Ice Dancer and Olympic Gold Medalist)</div></div></div><div><span>16</span><span>131</span></div><div><div><b>Birthdate: </b>April 25, 1994</div><div><b>Sun Sign: </b>Taurus</div><div><b>Birthplace: </b>Aktau, Kazakhstan</div><p>Best known for her phenomenal performances with partner Ruslan Zhiganshin, retired Russian ice dancer Elena Ilinykh is a former national champion. Though initially trained in single skating, she later moved on to ice dancing. She also has two <strong>Olympic </strong>medals, including a gold, and three European championship medals in her kitty.</p></div></article18><article19><div><div>\xa020\xa0</div><div>Tatiana Volosozhar<div>(Russian Pair Skater)</div></div></div><div><span>16</span><span>133</span></div><div><div><b>Birthdate: </b>May 22, 1986</div><div><b>Sun Sign: </b>Gemini</div><div><b>Birthplace: </b>Dnipro, Ukraine</div><div><b>Height: </b>5\'3" (160 cm)</div></div></article19><article20><div><div>\xa021\xa0</div><div>Daria Usacheva<div>(Russian Figure Skater)</div></div></div><div><span>19</span><span>124</span></div><div><div><b>Birthdate: </b>May 22, 2006</div><div><b>Sun Sign: </b>Gemini</div><div><b>Birthplace: </b>Khabarovsk, Russia</div><div><b>Height: </b>5\'1" (155 cm)</div></div></article20><article21><div><div>\xa022\xa0</div><div>Victoria Sinitsina<div>(Russian\xa0Ice Dancer and Olympic Gold Medalist)</div></div></div><div><span>18</span><span>25</span></div><div><div><b>Birthdate: </b>April 29, 1995</div><div><b>Sun Sign: </b>Taurus</div><div><b>Birthplace: </b>Moscow, Russia</div><p>Victoria Sinitsina is a Russian ice dancer who won the gold medal at the 2021 <em>World Figure Skating Championships</em> along with her skating partner\xa0Nikita Katsalapov. A successful ice dancer, Sinitsina also won gold medals at the 2020 <em>European Championships </em>and 2012 <em>World Junior Championships.</em></p></div></article21><article22><div><div>\xa023\xa0</div><div>Elena Berezhnaya<div>(Former Russian Pair Skater and 1998 and 1999 World Champion)</div></div></div><div><span>17</span><span>130</span></div><div><div><b>Birthdate: </b>October 11, 1977</div><div><b>Sun Sign: </b>Libra</div><div><b>Birthplace: </b>Nevinnomyssk, Russia</div><div><b>Height: </b>5\'1" (155 cm)</div></div></article22><article23><div><div>\xa024\xa0</div><div>Maxim Trankov<div>(Russian\xa0Pair Skater and Olympic Gold Medalist)</div></div></div><div><span>19</span><span>140</span></div><div><div><b>Birthdate: </b>October 7, 1983</div><div><b>Sun Sign: </b>Libra</div><div><b>Birthplace: </b>Perm, Russia</div></div></article23><article24><div><div>\xa025\xa0</div><div>Maria Sotskova<div>(Retired Russian Figure Skater)</div></div></div><div><span>15</span><span>131</span></div><div><div><b>Birthdate: </b>April 12, 2000</div><div><b>Sun Sign: </b>Aries</div><div><b>Birthplace: </b>Reutov, Russia</div><div><b>Height: </b>5\'8" (173 cm)</div></div></article24><article25><div><div>\xa026\xa0</div><div>Fedor Andreev<div>(Canadian Figure Skater and Ice Dancer)</div></div></div><div><span>14</span><span>131</span></div><div><div><b>Birthdate: </b>March 2, 1982</div><div><b>Sun Sign: </b>Pisces</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Height: </b>6\'1" (185 cm)</div></div></article25><article26><div><div>\xa027\xa0</div><div>Ivan Bukin<div>(Russian\xa0Ice Dancer)</div></div></div><div><span>22</span><span>26</span></div><div><div><b>Birthdate: </b>September 16, 1993</div><div><b>Sun Sign: </b>Virgo</div><div><b>Birthplace: </b>Moscow, Russia</div><p>Ivan Bukin is a Russian ice dancer who has won three bronze medals and a silver medal at <em>European Championships </em>along with Alexandra Stepanova. He also won the 2021 Russian national championship. Overall, Bukin and his skating partner Stepanova have won three <em>Finlandia Trophy</em> titles and seven <em>Grand Prix</em> series medals.</p></div></article26><article27><div><div>\xa028\xa0</div><div>Oksana Grishuk<div>(Russian Former Competitive Ice Dancer and Olympic Gold Medalist)</div></div></div><div><span>12</span><span>130</span></div><div><div><b>Birthdate: </b>March 17, 1971</div><div><b>Sun Sign: </b>Pisces</div><div><b>Birthplace: </b>Odesa, Ukraine</div><p>Best known for her performances with partner Evgeni Platov, competitive ice dancer Oksana Grishuk began figure skating at age 4. She and Platov are three-time European champions and four-time world champions. The duo also has also won two <strong>Olympic </strong>golds. She is now a coach and has appeared on several TV shows.</p></div></article27><article28><div><div>\xa029\xa0</div><div>Mikhail Kolyada<div>(Russian Figure Skater)</div></div></div><div><span>151</span><span>266</span></div><div><div><b>Birthdate: </b>February 18, 1995</div><div><b>Sun Sign: </b>Aquarius</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><div><b>Height: </b>5\'6" (168 cm)</div></div></article28><article29><div><div>\xa030\xa0</div><div>Sofia Samodurova<div>(Russian Retired Figure Skater)</div></div></div><div><span>15</span><span>126</span></div><div><div><b>Birthdate: </b>July 30, 2002</div><div><b>Sun Sign: </b>Leo</div><div><b>Birthplace: </b>Krasnoyarsk, Russia</div><div><b>Height: </b>5\'3" (160 cm)</div></div></article29><article30><div><div>\xa031\xa0</div><div>Maria Petrova<div>(Russian\xa0Pair Skater)</div></div></div><div><span>14</span><span>132</span></div><div><div><b>Birthdate: </b>November 29, 1977</div><div><b>Sun Sign: </b>Sagittarius</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><div><b>Height: </b>5\'0" (152 cm)</div></div></article30><article31><div><div>\xa032\xa0</div><div>Evgenia Tarasova<div>(Russian Pair Skater)</div></div></div><div><span>14</span><span>129</span></div><div><div><b>Birthdate: </b>December 17, 1994</div><div><b>Sun Sign: </b>Sagittarius</div><div><b>Birthplace: </b>Kazan, Russia</div><div><b>Height: </b>5\'3" (160 cm)</div></div></article31><article32><div><div>\xa033\xa0</div><div>Anastasia Mishina<div>(Russian Pair Skater)</div></div></div><div><span>11</span><span>7</span></div><div><div><b>Birthdate: </b>April 24, 2001</div><div><b>Sun Sign: </b>Taurus</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><div><b>Height: </b>5\'2" (157 cm)</div></div></article32><article33><div><div>\xa034\xa0</div><div>Tatiana Totmianina<div>(Russian Former Competitive\xa0Pair Skater and Olympic Gold Medalist)</div></div></div><div><span>12</span><span>130</span></div><div><div><b>Birthdate: </b>November 2, 1981</div><div><b>Sun Sign: </b>Scorpio</div><div><b>Birthplace: </b>Perm, Russia</div><div><b>Height: </b>5\'3" (160 cm)</div></div></article33><article35><div><div>\xa036\xa0</div><div>Tatiana Tarasova<div>(Russian Figure Skating Coach)</div></div></div><div><span>14</span><span>325</span></div><div><div><b>Birthdate: </b>February 13, 1947</div><div><b>Sun Sign: </b>Aquarius</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Height: </b>5\'4" (163 cm)</div></div></article35><article36><div><div>\xa037\xa0</div><div>Ludmila Belousova<div>(Russian\xa0Pair Skater\xa0and Two-Time Olympic Gold Medalist)</div></div></div><div><span>9</span><span>128</span></div><div><div><b>Birthdate: </b>November 22, 1935</div><div><b>Sun Sign: </b>Sagittarius</div><div><b>Birthplace: </b>Ulyanovsk, Russia</div><div><b>Died: </b>September 26, 2017</div><div><b>Height: </b>5\'3" (160 cm)</div></div></article36><article37><div><div>\xa038\xa0</div><div>Alexandra Stepanova<div>(Russian Ice Dancer and Figure Skater)</div></div></div><div><span>11</span><span>13</span></div><div><div><b>Birthdate: </b>August 19, 1995</div><div><b>Sun Sign: </b>Leo</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><p>Alexandra Stepanova is a Russian ice dancer who has won three bronze medals and a silver medal at <em>European Championships </em>with Ivan Bukin. She also won the 2018<em> Grand Prix of Helsinki</em> championship as well as the 2021 Russian national championship. Overall, Stepanova and her skating partner Bukin have won three <em>Finlandia Trophy</em> titles and seven <em>Grand Prix</em> series medals.</p></div></article37><article38><div><div>\xa039\xa0</div><div>Stanislava Konstantinova<div>(Russian Figure Skater)</div></div></div><div><span>14</span><span>122</span></div><div><div><b>Birthdate: </b>July 14, 2000</div><div><b>Sun Sign: </b>Cancer</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><div><b>Height: </b>5\'6" (168 cm)</div></div></article38><article40><div><div>\xa041\xa0</div><div>Alexei Mishin<div>(Russian Figure Skating Coach and Former Pair Skater)</div></div></div><div><span>144</span><span>9</span></div><div><div><b>Birthdate: </b>March 8, 1941</div><div><b>Sun Sign: </b>Pisces</div><div><b>Birthplace: </b>Sevastopol, Russia</div></div></article40><article41><div><div>\xa042\xa0</div><div>Dmitri Aliev<div>(Russian Figure Skater)</div></div></div><div><span>15</span><span>101</span></div><div><div><b>Birthdate: </b>June 1, 1999</div><div><b>Sun Sign: </b>Gemini</div><div><b>Birthplace: </b>Ukhta, Russia</div><div><b>Height: </b>5\'8" (173 cm)</div></div></article41><article43><div><div>\xa044\xa0</div><div>Maya Khromykh<div>(Russian Figure Skater)</div></div></div><div><span>15</span><span>12</span></div><div><div><b>Birthdate: </b>May 25, 2006</div><div><b>Sun Sign: </b>Gemini</div><div><b>Birthplace: </b>Nizhny Tagil, Russia</div><div><b>Height: </b>5\'5" (165 cm)</div></div></article43><article44><div><div>\xa045\xa0</div><div>Alexander Zhulin<div>(Russian Ice Dancer, Figure Skater and Coach)</div></div></div><div><span>24</span><span>9</span></div><div><div><b>Birthdate: </b>July 20, 1963</div><div><b>Sun Sign: </b>Cancer</div><div><b>Birthplace: </b>Moscow, Russia</div><p>Currently a popular ice dancing coach, Alexander Zhulin was also a renowned figure skater. Paired with Maya Usova, he not only won bronze and silver in the Olympics, but also several other competitions including the World and the European Championships. Retired in 1994, he now serves as a coach and choreographer. He has also\xa0appeared\xa0in movies like <em>Muzhchina v moey golove.</em></p></div></article44><article45><div><div>\xa046\xa0</div><div>Anton Sikharulidze<div>(Russian Former Pair Skater)</div></div></div><div><span>8</span><span>3</span></div><div><div><b>Birthdate: </b>October 25, 1976</div><div><b>Sun Sign: </b>Scorpio</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><div><b>Height: </b>6\'0" (183 cm)</div></div></article45><article46><div><div>\xa047\xa0</div><div>Anastasia Tarakanova<div>(Russian Retired Figure Skater)</div></div></div><div><span>14</span><span>22</span></div><div><div><b>Birthdate: </b>April 14, 2004</div><div><b>Sun Sign: </b>Aries</div><div><b>Birthplace: </b>Volgograd, Russia</div><div><b>Height: </b>5\'1" (155 cm)</div></div></article46><article47><div><div>\xa048\xa0</div><div>Kira Ivanova<div>(Russian Figure Skater)</div></div></div><div><span>9</span><span>16</span></div><div><div><b>Birthdate: </b>January 10, 1963</div><div><b>Sun Sign: </b>Capricorn</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Died: </b>December 18, 2001</div><div><b>Height: </b>5\'3" (160 cm)</div></div></article47><article48><div><div>\xa049\xa0</div><div>Nikolai Morozov<div>(Russian Former Competitive Ice Dancer, Figure Skating Coach and Choreographer)</div></div></div><div><span>14</span><span>148</span></div><div><div><b>Birthdate: </b>December 17, 1975</div><div><b>Sun Sign: </b>Sagittarius</div><div><b>Birthplace: </b>Moscow, Russian SFSR, Soviet Union</div><p>Nikolai Morozov is a Russian retired ice dancer, choreographer, and figure skating coach. Morozov represented Belarus at the 1998 <em>Winter Olympics</em> and <em>World Championships.</em> He is credited with coaching Japanese figure skater Shizuka Arakawa at the 2006 <em>Olympics </em>where she won the gold medal. Nikolai Morozov also coached Miki Ando to a couple of world titles.</p></div></article48><article49><div><div>\xa050\xa0</div><div>Tamara Moskvina<div>(Russian Pair Skating Coach and Former Competitive Skater)</div></div></div><div><span>8</span><span>2</span></div><div><div><b>Birthdate: </b>June 26, 1941</div><div><b>Sun Sign: </b>Cancer</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><div><b>Height: </b>4\'10" (147 cm)</div></div></article49>\n</div1>\n</div0>\n</html8>\n<html9>\n<title>‘Beautiful’ Russian pairs figure skaters strike silver at Olympics — RT Sport News</title>\n<div><h1>\n        ‘Beautiful’ Russian pairs figure skaters strike silver at Olympics\n    </h1><p0>Russian pairs figure skaters Evgenia Tarasova and Vladimir Morozov have been hailed for a<em> "beautiful"</em> showing at the Winter Olympics as the duo who are part of coach\xa0Eteri Tutberidze\'s group came within a\xa0whisker of gold at the Beijing Games.</p0>\n</div>\n</html9>\n<html10><title>Russian figure skater Sofya Akatyeva on quads, Beijing 2022 and her future</title>\n<section0><section0><section0><p>Russia\'s newest <a>figure skating</a> rising star, <strong>Sofya Akatyeva</strong>, was thrust into the limelight after becoming the first female skate to land a <a>triple Axel alongside two quadruple jumps</a> in a single program.</p>\n<p>Akatyeva achieved the feat last week at the <strong>Junior Cup of Russia</strong> stop in Kazan, receiving a score <strong>higher than</strong> that of European champion <strong>Alena Kostornaia</strong>\'s in the <a>senior event</a>. She landed two <strong>quadruple toe loops</strong> (one in combination) in addition to the <strong>triple Axel</strong>.</p>\n<p>Speaking exclusively to Olympic Channel a few days after her historic skate, she frankly stated: <strong>"I love to jump quads"</strong>. The <strong>Eteri Tutberidze</strong>-trained athlete, who <strong>aged 13</strong> remains ineligible for senior international competition until the 2023/24 season, has her eyes set on big things.</p>\n<p>There is one more stop on the Cup of Russia circuit in Moscow, with the best performers qualifying for the senior and junior Russian Nationals.</p>\n<em>The interview below has been translated from Russian and edited for clarity and length.</em></section0>\n<p0><strong>Sofya Akatyeva (SA):</strong> Yesterday in the free program, the first half I skated quite well and in the second half I made very rough mistakes. I think this is because I did not work enough in training. That\'s for sure. I will work more in the second half at the next competition, I hope to skate well.</p0>\n<h2>Akatyeva on herself and figure skating</h2>\n<p3><strong>SA:</strong> In my training group I have <a>Alina Zagitova</a> and Anna Shcherbakova, their results motivate me to work. Their skating is so amazing, Anya (Shcherbakova) does such difficult jumps. It motivates me very much.</p3>\n</section0><div><h2>Related content</h2><h3>Packed women’s field highlights figure skating at Rostelecom Cup</h3><h3>Back injury sidelines Evgenia Medvedeva on eve of Rostelecom Cup</h3><h3>Is Sofya Akatyeva Russia\'s next figure skating superstar?</h3><h3>Zagitova, Fernández thrill in Moscow exhibition as fans return to figure skating</h3></div></section0></html10>\n<html11><title>Famous Figure Skaters from Russia | List of Top Russian Figure Skaters</title><div><div1><div><div><span>COLLECTION</span><span>58 LISTS</span></div><a>To Russia, With Love</a></div><ul><span>Must-See in Russia</span><span>Must-See in Moscow</span><span>Beers</span><span>Tennis Players</span><span>Actors</span><span>Soccer Players</span><span>Pop Artists</span><span>Authors</span><span>Pianists</span><span>Boxers</span><span>Vladamir Lenin Quotes</span><span>TV Channels</span><span>Bands</span><span>Books About Stalin</span><span>Movies</span><span>Politicians</span><span>Female Athletes</span><span>Basketball Players</span><span> Violinists</span><span>Artists</span><span>Hockey Players</span><span>Photographers</span><span>Gymnasts</span><span>Philosophers</span><span>Olympic Athletes</span><span>Guitar Players</span><span>Film Directors</span><span>Writers</span><span>Mathematicians</span><span>Conductors</span><span>Playwrights</span><span>Psychologists</span><span>Film Actors</span><span>Athletes</span><span>Journalists</span><span>Volleyball Players</span><span>Astronauts</span><span>Famous People</span><span>Architects</span><span>Russian Olympic Athletes</span><span>Best Russians</span><span>Scientists</span><span>Chemists</span><span>Physicists</span><span>Figure Skaters</span><span>Voice Actors</span><span>Physicians</span><span>Painters</span><span>Engineers</span><span>Guitarists</span><span>Composers</span><span>TV Producers</span><span>Poets</span><span>Track and Field Athletes</span><span>Cross-Country Skiers</span><span>Screenwriters</span><span>Animators</span><span>Novelists</span></ul></div1><article><div0><a>People</a><div><span>Famous Figure Skaters from Russia</span><div><a>Reference</a><div><span>Updated June 15, 2019</span><span> 18 items</span></div></div></div></div0><div1><p>List of notable or famous figure skaters from Russia, with bios and photos, including the <a>top figure skaters</a> <a>born in Russia</a> and even some popular figure skaters who immigrated to Russia. If you\'re trying to find out the names of famous Russian figure skaters then this list is the perfect resource for you. These figure skaters are among the most prominent in their field, and information about each well-known figure skater from Russia is included when available. </p><p>The list you\'re viewing is made up of people like Jana Khokhlova and Maxim Shabalin. </p><p>This historic figure skaters from Russia list can help answer the questions "Who are some Russian figure skaters of note?" and "Who are the <a>most famous figure skaters</a> from Russia?" These prominent figure skaters of Russia may or may not be currently alive, but what they all have in common is that they\'re all respected Russian figure skaters.</p>Use this list of renowned Russian figure skaters to discover some new figure skaters that you aren\'t familiar with. Don\'t forget to share this list by clicking one of the social media icons at the top or bottom of the page.\xa0</div1><ul><li0><li>Photo: <ul><a>Metaweb (FB)</a><a>CC-BY-SA</a></ul></li><a>Alena Leonova</a><span>Alena Igorevna Leonova (Russian: Алёна Игоревна Леонова; born 23 November 1990) is a Russian figure skater. She is the 2012 World silver medalist, the 2011 Grand Prix Final bronze medalist, the 2009 World Junior champion, and a three-time (2010–2012) Russian national medalist. She is also the 2014–15 ISU Challenger Series runner-up.</span></li0><li1><li>Photo: <ul><a>Metaweb (FB)</a><a>GNU Free Documentation License</a></ul></li><a>Alexander Enbert</a><span>Alexander Yuryevich Enbert (Russian: Александр Юрьевич Энберт; born 17 April 1989) is a Russian pair skater. With Natalya Zabiyako, he is the 2019 World bronze medalist, 2018 European bronze medalist, 2018 Grand Prix of Helsinki champion, 2018 NHK Trophy champion, 2016 Rostelecom Cup silver medalist, and three-time Russian national medalist.\nFrom 2010 to 2014, he competed with Katarina Gerboldt. The pair placed fourth at the 2011 European Championships.</span></li1><li2><li>Photo: <ul><a>Metaweb (FB)</a><li>Public domain</li></ul></li><a>Alexander Smirnov</a><span>Alexander Viktorovich Smirnov (Russian: Александр Викторович Смирнов, born 11 October 1984) is a Russian pair skater. Smirnov teamed up with Yuko Kavaguti in May 2006. They are two-time European champions (2010, 2015), two-time World bronze medalists (2009, 2010), two-time ISU Grand Prix Final bronze medalists (11–12 and 15–16), and three-time Russian national champions (2008–2010). In 2015, they became the first pair in history to complete two quadruple throw jumps in one program and the first to land a quadruple throw loop</span></li2><li3><li>Photo: <ul><a>Metaweb (FB)</a><a>CC-BY-SA-2.5</a></ul></li><a>Andrei Lutai</a><span>Andrei Vladimirovich Lutai (Russian: Андрей Владимирович Лутай; born 24 July 1986) is a Russian former competitive figure skater. He is the 2006 Karl Schäfer Memorial champion, 2006 International Cup of Nice champion, and a three-time Russian national medalist (2007 &amp; 2008 silver; 2009 bronze). He placed as high as 5th at the European Championships (2007) and 10th at the World Championships (2009).</span></li3><li5><li>Photo: <ul><a>Metaweb (FB)</a><a>CC-BY-SA-2.5</a></ul></li><a>Igor Lukanin</a><span>Igor Lukanin (born 3 February 1976) is a former competitive ice dancer who competed internationally for Azerbaijan with Kristin Fraser.\nHe previously competed for Germany with Ksenia Smetanenko. He competed with Jenny Dahlen for two years, representing Azerbaijan. He teamed up with Kristin Fraser in 2000. They are four-time Azerbaijani national champions and have competed at the 2002 Winter Olympics and the 2006 Winter Olympics.\nLukanin was married to Daria Timoshenko from 2000 to 2005. He married Kristin Fraser on December 31, 2010.\nLukanin was the flag bearer for Azerbaijan at the 2006 Winter Olympics.</span></li5></ul></article></div></html11>\n<html12>\n<title>List of Russian pair skaters - FamousFix List</title>\n<div>\n<div0>\n<div>\n<h1>Russian pair skaters</h1>\n<div>The list "Russian pair skaters" has been viewed 262 times.</div>This list has <i>2 sub-lists</i> and <i>141 members</i>. See also <a>Russian figure skaters</a>, <a>Pair skaters</a>\n</div>\n<div>\r\nFLAG\r\n</div>\n<div>204 views · <span>\r\nLike \n</span></div>\n</div0>\n<div1>\n<a>Russian female pair skaters</a>\r\n\t\t\t\t\t\t\t\t116 T\r\n\t\t\t\t\t\t\t</div1>\n<div2>\n<a>Russian male pair skaters</a>\r\n\t\t\t\t\t\t\t\t102 T\r\n\t\t\t\t\t\t\t</div2>\n<ul>\n<div0>\n<div>\n<a>Ekaterina Gordeeva</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#1</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>36 </span>\n<span>3 </span>\n<span>2 </span>\n</div>\n<div>Ekaterina "Katia" Alexandrovna Gordeeva (Russian: Екатерина Александровна Гордеева; born 28 May 1971) is a Russian former figure skater. Together with her husband, the late Sergei Grinkov, she was the 1988 and 1994 Olympic Champion and four-time World Champion (1986, 1987, 1989, 1990) in pair skating. After Grinkov\'s death, Gordeeva continued performing as a singles skater.</div>\n<ul>\n<li>\n<a>Figure skaters from Moscow</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>232T</span>\n</li>\n<li>\n<a>Competitors at the 1990 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>351T</span>\n</li>\n<li>\n<a>Russian State University of Physical Education, Sport, Youth and Tourism alumni</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>49T</span>\n</li>\n</ul>\n</div0>\n<div1>\n<div>\n<a>Irina Rodnina</a>\n<span>Russian politician and figure skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#2</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>49 </span>\n</div>\n<div>Irina Konstantinovna Rodnina (Russian: Ирина Константиновна Роднина, born 12 September 1949) is a Russian politician and figure skater, who is the only pair skater to win 10 successive World Championships (1969–78) and three successive Olympic gold medals (1972, 1976, 1980). She was elected to the State Duma in the 2007 legislative election as a member of President Vladimir Putin\'s United Russia party. As a figure skater, she initially competed with Alexei Ulanov and later teamed up with Alexander Zaitsev. She is the first pair skater to win the Olympic title with two different partners, followed only by Artur Dmitriev.</div>\n<ul>\n<li>\n<a>Eighth convocation members of the State Duma (Russian Federation)</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>453T</span>\n</li>\n<li>\n<a>Recipients of the Order of the Red Banner of Labour</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>1,697T</span>\n</li>\n<li>\n<a>Figure skaters from Moscow</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>272T</span>\n</li>\n</ul>\n</div1>\n<div2>\n<div>\n<a>Yelena Berezhnaya</a>\n<span>Pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#3</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n</div>\n<div>Elena Viktorovna Berezhnaya (Russian: Елена Викторовна Бережная, born 11 October 1977) is a Russian former pair skater. With partner Anton Sikharulidze, she is the 1998 and 1999 World champion, 1998 Olympic silver medalist and 2002 Olympic champion.</div>\n<ul>\n<li>\n<a>Competitors at the 1998 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>314T</span>\n</li>\n<li>\n<a>Competitors at the 2001 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>345T</span>\n</li>\n<li>\n<a>Sportspeople from Stavropol Krai</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>47T</span>\n</li>\n</ul>\n</div2>\n<div4>\n<div>\n<a>Sergei Grinkov</a>\n<span>Russian pair skater (1967–1995)</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#5</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n</div>\n<div>Sergei Mikhailovich Grinkov (Russian: Серге́й Миха́йлович Гринько́в; 4 February 1967 — 20 November 1995) was a Russian pair skater. Together with partner and wife Ekaterina Gordeeva, he was the 1988 and 1994 Olympic Champion and a four-time World Champion (1986, 1987, 1989, 1990).</div>\n<ul>\n<li>\n<a>Figure skaters from Moscow</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>232T</span>\n</li>\n<li>\n<a>Competitors at the 1990 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>351T</span>\n</li>\n<li>\n<a>Russian State University of Physical Education, Sport, Youth and Tourism alumni</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>49T</span>\n</li>\n</ul>\n</div4>\n<div5>\n<div>\n<a>Vasilisa Davankova</a>\n<span>Russian Ice dancer</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#6</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n</div>\n<div>Vasilisa Alexandrovna Davankova (Russian: Василиса Александровна Даванкова; born 2 May 1998) is a Russian YouTuber and a former Ice dancer and pair skater. With former partner Andrei Deputat, she was the 2012 World Junior bronze medalist, the 2012–13 JGP Final silver medalist, and the 2012 Russian Junior champion. She then briefly teamed up with Alexander Enbert but ended their partnership in 2015.</div>\n<ul>\n<li>\n<a>Figure skaters from Moscow</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>241T</span>\n</li>\n<li>\n<a>Competitors at the 2017 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>220T</span>\n</li>\n<li>\n<a>Russian YouTubers</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>64T</span>\n</li>\n</ul>\n</div5>\n<div6>\n<div>\n<a>Ksenia Stolbova</a>\n<span>Figure skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#7</b> ·\r\n\t\t\t\t\t\t\t</div>\n<div>Ksenia Andreyevna Stolbova (Russian: Ксе́ния Андре́евна Столбо́ва; born 7 February 1992) is a retired Russian pair skater who skated with Andrei Novoselov and Fedor Klimov. She is the 2014 Olympic silver medalist, the 2014 Olympic champion in the team event, the 2014 World silver medalist, a three-time European medalist (2012 bronze, 2014, 2015 silver), the 2015–16 Grand Prix Final champion, the 2013 Winter Universiade champion, a two-time World Junior medalist (2010 bronze, 2011 silver), and a three-time Russian national champion (2014, 2015, 2017). Stolbova announced her retirement from competitions on 12 February 2020.</div>\n<ul>\n<li>\n<a>Figure skaters from Saint Petersburg</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>99T</span>\n</li>\n<li>\n<a>Competitors at the 2013 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>254T</span>\n</li>\n<li>\n<a>Universiade gold medalists for Russia</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>495T</span>\n</li>\n</ul>\n</div6>\n<div7>\n<div>\n<a>Tatiana Volosozhar</a>\n<span>Russian-Ukrainian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#8</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>1 </span>\n</div>\n<div>Tatiana Andreyеvna Volosozhar (Russian: Татья́на Андре́евна Волосожа́р, Ukrainian: Тетяна Андріївна Волосожар; born 22 May 1986) is a Ukrainian-born Russian pair skater. With Maxim Trankov, she is the two-time 2014 Olympic champion in the pairs and in team events, the 2013 World champion, a four-time (2012, 2013, 2014 and 2016) European champion, the 2012 Grand Prix Final champion, and a three-time (2011, 2013 and 2016) Russian national champion. They have also won six events on the Grand Prix series.</div>\n<ul>\n<li>\n<a>Universiade silver medalists for Ukraine</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>146T</span>\n</li>\n<li>\n<a>Season\'s world number one figure skaters</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>32T</span>\n</li>\n<li>\n<a>Season-end world number one figure skaters</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>57T</span>\n</li>\n</ul>\n</div7>\n<div8>\n<div>\n<a>Maxim Trankov</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#9</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>1 </span>\n</div>\n<div>Maxim Leonidovich Trankov (Russian: Макси́м Леони́дович Транько́в, born 7 October 1983) is a Russian pair skater. With Tatiana Volosozhar, he is a two-time  2014 Olympic champion in the pairs and in team events, the 2013 World champion, a four-time (2012, 2013, 2014, 2016) European champion, the 2012 Grand Prix Final champion, and a three-time (2011, 2013, 2016) Russian national champion. They have also won six events on the Grand Prix series.</div>\n<ul>\n<li>\n<a>Competitors at the 2005 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>131T</span>\n</li>\n<li>\n<a>Universiade bronze medalists for Russia</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>263T</span>\n</li>\n<li>\n<a>Season\'s world number one figure skaters</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>31T</span>\n</li>\n</ul>\n</div8>\n<div9>\n<div>\n<a>Yelena Bechke</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#10</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>1 </span>\n</div>\n<div>Elena Yurievna Bechke (Russian: Елена Юрьевна Бечке; born 7 January 1966) is a Russian former pair skater. She competed for the Soviet Union, the Commonwealth of Independent States, and the Unified Team. With partner and then-husband Denis Petrov, she is the 1992 Olympic silver medalist, the 1989 World bronze medalist, and the 1991 and 1992 European silver medalist.</div>\n<ul>\n<li>\n<a>Figure skaters from Saint Petersburg</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>116T</span>\n</li>\n<li>\n<a>Competitors at the 1990 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>355T</span>\n</li>\n<li>\n<a>Competitors at the 1987 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>32T</span>\n</li>\n</ul>\n</div9>\n</ul>\n</div>\n</html12>\n<div0>\n<div><strong0>Ekaterina Gordeeva is a professional figure skater from Russia, currently living in Los Angeles, with ethereal beauty and soulful athleticism, a two-time <em>Olympic Champion</em>\xa0and a four-time <em>World Champion\xa0</em>in pair skating. Ekaterina and her daughter Daria have also appeared in a few documentaries and movies.</strong0>\n<p0>Her beloved lover and husband, <strong>Sergei Grinkov</strong>, was an unbeatable pair with his wife in the Olympics pair-skating.</p0>\n<strong1>\xa0EKATERINA GORDEEVA Olympic skating champion Ekaterina Gordeeva poses for a portrait (Credit: Photo by Susan Sterner/AP/Shutterstock )</strong1>\n<p3>Ekaterina started skating when she was only four and finally retired <em>37 years</em> later. Her Father was a professional dancer and directed her toward figure skating.</p3>\n<p5>The champion figure skater has married <strong>thrice</strong> and has a daughter from each marriage. She welcomed her first child with her first husband almost three years before he died.</p5>\n<span9>Ekaterina Gordeeva | Professional Figure Skating Career</span9>\n<span11>The two-time Olympic Champion started her figure skating career early. At four, she joined the Children and Youth Sports School in a significant Russian club <em>CSKA Moscow.</em> </span11>\n<span12>She was so small, so she wore multiple socks to fit her skates. Initially, she was paired with her first husband, <strong>Sergei Grinkov</strong>.</span12>\n<strong3>Ekaterina Gordeeva With Her Dance Partner</strong3>\n<p14>However, she disagreed, and instead of changing her partner, she changed her coach. The duo has been partners since Katina was <em>11 years old</em>\xa0and Sergei was <em>15.</em></p14>\n<span14>Later, in <em>1985</em> and <em>1986,</em> the pair won annual figure skating competitions like the <em>World</em> <em>Junior Figure Skating Championships</em> and the <em>World Figure Skating Championships</em>. Moreover, they held the title in <em>1987,\xa01989,</em> and <em>1990.</em> </span14>\n<strong4>The Four-Time World Champion Figure Skating</strong4>\n<span20>The figure skater started her solo career in <em>1996</em> after Sergei passed away suddenly on November 20, 1995, due to a heart attack. </span20>\n<span29>Ekaterina married her skating partner and first love, <strong>Sergei Grinkov</strong>. The pair met when Gordeeva was <em>11</em>\xa0and Grinkov was <em>15</em> through their passion for figure skating.</span29>\n<span36>Gordeeva met her second husband, <strong>Ilia Kulik,</strong> through figure skating too. They started their romance during the <em>1999-2000</em> season of Stars on Ice and even performed a romantic duet. </span36>\n<strong5>The Figure Skater With Ex-Husband Ilia Kulik And Daughter Elizaveta Kulik</strong5>\n<div>Ekaterina Gordeeva is married to Canadian figure skater <strong>David Pelletier</strong>. The couple got married on <strong>July 25,</strong><em><strong>\xa02020</strong>.</em></div>\n<span44>The exceptional figure skater has built a whopping net worth after almost <em>22 years</em> of a professional career. </span44>\n<h31>Who was Ekaterina Gordeeva’s partner at Battle of the Blades?</h31>\n<p32>Yes, Ekaterina Gordeeva was paired with hockey player Valeri Bure in the figure skating reality show “<em>Battle of the Blades” </em>in 2010.</p32>\n</div>\n<article2>\n<footer>\n<cite>Nancy Miller</cite>\n<time>\nFebruary 20, 2021 At 11:55 am</time>\n</footer>\n<p>This is an interesting article with some great facts and photos. HOWEVER, the very first photo (in the dark red box) is not a picture of Katia. I don’t know how a photo of a different woman got into this article, but it definitely is some other woman. I have been following Katia’s career for many years and I know her distinctive features. The woman in the red box is very attractive, and about the same age, but she is NOT Ekaterina Gordeeva.</p>\n<a>Reply</a>\n</article2>\n</div0>\n<html14>\n<title>15 Most Famous Russian Figure Skaters of All Time | Stadium Talk</title>\n<article>\n<header><h1>15 Most Famous Russian Figure Skaters of All Time</h1><div>Russian figure skaters have a reputation for over-the-top training practices, but their results are hard to argue with.</div></header><section0><h2>The Most Famous Russian Figure Skaters of All Time</h2><p>Of all the Olympic sports, Russia is known for dominating one in particular: figure skating. Russian figure skaters are trained to operate like athletic machines before they\'re old enough to attend kindergarten.\xa0</p><p>While their methods are questionable, Russia\'s churned out many memorable figure skating champions. Which ones <em>were</em> the most memorable? Here are the most popular to date.</p></section0><section1><h2>Sergei Ponomarenko</h2><p><strong>Date of birth:</strong> Oct. 6, 1960</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> Olympic Champion, three-time World Champion</p><p><strong>Bottom Line:</strong> Sergei Ponomarenko and his skating partner and wife Marina Klimova made waves during the 1980s and \'90s as ice dancers. They won four European titles in a row, and they had one of the longest careers at the time, spanning over a decade. Ponomarenko was one of the oldest skating champions, delaying retirement until he was in his early 30s.</p><p>Following retirement, he and Klimova were both inducted into the World Figure Skating Hall of Fame, and Ponomarenko remains an ISU technical specialist for Russia. Both of them now coach in San Jose, California, and one of their sons, Anthony Ponomarenko, is now a competitive ice dancer.\xa0</p></section1><section2><h2>Ilia Kulik</h2><p><strong>Date of birth:</strong> May 23, 1977</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> First figure skater to win Olympic gold with a quadruple jump</p><p><strong>Bottom Line:</strong> Ilia Kulik was 20 when he won a gold medal at the 1998 Winter Olympics in Nagano, Japan. He was only the second man in the sport\'s history to win gold on his first try. He started skating at 5 and was initially more on the clumsy side. But his coach noted Kulik\'s refusal to give up and encouraged him to continue lessons.</p><p>Kulik went on to win two Russian national championships, followed by a stunning Olympic debut with one of the most challenging programs ever successfully skated at that time. It included a quadruple toe loop and eight triples. Later in life, he became a distinguished choreographer, touring with both "Champions on Ice" and "Stars on Ice."\xa0</p></section2><section3><h2>Adelina Sotnikova</h2><p><strong>Date of birth:</strong> July 1, 1996</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> Four-time Russian National Champion</p><p><strong>Bottom Line:</strong> Adelina Sotnikova isn\'t the most famous Russian figure skater, but she had a loyal fanbase throughout the 2010s. She started skating when she was just 4 at a rink near her home in Moscow. Just eight years later, she won gold at the senior-level Russian Nationals.</p><p>She was too young to compete on the Junior Grand Prix circuit that season, but she continued working her way up, eventually winning Olympic gold in 2014. She had a string of lingering injuries, eventually retiring due to chronic pain.\xa0</p><strong>RELATED: Meet Ilia Malinin, the \'Quad God\' of Figure Skating</strong></section3><section4><h2>Alexei Yagudin</h2><p><strong>Date of birth:</strong> March 18, 1980</p><p><strong>Birthplace:</strong> Saint Petersburg, Russia</p><p><strong>Biggest achievement:</strong> Only skater to have achieved a Golden Slam</p><p><strong>Bottom Line:</strong> One of the most memorable Russian figure skaters of all time, Alexei Yagudin is the only skater to have won a Golden Slam – a victory in every major figure skating championship in the same season. This includes the Olympics, World Championships, European Championships, Grand Prix assignments and Grand Prix final. He was primed for success from the start, landing all his double jumps by age 10 and five triples before the age of 12.\xa0</p><p>He was awarded the Order of Merit by the Russian Federation in 2003 and joined the figure skating Hall of Fame in 2017. Since then, he\'s worked as an actor, TV host and sports commentator.</p></section4><section5><h2>Irina Slutskaya</h2><p><strong>Date of birth:</strong> Feb. 9, 1979</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> Two-time World Champion</p><p><strong>Bottom Line:</strong> Irina Slutskaya had a rocky road in figure skating, but her athleticism and innovation set her performances apart from the rest. After initial success in the sport, she hit a plateau in her skating career, failing to qualify for both the European and World championships. Instead of throwing in the towel, she continued training.\xa0</p><p>It\'s a good thing she did because she went on to beat Michelle Kwan at the 2000 Grand Prix Final, plus won a silver medal at the 2002 Winter Olympics in Salt Lake City, Utah. She actually tied for gold, but Sara Hughes of the U.S. took the gold in a tiebreaker free skate.</p><p>Slutskaya was known for her power on the ice, making history as the first woman to land a triple-Lutz-triple-loop combination. She was also known for her signature double Biellmann spin, which she made up herself.\xa0</p></section5><section6><h2>Alexandra Trusova</h2><p><strong>Date of birth:</strong> June 23, 2004</p><p><strong>Birthplace:</strong> Ryazan, Russia</p><p><strong>Biggest achievement:</strong> First female to land two quads in a free skate</p><p><strong>Bottom Line:</strong> Alexandra Trusova is one of Russia\'s favorite figure skaters at the moment. She was the Beijing 2022 Olympic silver medalist, and quads are her claim to fame. She\'s the first female to land the quad Lutz, flip and toe loop jumps in competition and was the first to land two, then three quads in a free skate.\xa0</p><p>She has three Grand Prix Champion titles under her belt, and she\'s expected to continue on her quad-packed career path at the next Olympic games.</p></section6><section8><h2>Anna Shcherbakova</h2><p><strong>Date of birth:</strong> March 28, 2004</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> First woman to land a quad Lutz in competition</p><p><strong>Bottom Line:</strong> Anna Scherbakova is still young, and she\'s one of the most promising Russian figure skaters in competition. She was the second figure skater to perform two quadruple Lutz jumps in one program, following American skater Nathan Chen. She\'s also the first to land a quad flip in combination with a triple.\xa0</p><p>She gets her inspiration from retired Japanese figure skater Mao Asada, and she loves caring for her pets in her limited free time, including a dog named Sandy, a street cat named Mafia and a Maine coon cat named Shiny.</p><p><strong>RELATED: 32 Best Male Figure Skaters of All Time, Ranked</strong>\xa0</p></section8><section9><h2>Evgeni Plushenko</h2><p><strong>Date of birth:</strong> Nov. 3, 1982</p><p><strong>Birthplace:</strong> Dzhamku, Russia</p><p><strong>Biggest achievement:</strong> Earned four Olympic medals</p><p><strong>Bottom Line:</strong> If you haven\'t heard the name Plushenko, you\'re probably not a figure skating fan. Plushenko was one of the biggest names in figure skating in the early 2000s, dominating the competition between 2003 and 2006. He was trying quads before it was popular, but his artistry was what made him so memorable.\xa0</p><p>Despite several injuries, including a back injury resulting in surgery, he continued competing all the way up until 2014. He was crowned European champion for the seventh time in 2012 and won a fourth Olympic medal at the Sochi winter games in 2014.</p></section9><section10><h2>Kamila Valieva</h2><p><strong>Date of birth:</strong> April 26, 2006</p><p><strong>Birthplace:</strong> Kazan, Russia</p><p><strong>Biggest achievement:</strong> First woman to land a quad at the Olympics\xa0</p><p><strong>Bottom Line:</strong> Kamila Valieva is one of five Russian figure skaters coached by Eteri Tutberidze at the same rink. That means she trains alongside Alexandra Trusova and Anna Shcherbakova. She started skating when she was only 3 and trains off the ice in ballet. She\'s best known for her artistry, inspired by her love of painting. Many of her programs are inspired by famous works of art.\xa0</p><p>In 2022, she became the center of an unfortunate doping scandal. She tested positive for a banned heart medication that\'s known to help improve cardiovascular endurance. She was denied her gold medal in the 2022 Russian Figure Skating Championships due to the positive drug test and is currently suspended.</p><p>Make no mistake, however. She was only 15 at the time of the scandal, and her exclusion from the sport is tragic. The adults, including her coaches and parents, are the ones to blame, not the teenager.\xa0</p></section10><section11><h2>Yulia Lipnitskaya</h2><p><strong>Date of birth:</strong> June 5, 1998</p><p><strong>Birthplace:</strong> Yekaterinburg, Russia</p><p><strong>Biggest achievement:</strong> 2014 European champion</p><p><strong>Bottom Line:</strong> Ready to be impressed? Yulia Lipnitskaya didn\'t fall a single time during the 2011-2012 season. Skating through multiple competitions without a single fall is almost unheard of. The Russian skater adores competition, and her intense training was almost entirely self-motivated.</p><p>She even competed with a twisted ankle and ended up winning first in the short program and third in the free skate. She was the youngest woman figure skater to win Olympic gold under modern rules, at only 15 years, 249 days, and she was the youngest gold medalist at the European Championships in the ladies\' singles category. Sadly, she retired in 2017 due to injuries and a battle with an eating disorder.</p><strong>RELATED: Why Don’t We Ever See Black Figure Skaters?</strong></section11><section12><h2>Alina Zagitova</h2><figcaption>\\</figcaption><p><strong>Date of birth:</strong> May 18, 2002</p><p><strong>Birthplace:\xa0</strong>Izhevsk, Russia</p><p><strong>Biggest achievement:\xa0</strong>2018 Olympic Champion, World Champion</p><p><strong>Bottom Line:</strong> Alina Zagitova was ranked as the best female skater in the world in the 2018-2019 season, with countless titles and medals under her name. She is the only Russian female figure skater to win gold at the Olympics, World Figure Skating Champtionships, European Championships and Grand Prix Final, and she\'s the first Muslim athlete to win an Olympic gold, a world title and a Super Slam.\xa0</p><p>She also broke the world record under the old system once as well as four times under the new system. She got an early start, beginning formal lessons when she was 4 at the rink her father worked at as a hockey coach. She was named female athlete of the year by the Ministry of Sport of Russia in 2018 and was listed as the top athlete in the sports category of Forbes Russia\'s Top 30 Under 30 list.\xa0</p></section12><section13><h2>Evgenia Medvedeva</h2><p><strong>Date of birth:</strong> Nov. 19, 1999</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> Only one of four women to achieve a Grand Slam</p><p><strong>Bottom Line:</strong> Evgenia Medvedeva is the first ladies\' singles skater to win Senior Worlds the year after winning Junior Worlds. In 2017, she also became the first female skater to win back-to-back world titles since Michelle Kwan did so in 2000 and 2001. And she\'s the first skater in all disciplines to complete two Grand Slams in a row.\xa0</p><p>Not impressed yet? Medvedeva set the world record score 13 times and is the first woman to pass the 80-point short program score, the 160-point free skating score, and both the 230 and 240-point total score. Now, she\'s studying at Moscow State University.\xa0</p><p>Fun fact: Medvedeva has lepidopterophobia, the fear of moths and butterflies. She also has two French bulldogs named Jerry and Tofu.</p></section13><section14><h2>Sergei Grinkov</h2><p><strong>Date of birth:</strong> Feb. 4, 1967</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> Two-time Olympic Champion, four-time World Champion in pair skating</p><p><strong>Bottom Line:</strong> Sergei Grinkov was a Russian pairs skater who hit the ice at the age of 5. He began training at the Children and Youth Sports School in Moscow, but he wasn\'t a strong solo skater. His coach opted to give him a shot in pairs skating, and he teamed up with 11-year-old skater Ekaterina Gordeeva when he was 14. They won the Junior World Championship in 1985, followed by a string of gold medals at the European Championships, World Championships and Olympic Games.\xa0</p><p>Tragically, Grinkov collapsed during practice in Lake Placid, New York, in 1995, dying of a massive heart attack. He was only 28, with a 4-year-old daughter whom he shared with partner and wife Ekaterina Gordeeva.\xa0</p></section14><section15><h2>Ekaterina Gordeeva</h2><p><strong>Date of birth:</strong> May 28, 1971</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> Two-time Olympic Champion, four-time World Champion in pair skating</p><p><strong>Bottom Line:</strong> Sergei Grinkov may not be around anymore, but his partner, Ekaterina Gordeeva, is still going strong. She started skating when she was only 4 and initially had to wear several pairs of socks because the smallest available pair of skates were still too big. When she teamed up with Grinkov, their chemistry was instant. Their coach once tried to get her to team up with a new partner after Grinkov was repeatedly late, but she was adamant that he was the only one for her. Instead of switching partners, they switched coaches.\xa0</p><p>The pair went on to win almost every competition they entered. Out of 31 competitions, they finished first 24 times and never placed lower than second. After Grinkov passed away, Gordeeva remarried another famous Russian figure skater on this list, Ilia Kulik. She has two daughters, both of whom skate.\xa0</p></section15>\n</article>\n</html14>\n<html15><title>The Greatest Russian Female Figure Skaters of All Time, Ranked</title><div><div1><div><div><span>COLLECTION</span><span>22 LISTS</span></div><a>Figure Skating</a><span>Lists about the beautiful, technically grueling sports of figure skating and ice dancing and the talented athletes who compete in them all around the world.</span></div><ul><span>The Best Movies About Figure Skating</span><span>The Greatest Male Figure Skaters</span><span>The Greatest Ice Dancers Ever</span><span>The Top Russian Female Figure Skaters</span><span>The Greatest Olympic Figure Skaters</span><span>The Top American Figure Skaters</span><span>Top Female Skaters Working Today</span><span>The Top American Women in History</span><span>The Top Russian Male Figure Skaters</span><span>The Top Russian Figure Skaters</span><span>The Top Skaters Working Today</span><span>The Greatest Women of All Time</span><span>The Top American Male Figure Skaters</span><span>The Top Current Figure Skating Pairs</span><span>The Greatest Ice Dancing Pairs Ever</span><span>The Best Retired Figure Skaters</span><span>The Greatest Men Working Today</span><span>Top Ice Dancing Pairs Working Today</span><span>The Greatest Figure Skating Pairs Ever</span><span>The Top American Ice Dancers</span><span>The Greatest Ice Dancers in the World</span><span>The Top Russian Ice Dancers</span></ul></div1><article><div1><span1>It\'s easy to underestimate how popular figure skating has been in Russia since the first skating rink opened in St. Petersburg around 1865? The first World Figure Skating Championships, in fact, were held in the city in 1896. Although only male skaters were allowed to participate, female figures skaters, especially Russian ones, were soon competing and winning medals for their countries in all types of figure skating events, including the Olympics. And as far as Olympic medals go, Russia has one of the highest medal counts in the sport for the category.</span1><p0>Of course, Russian female figure skaters aren’t known and celebrated only in Russia. Most fans of figure skating across the world are familiar with the names of some of the greatest ladies in Russian figure skating history. From Irina Slutskaya, who has medaled at the Olympics, World Figure Skating Championships, and <a>Russian Figure Skating Championships</a>, to Viktoria Volchkova, who medaled at the Russian Figure Skating Championships seven times, these amazing Russian women have had a worldwide impact on the sport and made Russia proud.</p0><p1>This list contains all of the greats in Russian ladies’ single skating history, whether they are an all-time ice dance legend or a young girl in the sport.\xa0Although many have left the sport, their greatest performances will be watched and studied by aspiring skaters and sports fans for years to come. These women have left a lasting legacy, giving them the honor of being known as the greatest female figure skaters from Russia.</p1></div1><div2><ul><li><div><div><div>1</div><div>242 VOTES</div></div><a>Ekaterina Gordeeva</a></div><li>Photo: <a>YouTube</a></li><div><span>Although Ekaterina Gordeeva is remembered for her pair work, she also competed in ladies’ single pair skating.</span><span>242 votes</span><div><div>More\xa0<a>Ekaterina Gordeeva</a></div><ul><li><span>#7<span> of 43 on</span></span><a>The Greatest Female Figure Skaters of All Time</a></li><li><span>#1<span> of 30 on</span></span><a>The Greatest Figure Skating Pairs of All Time</a></li><li><span>#1<span> of 37 on</span></span><a>The Greatest Russian Figure Skaters of All Time</a></li></ul></div></div></li><li><div><div><div>2</div><div>369 VOTES</div></div><a>Alina Zagitova</a></div><li>Photo: <a>YouTube</a></li><span>369 votes</span></li><li><div><div><div>3</div><div>276 VOTES</div></div><h2>Julia Lipnitskaia</h2></div><li>Photo: <a>YouTube</a></li><div><span>Julia Lipnitskaia placed first at the European Figure Skating Championships in 2014.</span><span>276 votes</span></div></li><li><div><div><div>4</div><div>219 VOTES</div></div><a>Irina Slutskaya</a></div><li>Photo: <a>YouTube</a></li><div><span>Irina Slutskaya competed (and medaled) at both the 2002 and 2006 Winter Olympic Games.</span><span>219 votes</span></div></li><li><div><div><div>5</div><div>165 VOTES</div></div><h2>Elizaveta Tuktamysheva</h2></div><li>Photo: <a>YouTube</a></li><div><span>Elizaveta Tuktamysheva is a medalist many times over; she won, for example, the gold medal at the Trophée Eric Bompard in 2011.</span><span>165 votes</span></div></li><li><div><div><div>6</div><div>112 VOTES</div></div><a>Maria Butyrskaya</a></div><li>Photo: <a>YouTube</a></li><div><span>Maria Butyrskaya won the gold medal at the 2002 European Figure Skating Championships at the age of 29, giving her the distinction of <a>being the oldest</a> ladies’ single figure skater to have done so.</span><span>112 votes</span></div></li></ul><button>load more</button></div2></article></div></html15>\n<html16>\n<title>The Most Famous Russian Figure Skater, Ranked - StrawPoll</title>\n<div>\n<div0>\n<div0>\n<h1>\n\t\t\t\tThe Most Famous Russian Figure Skater, Ranked\n\t\t\t</h1>\n<div2>\n\t\t\t\tFigure skating, with its blend of athleticism and artistry, has always captured the hearts of audiences across the globe. Narrowing down the most notable among Russian figure skaters can provide a meaningful snapshot of the sport\'s evolution and influence within this powerhouse nation. It helps to highlight key figures who have shaped the legacy and pushed the boundaries of what is achievable on ice.\nBy participating in the ranking process, users contribute to a collective recognition of talent and achievement, offering a unique perspective on who stands out in the world of figure skating. This dynamic ranking adjusts with each vote, reflecting the current consensus on the most impactful skaters. It\'s an engaging way to see how preferences and perceptions change over time and to appreciate the stars who have defined this sport.\n\t\t\t</div2>\n<section>\n<h2>Who Is the Most Famous Russian Figure Skater?</h2>\n<ol>\n<div0>\n<div>\n<div>\n\t\t\t\t\t\t1\n\t\t\t\t\t</div>\n<div>\n<span>48</span>\n<div>points</div>\n</div>\n</div>\n<div>\n<button>Source\n</button>\n<h3>\n\t\t\t\t\t\t\tEvgeni Plushenko\n\t\t\t\t\t\t</h3>\n<div>A retired figure skater and two-time Olympic gold medalist, known for his athleticism and artistry on the ice.</div>\n<ul>\n<li>Olympic medals: 4</li>\n<li>World Championships titles: 3</li>\n</ul>\n</div>\n</div0>\n<div1>\n<div>\n<div>\n\t\t\t\t\t\t2\n\t\t\t\t\t</div>\n<div>\n<span>15</span>\n<div>points</div>\n</div>\n</div>\n<div>\n<button>Source\n</button>\n<h3>\n\t\t\t\t\t\t\tAlina Zagitova\n\t\t\t\t\t\t</h3>\n<div>An Olympic gold medalist, Zagitova is known for her technical precision and became the first Russian female skater to win the Olympic gold in PyeongChang 2018.</div>\n<ul>\n<li>Olympic medals: 1</li>\n<li>World Championships titles: 1</li>\n</ul>\n<div>\n<div>\n\t\t\t\t\t\t\tIn other topics\n\t\t\t\t\t\t</div>\n<a>Position 3 in the ranking of the most beautiful figure skater</a>\n</div>\n</div>\n</div1>\n<div2>\n<div>\n<div>\n\t\t\t\t\t\t3\n\t\t\t\t\t</div>\n<div>\n<span>7</span>\n<div>points</div>\n</div>\n</div>\n<div>\n<button>Source\n</button>\n<h3>\n\t\t\t\t\t\t\tIrina Slutskaya\n\t\t\t\t\t\t</h3>\n<div>A retired figure skater known for her technical skills, Slutskaya is a two-time World Champion and Olympic medalist.</div>\n<ul>\n<li>Olympic medals: 2</li>\n<li>World Championships titles: 2</li>\n</ul>\n</div>\n</div2>\n<div5>\n<div>\n<div>\n\t\t\t\t\t\t6\n\t\t\t\t\t</div>\n<div>\n<span>0</span>\n<div>points</div>\n</div>\n</div>\n<div>\n<button>Source\n</button>\n<h3>\n\t\t\t\t\t\t\tAnastasia Mishina and Aleksandr Galliamov\n\t\t\t\t\t\t</h3>\n<div>A pairs skating team, Mishina and Galliamov are World Champions and known for their technical skills and expressive performances.</div>\n<li>World Championships titles: 1</li>\n</div>\n</div5>\n<div7>\n<div>\n<div>\n\t\t\t\t\t\t8\n\t\t\t\t\t</div>\n<div>\n<span>0</span>\n<div>points</div>\n</div>\n</div>\n<div>\n<button>Source\n</button>\n<h3>\n\t\t\t\t\t\t\tTatiana Navka\n\t\t\t\t\t\t</h3>\n<div>A former competitive ice dancer, Navka is an Olympic gold medalist and two-time World Champion.</div>\n<ul>\n<li>Olympic medals: 1</li>\n<li>World Championships titles: 2</li>\n</ul>\n</div>\n</div7>\n<div8>\n<div>\n<div>\n\t\t\t\t\t\t9\n\t\t\t\t\t</div>\n<div>\n<span>0</span>\n<div>points</div>\n</div>\n</div>\n<div>\n<button>Source\n</button>\n<h3>\n\t\t\t\t\t\t\tEkaterina Gordeeva and Sergei Grinkov\n\t\t\t\t\t\t</h3>\n<div>A legendary pairs skating team, Gordeeva and Grinkov won two Olympic gold medals and were known for their deep connection on the ice.</div>\n<li>Olympic medals: 2</li>\n<div>\n<div>\n\t\t\t\t\t\t\tIn other topics\n\t\t\t\t\t\t</div>\n<a>Position 10 in the ranking of the most famous ice dancer</a>\n</div>\n</div>\n</div8>\n<div9>\n<div>\n<div>\n\t\t\t\t\t\t10\n\t\t\t\t\t</div>\n<div>\n<span>0</span>\n<div>points</div>\n</div>\n</div>\n<div>\n<button>Source\n</button>\n<h3>\n\t\t\t\t\t\t\tAdelina Sotnikova\n\t\t\t\t\t\t</h3>\n<div>Sotnikova is known for being the first Russian woman to win Olympic gold in ladies\' singles figure skating, achieving this feat at the Sochi 2014 Winter Olympics.</div>\n<ul>\n<li>Olympic medals: 1</li>\n<li>World Championships titles: 0</li>\n</ul>\n</div>\n</div9>\n</ol>\n</section>\n</div0>\n<div1>\n<div>\n<h4>About this ranking</h4>\n<p>This is a community-based ranking of the most famous Russian figure skater. We do our best to provide fair voting, but it is not intended to be exhaustive. So if you notice something or skater is missing, feel free to help improve the ranking!</p>\n</div>\n<div>\n<h4>Statistics</h4>\n<ul>\n<span>2651 views</span>\n<span>75 votes</span>\n<span>10 ranked items</span>\n</ul>\n</div>\n<h4>Movers &amp; Shakers</h4>\n<div>\n<h4>Voting Rules</h4>\n<p>\n\t\t\t\tA participant may cast an up or down vote for each skater once every 24 hours. The rank of each skater is then calculated from the weighted sum of all up and down votes.\n\t\t\t</p>\n</div>\n<div>\n<h4>Categories</h4>\n<ul>\n<a>Athletes</a>\n<a>Famous Personalities</a>\n<a>Voting</a>\n</ul>\n</div>\n</div1>\n</div0>\n<div0>\n<h2>More about the Most Famous Russian Figure Skater</h2>\n<div1>\n\t\t\t\t\t\tRank #1 for the most famous Russian figure skater: Evgeni Plushenko (<a>Source</a>)\n\t\t\t\t\t</div1>\n<div2>Figure skating holds a special place in Russian sports culture. The country has produced many renowned skaters who have left a mark on the sport. These athletes often start training at a young age, dedicating countless hours to perfecting their craft. Their journey usually begins in local skating clubs, where coaches identify and nurture talent.Russian figure skaters are known for their technical skills and artistic expression. They perform complex jumps, spins, and footwork with precision. Their routines often tell a story, captivating audiences worldwide. This blend of athleticism and artistry sets them apart.The path to success is not easy. Skaters face intense competition and rigorous training schedules. They must maintain peak physical condition and mental focus. Injuries are common, but resilience defines these athletes. Support from coaches, family, and fans plays a crucial role in their careers.International competitions are the ultimate stage for Russian skaters. They compete in events like the World Championships and the Winter Olympics. These platforms provide an opportunity to showcase their talent and represent their country. Winning medals brings national pride and personal fulfillment.Coaching is another key element in the success of Russian figure skaters. Experienced coaches guide skaters through their careers, offering technical advice and emotional support. Many coaches are former skaters themselves, bringing a wealth of knowledge and experience to the rink.Choreographers also contribute to the success of these athletes. They design routines that highlight the skater\'s strengths and personality. Music selection and costume design are integral parts of the performance, creating a complete package that appeals to judges and audiences alike.Training facilities in Russia are among the best in the world. State-of-the-art rinks and equipment provide skaters with the resources they need to excel. Government support for sports development ensures that young talent receives the necessary training and opportunities.The legacy of Russian figure skating is rich with memorable performances and historic achievements. Each generation of skaters builds on the success of those who came before them. They inspire future athletes to pursue their dreams and continue the tradition of excellence.Media coverage and public interest in figure skating remain high in Russia. Skaters often become national celebrities, appearing in interviews and on television. Their success stories inspire young people to take up the sport and work hard to achieve their goals.In conclusion, Russian figure skaters are celebrated for their skill, artistry, and dedication. Their journey involves hard work, support, and a passion for the sport. They continue to set high standards and inspire future generations, ensuring that the legacy of Russian figure skating endures.</div2>\n</div0>\n</div>\n</html16>\n<figcaption0><span>Kamila Valieva channeled her inner Addams at the Russian Figure Skating Championship in Krasnoyarsk, Russia.</span><span>Alexander Vilf / AP</span></figcaption0>\n<div1>\n<div0>\n<div><a>News</a><a>TV News</a></div>\n<h1>Watch pro figure skater recreate Jenna Ortega’s iconic ‘Wednesday’ dance</h1>\n<p>16-year-old Kamila Valieva was performing at the 2023 Russian Figure Skating Championships\r\n</p>\n<div><span>By</span> <a>Ali Shutler</a></div>\n<time>29th December 2022</time> </div0>\n<figcaption>Kamila Valieva at the 2023 Russian Figure Staking Championships. CREDIT: YouTube</figcaption>\n</div1>\n<html19>\n<title>Aleksandr Gorshkov, Half of First Ice Dance Olympic Gold Medal Pair, Dies - The Moscow Times</title>\n<article>\n<a>Aleksandr Gorshkov, Half of First Ice Dance Olympic Gold Medal Pair, Dies</a>\n<figcaption>\n<span>\n\t\t\t\t\t\t\t\tSoviet ice dancers Lyudmila Pakhomova and Alexander Gorshkov, 1976.\t\t\t\t\t\t\t</span>\n<span>\n\t\t\t\t\t\t\t\tYuri Zhiltukhin and V.Nikolayev / TASS \t\t\t\t\t\t\t</span>\n</figcaption>\n<div0>\n<p>Russian skater Aleksandr Gorshkov, the first Olympic ice dancing champion and president of the Russian Figure Skating Federation, has died at the age of 76, the federation said Thursday.\xa0</p>\n<p>"Aleksandr Georgyevich gave his whole life to figure skating, his contribution to the development of Russian and world skating is invaluable," federation director general Alexander Kogan\xa0told the official TASS news agency.\xa0</p>\n<p>"The whole family of Russian skaters ... is in mourning with his family. This is a huge loss for us," he added.\xa0</p>\n<p>Gorshkov with Lyudmila Pakhomova, who was also his wife, won six world titles in seven years starting in 1969. When ice dance became an Olympic event the pair won the first gold at Innsbruck in 1976.\xa0</p>\n<p>They also won the European championship six times.\xa0</p>\n<p>Pakhomova died of leukemia in 1986 aged 39.</p>\n<p>Gorshkov had headed the Russian Figure Skating Federation since 2010.\xa0</p> </div0>\n</article>\n</html19>\n<html20>\n<title>Yekaterina Gordeeva and Sergey Grinkov | Olympic Gold, Pairs Skating, Ice Dancing | Britannica</title>\n<div0>\n<div1>\n<div>\n<dl>\n<dt>Born: </dt>\n<dd>May 28, 1971, Moscow, Russia, U.S.S.R.</dd>\n</dl>\n<em>(Show\xa0more)</em>\n</div>\n<div>\n<dl>\n<dt>Born: </dt>\n<dd>February 4, 1967, Moscow</dd>\n</dl>\n<em>(Show\xa0more)</em>\n</div>\n<div>\n<dl>\n<dt>Died: </dt>\n<dd>November 20, 1995, Lake Placid, New York, U.S.</dd>\n</dl>\n<em>(Show\xa0more)</em>\n</div>\n</div1>\n<section><p0><strong>Yekaterina Gordeeva and Sergey Grinkov</strong> (respectively,  born May 28, 1971, Moscow, Russia, U.S.S.R.; born February 4, 1967, Moscow—died November 20, 1995, <a>Lake Placid</a>, <a>New York</a>, U.S.) were a Russian-born <a>figure-skating</a> pair who gained <a>worldwide</a> acclaim with four world championship titles and two <a>Olympic</a> gold medals.</p0><p1><a>Gordeeva</a> and Grinkov were teamed at age 11 and 15, respectively, and initially appeared to many critics as an awkward couple owing to a large disparity in height (Grinkov was about a foot [30 cm] taller than Gordeeva). In 1985 they proved the critics wrong by capturing the junior world championship pairs title. The following year they went on to claim the first of four world championships (1986–87 and 1989–90) and were described as lithesome and creative, mature and strong. Their career peaked when they won their first Olympic gold medal with a unanimous vote at the <a>1988 Winter Games</a> in Calgary, Alberta, Canada, but it took a surprising turn in 1990 when, claiming their competitive drive had diminished, they retired as world champions.</p1><p2>During what turned out to be a <a>hiatus</a>, the pair married in April 1991. Thanks to rule changes in amateur skating, they were able to return to Olympic competition in <a>1994 at Lillehammer</a>, Norway, where they won a second gold medal despite a few minor errors in their routine. Afterward they immigrated to the <a>United States</a> to skate professionally. Their partnership ended abruptly in November 1995 when Grinkov died of a <a>heart attack</a> while practicing a routine. Gordeeva subsequently pursued a singles skating career. She entered professional skating competitions and appeared on television in skating-themed specials. She also toured with <a>Stars on Ice</a>, the professional skating company cofounded by Olympic gold medalist <a>Scott Hamilton</a>. She coauthored (with E.M. Swift) <em>My Sergei: A Love Story</em> (1996).</p2></section> </div0>\n</html20>\n<div><section1><h2>Biography</h2><section><p><strong>Alexandra Trusova</strong> is a Russian figure skater who is the Beijing 2022 Olympic silver medallist (representing ROC), 2021 World Figure Skating Championships bronze medallist (representing FSR), and 2020 European Championships bronze medallist.</p>\n<p>The 17-year-old is best known for her quadruple-jumping abilities. She was the first female to land each of the quad Lutz, flip and toe jumps in competition, and was also the first to land two – and then three – quads in a free skate.</p>\n<p>The three-time Russian national medallist has three Grand Prix gold medals to her name, as well, and trains under <strong>Eteri Tutberidze</strong>, having briefly worked with Olympic gold medallist <strong>Evgeni Plushenko</strong>.</p>\n<p>After years of hype around her and her "quad squad" teammates, Trusova made her Olympic debut at the Olympic Winter Games <strong>Beijing 2022</strong>, competing in the women\'s event after being passed over for the team event.</p>\n<p>In the short program, Trusova fell on a triple Axel attempt, finishing in a disappointing fourth place. However, she picked herself up in the free skate. Opting against the triple Axel, she instead packed five quadruple jumps into her free routine, landing three of them cleanly to score a personal best 177.13 points in the segment for a total of 251.73 and win silver.</p><p>(As of 2 March 2022, the women\'s singles results from Beijing 2022 are provisional.)</p></section></section1><section2><h2>Featured</h2> <div><article><a>Figure Skating</a><h3>8 things we learned from the figure skating season ahead of Beijing 2022</h3></article><article><a>Figure Skating</a><h3>World figure skating championships 2021: Preview, TV &amp; streaming details, who’s competing, and more</h3></article><article><a>Figure Skating</a><h3>Evgeni Plushenko exclusive: The coach of Kostornaia and Trusova says, we are ‘a very powerful team’</h3></article></div></section2></div>
"""


# 解析并生成文档树
document_tree = build_document_tree(HTML_test)

print_tree(document_tree)

/tmp/ipykernel_2263634/1275741401.py:90: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  direct_text = ''.join(list_item.find_all(text=True, recursive=False)).strip()


Tag: root, Text: , Level: 0
Tag: span, Text: COLLECTION, Level: 1
Tag: span, Text: 22 LISTS, Level: 1
Tag: a, Text: Figure Skating, Level: 1
Tag: span, Text: Lists about the beautiful, technically grueling sports of figure skating and ice dancing and the talented athletes who compete in them all around the world., Level: 1
Tag: ul, Text: None, Level: 1
Tag: span, Text: The Best Movies About Figure Skating, Level: 1
Tag: span, Text: The Greatest Male Figure Skaters, Level: 1
Tag: span, Text: The Greatest Ice Dancers Ever, Level: 1
Tag: span, Text: The Top Russian Female Figure Skaters, Level: 1
Tag: span, Text: The Greatest Olympic Figure Skaters, Level: 1
Tag: span, Text: The Top American Figure Skaters, Level: 1
Tag: span, Text: Top Female Skaters Working Today, Level: 1
Tag: span, Text: The Top American Women in History, Level: 1
Tag: span, Text: The Top Russian Male Figure Skaters, Level: 1
Tag: span, Text: The Top Russian Figure Skaters, Level: 1
Tag: span, Text: The Top Skaters Wo

In [ ]:
HTML_test = """
<html0><h3 class="text-center">News!</h3>
<br>
<ul class="news-list">
    <li class="news-item">
        <h5>
            2024-11-22: Our latest work on the robustness of Vision-Language Models (VLMs), titled "<a href=""><b>Safeguarding Vision-Language Models: Mitigating Vulnerabilities to Gaussian Noise in Perturbation-based Attacks</b></a>, has been submitted to CVPR 2025. This work identifies vulnerabilities in Vision-Language Models (VLMs) to noisy image attacks due to the lack of noise augmentation during training. It introduces Robust-VLGuard, a dataset to enhance robustness against Gaussian noise, and proposes DiffPure-VLM, a defense leveraging diffusion models to transform adversarial noise into Gaussian noise. The dataset and code will be open-sourced to advance secure VLM research.
        </h5>
    </li>
    <li class="news-item">
        <h5>
            2024-11-2: Grateful to have reached <b>100 citations</b> on <a href="https://scholar.google.com/citations?user=VKT0Vg4AAAAJ&hl=en">Google Scholar</a>! This is a small milestone that encourages me to keep learning and contributing. Thank you to everyone who has supported my work along the way.
        </h5>
    </li>
    <li class="news-item">
        <h5>
            2024-8-30~9-4: I was thrilled to be an <b>oral speaker</b>, presenting two of our papers, DLAFormer and DRFormer, at ICDAR in Athens, Greece (Photos can be found <a href="https://photos.app.goo.gl/8aw4mYxDUtA5ELuJ6">here</a>.). Athens is a beautiful city steeped in history, and it was wonderful to experience its rich cultural atmosphere. I also enjoyed meeting many new colleagues in the research community, making it a truly memorable and unforgettable journey.
        </h5>
    </li>
    <li class="news-item">
        <h5>
            2024-8-16: Our latest progress, the complete version of DLAFormer, titled "<a href=""><b>UniHDSA: A Unified Relation Prediction Approach for Hierarchical Document Structure Analysis</b></a>", has been submitted to Pattern Recognition Journal.
        </h5>
    </li>
    <li class="news-item">
        <h5>
            2024-7-26: Detect-Order-Construct have been accepted by Pattern Recognition!
        </h5>
    </li>
    <li class="news-item">
        <h5>
            2024-6-29: DLAFormer, UniVIE, and DRFormer have been selected for oral presentation in ICDAR 2024!
        </h5>
    </li>
    <li class="news-item">
        <h5>
            2024-4-12: DLAFormer, UniVIE, and DRFormer have been accepted by ICDAR 2024!
        </h5>
    </li>
    <li class="news-item">
        <h5>
            2024-3-15: It's thrilling to share that Azure AI Document Intelligence has introduced support for Hierarchical Document Structure Analysis (HDSA), as detailed in our recently published paper, "<b><a href="https://arxiv.org/abs/2401.11874">Detect-Order-Construct: A Tree Construction based Approach for Hierarchical Document Structure Analysis.</a></b>" This advancement opens up new possibilities for analyzing complex document structures more effectively. For more details on this breakthrough, check out our publication on <a href="https://arxiv.org/abs/2401.11874">arXiv</a> and the <a href="https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/document-intelligence-preview-adds-more-prebuilts-support-for/ba-p/4084608">official announcement</a>.
        </h5>
    </li>
    <li class="news-item">
        <h5>
            2024-3-4: We released the <a href="https://github.com/JarvisUSTC/Language-Enhanced-CLIP-For-Multi-label-Image-Recognition"><b>source code of our solution for the Language-Enhanced Image New Category Discovery</b></a> at the 2nd The Guangdong–Hong Kong–Macao Greater Bay Area International Algorithm and Programming Contest (Visual Prompt Tuning Challenge @ CVPR 2023 HIT Workshop (2023)).
        </h5>
    </li>
    <li class="news-item">
        <h5>
            2024-2-1: We submitted our new progress on Document Layout Analysis, named <b>DLAFormer: A End-to-End Transformer for Document Layout Analysis</b>, to ICDAR 2024.
        </h5>
    </li>
    <li class="news-item">
        <h5>2024-1-18: We released a unified label space approach for Visual Information Extraction, named <a href="https://arxiv.org/abs/2401.09220"><b>UniVIE: A Unified Label Space Approach to Visual Information Extraction from Form-like Documents</b></a>! In this work, we present a new perspective, reframing VIE as a relation prediction problem and unifying labels of different tasks into a single label space. This unified approach allows for the definition of various relation types and effectively tackles hierarchical relationships in form-like documents.</h5>
    </li>
    <li class="news-item">
        <h5>2024-1-16: We released a new technical paper for logical layout analysis in natural scenes, named <a href="https://arxiv.org/abs/2401.09232"><b>Dynamic Relation Transformer for Contextual Text Block Detection</b>!</h5>
    </li>
    <li class="news-item">
        <h5>2023-12-7: <a href="https://mp.weixin.qq.com/s/B1r2uJ0bNg_u50vuNI5MPw"><b>2nd Prize</b>, 2023 International Algorithm Case Competition (Visual Prompt Tuning Challenge @ CVPR 2023 HIT Workshop (2023))</a>, <b>200,000 RMB bonus</b>!</h5>
    </li>
    <li class="news-item">
        <h5>2023-11-14: We submitted our new progress on Hierarchical Document Structure Analysis to Pattern Recognition Journal.</h5>
    </li>
    <li class="news-item">
        <h5>2023-7-12: <a href="https://arxiv.org/abs/2303.11615"><b>Robust Table Structure Recognition with Dynamic Queries Enhanced Detection Transformer</b></a> is accepted by Pattern Recognition Journal!</h5>
    </li>
    <li class="news-item">
        <h5>2023-4-20: Two papers are accepted by <b>ICDAR 2023!</b></h5>
    </li>
    <li class="news-item">
        <h5>2023-3-10: We proposed a new <a href="https://arxiv.org/abs/2303.11615"><b>Dynamic Queries based Detection Transformer</b></a> for more robust table structure recognition!</h5>
    </li>
    <li class="news-item">
        <h5>2022-12-1: <a href="https://mp.weixin.qq.com/s/Au8oRHbX0Ls2gL4WiBNmqw"><b>2nd Prize</b>, 2022 International Algorithm Case Competition (Panoptic Scene Graph Challenge @ ECCV 2022 SenseHuman Workshop (2022))</a>, <b>100,000 RMB bonus</b>!</h5>
    </li>
    <li class="news-item">
        <h5>2022-9-1: A paper is accepted by <b>ACM MM 2022!</b></h5>
    </li>
</ul>
<br>
<h3 class="text-center">Academic Services</h3>
<h4><b>Journal Reviewer</b></h4>
<ul>
    <li class="news-item">
        International Journal on Document Analysis and Recognition
    </li>
</ul>
<h4><b>Conference Reviewer</b></h4>
<ul>
    <li class="news-item">
        International Conference on Document Analysis and Recognition
    </li>
</ul>

<h3 class="text-center">Publications</h3>
<br>

<h4><b>Document Intelligence:</b></h4>
<br>
<ol class="custom-list">
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="">Safeguarding Vision-Language Models: Mitigating Vulnerabilities to Gaussian Noise in Perturbation-based Attacks</a></b></span>
            <p class="author"><u><b>Jiawei Wang<b class="equal-author">*</b></b></u>, Yushen Zuo<b class="equal-author">*</b>, Yuanjun Chai, Zhendong Liu, Yicheng Fu, Yichun Feng<span class="corresponding-author"></span>, Kin-man Lam<span class="corresponding-author"></span></p>
			<p class="oral">Submitted to CVPR 2025</p>
            <p class="project"><a href="">PDF will be available soon.</a></p>
        </div>
    </li>
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="">UniHDSA: A Unified Relation Prediction Approach for Hierarchical Document Structure Analysis</a></b></span>
            <p class="author"><u><b>Jiawei Wang</b></u><span class="corresponding-author"></span>, Kai Hu, Qiang Huo</p>
			<p class="oral">Submitted to Pattern Recognition</p>
            <p class="project"><a href="">PDF will be available soon.</a></p>
        </div>
    </li>
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="https://arxiv.org/abs/2405.11757">DLAFormer: An End-to-End Transformer for Document Layout Analysis</a></b></span>
            <p class="author"><u><b>Jiawei Wang<b class="equal-author">*</b></b></u><span class="corresponding-author"></span>, Kai Hu<b class="equal-author">*</b><span class="corresponding-author"></span>, Qiang Huo</p>
			<p class="oral">2024 International Conference on Document Analysis and Recognition, Oral</p>
            <p class="project"><a href="https://arxiv.org/abs/2405.11757">PDF</a></p>
        </div>
    </li>
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="https://arxiv.org/abs/2401.09220">UniVIE: A Unified Label Space Approach to Visual Information Extraction from Form-like Documents</a></b></span>
            <p class="author">Kai Hu<span class="corresponding-author"></span>, <u><b>Jiawei Wang</b></u>, Weihong Lin, Zhuoyao Zhong, Lei Sun, Qiang Huo</p>
			<p class="oral">2024 International Conference on Document Analysis and Recognition, Oral</p>
            <p class="project"><a href="https://arxiv.org/abs/2401.09220">PDF</a></p>
        </div>
    </li>
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="https://arxiv.org/abs/2401.09232">Dynamic Relation Transformer for Contextual Text Block Detection</a></b></span>
            <p class="author"><u><b>Jiawei Wang<b class="equal-author">*</b></b></u><span class="corresponding-author"></span>, Shunchi Zhang<b class="equal-author">*</b><span class="corresponding-author"></span>, Kai Hu<b class="equal-author">*</b><span class="corresponding-author"></span>, Chixiang Ma, Zhuoyao Zhong, Lei Sun, Qiang Huo</p>
			<p class="oral">2024 International Conference on Document Analysis and Recognition, Oral</p>
            <p class="project"><a href="https://shunchizhang.github.io/drformer/">Project page</a></p>
        </div>
    </li>
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="https://arxiv.org/abs/2401.11874">Detect-Order-Construct: A Tree Construction based Approach for Hierarchical Document Structure Analysis</a></b></span>
            <p class="author"><u><b>Jiawei Wang</b></u><span class="corresponding-author"></span>, Kai Hu, Zhuoyao Zhong, Lei Sun, Qiang Huo</p>
            <p class="oral">Pattern Recognition, 2024 (Q1 Journal, IF 7.5)</p>
            <p class="project"><a href="https://www.sciencedirect.com/science/article/pii/S0031320324005879">PDF</a>,<a href="https://github.com/microsoft/CompHRDoc">Dataset</a></p>
        </div>
    </li>
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="https://link.springer.com/chapter/10.1007/978-3-031-41679-8_14">DQ-DETR: Dynamic Queries Enhanced Detection Transformer for Arbitrary Shape Text Detection</a></b></span>
            <p class="author">Chixiang Ma<span class="corresponding-author"></span>, Lei Sun, <u><b>Jiawei Wang</b></u>, Qiang Huo</p>
			<p class="oral">2023 International Conference on Document Analysis and Recognition, Oral</p>
            <p class="project"><a href="https://link.springer.com/chapter/10.1007/978-3-031-41679-8_14">PDF</a></p>
        </div>
    </li>
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="https://link.springer.com/chapter/10.1007/978-3-031-41734-4_12">A Hybrid Approach to Document Layout Analysis for Heterogeneous Document Images</a></b></span>
            <p class="author">Zhuoyao Zhong<span class="corresponding-author"></span>, <u><b>Jiawei Wang</b></u>, Haiqing Sun, Kai Hu, Erhan Zhang, Lei Sun, Qiang Huo</p>
			<p>2023 International Conference on Document Analysis and Recognition, Poster</p>
            <p class="project"><a href="https://link.springer.com/chapter/10.1007/978-3-031-41734-4_12">PDF</a></p>
        </div>
    </li>
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="https://www.sciencedirect.com/science/article/abs/pii/S0031320323005150">Robust Table Structure Recognition with Dynamic Queries Enhanced Detection Transformer</a></b></span>
            <p class="author"><u><b>Jiawei Wang</b></u>, Weihong Lin, Chixiang Ma, Mingze Li, Zheng Sun, Lei Sun<span class="corresponding-author"></span>, Qiang Huo</p>
			<p class="oral">Pattern Recognition, 2023 (Q1 Journal, IF 7.5)</p>
            <p class="project"><a href="https://www.sciencedirect.com/science/article/abs/pii/S0031320323005150">PDF</a></p>
        </div>
    </li>
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="https://arxiv.org/pdf/2208.04921v1.pdf"> TSRFormer: Table Structure Recognition with Transformers</a></b></span>
            <p class="author">Weihong Lin, Zheng Sun, Chixiang Ma, Mingze Li, <u><b>Jiawei Wang</b></u>, Lei Sun, Qiang Huo<span class="corresponding-author"></span></p>
			<p>2022 ACM Multimedia, Poster</p>
            <p class="project"><a href="https://arxiv.org/pdf/2208.04921v1.pdf">PDF</a></p>
        </div>
    </li>
</ol>
<br>
<h4><b>Physics Related:</b></h4>
<br>
<ol class="custom-list">
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="https://epjplus.epj.org/articles/epjplus/abs/2022/10/13360_2022_Article_3372/13360_2022_Article_3372.html">	
				Study of nonequilibrium phase transitions mechanisms in exclusive network and node model of heterogeneous assignment based on real experimental data of KIF3AC and KIF3CC motors</a></b></span>
            <p class="author">Yuqing Wang<span class="corresponding-author"></span>, Chang Xu, Molin Fang, Tianze Li, Liwen Zhang, Dasen Wei, Kaichen Ouyang, Tunyu Zhang, Chuzhao Xu, Haosong Sun, Yunzhi Wang, <u><b>Jiawei Wang</b></u></p>
			<p>The European Physical Journal Plus, 2022</p>
        </div>
    </li>
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="https://www.worldscientific.com/doi/abs/10.1142/S0217979219502175">Physical mechanisms in impacts of interaction factors on totally asymmetric simple exclusion processes</a></b></span>
            <p class="author">Yuqing Wang<span class="corresponding-author"></span>, <u><b>Jiawei Wang</b></u>, Binghong Wang</p>
			<p>International Journal of Modern Physics B, 2019</p>
        </div>
    </li>
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="https://www.worldscientific.com/doi/abs/10.1142/S0217979219502291">Stochastic dynamics in nonequilibrium phase transitions of multiple totally asymmetric simple exclusion processes coupled with strong and weak interacting effects</a></b></span>
            <p class="author">Yuqing Wang<b class="equal-author">*</b><span class="corresponding-author"></span>, <u><b>Jiawei Wang<b class="equal-author">*</b></b></u>, Ziang Zhu<b class="equal-author">*</b>, Binghong Wang</p>
			<p>International Journal of Modern Physics B, 2019</p>
        </div>
    </li>
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="https://www.worldscientific.com/doi/abs/10.1142/S021797921950228X">Evolvement laws and stability analyses of traffic network constituted by changing ramps and main road</a></b></span>
            <p class="author">Yuqing Wang<span class="corresponding-author"></span>, Chaofan Zhou, <u><b>Jiawei Wang</b></u>, Xinpeng Ni</p>
			<p>International Journal of Modern Physics B, 2019</p>
        </div>
    </li>
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="https://www.worldscientific.com/doi/abs/10.1142/S0217984918502093">A macroscopic model for VOC emissions process complemented by real data</a></b></span>
            <p class="author">Yuqing Wang<span class="corresponding-author"></span>, Chaofan Zhou, Ziang Zhu, <u><b>Jiawei Wang</b></u>, Zimeng Wang, Chenhao Fang, Bin Jia</p>
			<p>Modern Physics Letters B, 2018</p>
        </div>
    </li>
    <li class="custom-item">
        <div class="item-content">
            <span class="title"><b><a href="https://ieeexplore.ieee.org/abstract/document/8490540/">Control Strategies for Reducing VOCs Emission Process Based on Empirical Data</a></b></span>
            <p class="author">Yuqing Wang<b class="equal-author">*</b><span class="corresponding-author"></span>, <u><b>Jiawei Wang<b class="equal-author">*</b></b></u>, Ziang Zhu<b class="equal-author">*</b>, Chaofan Zhou, Yiyao Kou, Jing Sun, Zhengwei Mei, Ziwu Li, Peng Wu, Donghu Wang, Si Zhang, Wenli Zhang</p>
			<p class="oral">2018 IEEE International Conference on Mechatronics, Robotics and Automation (ICMRA), Best Presentation Award</p>
        </div>
    </li>
</ol>

<div><span class="corresponding-author"></span>: Corresponding Author<br><b class="equal-author">*</b>: Equal Contribution</div>
</html0>
"""

import sys
sys.path.append('..')
from html4rag.html_utils import simplify_html
from bs4 import BeautifulSoup

print(simplify_html(BeautifulSoup(HTML_test, 'html.parser')))


# 解析并生成文档树
document_tree = build_document_tree(HTML_test)

print_tree(document_tree)

<html0><h3>News!</h3>
<ul>
<h5>
            2024-11-22: Our latest work on the robustness of Vision-Language Models (VLMs), titled "<b>Safeguarding Vision-Language Models: Mitigating Vulnerabilities to Gaussian Noise in Perturbation-based Attacks</b>, has been submitted to CVPR 2025. This work identifies vulnerabilities in Vision-Language Models (VLMs) to noisy image attacks due to the lack of noise augmentation during training. It introduces Robust-VLGuard, a dataset to enhance robustness against Gaussian noise, and proposes DiffPure-VLM, a defense leveraging diffusion models to transform adversarial noise into Gaussian noise. The dataset and code will be open-sourced to advance secure VLM research.
        </h5>
<h5>
            2024-11-2: Grateful to have reached <b>100 citations</b> on <a>Google Scholar</a>! This is a small milestone that encourages me to keep learning and contributing. Thank you to everyone who has supported my work along the way.
        </h5>
<h5>
            20

In [35]:
from bs4 import BeautifulSoup

def remove_nonsemantic_tags(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # 定义需要保留的语义化标签
    semantic_tags = {"h1", "h2", "h3", "h4", "h5", "h6", "ul", "ol", "li", "p", "table", "tr", "td", "title"}
    # semantic_tags = {
    #     "html", "head", "body", "header", "footer", "nav", "main", 
    #     "section", "article", "title", "aside", "h1", "h2", "h3", "h4", "h5", "h6", 
    #     "p", "blockquote", "ul", "ol", "li", "dl", "dt", "dd", "figure", 
    #     "figcaption", "address", "table", "thead", "tbody", "tfoot", "tr", 
    #     "th", "td", "caption", "colgroup", "col", "form", "fieldset", 
    #     "legend", "label", "input", "textarea", "select", "option", "button", 
    #     "audio", "video", "source", "track", "picture", "img", "map", "area", 
    #     "strong", "em", "code", "pre", "kbd", "samp", "abbr", "cite", "dfn", 
    #     "mark", "time", "var"
    # }

    # 遍历所有标签
    for tag in soup.find_all(True):  # True 表示查找所有标签
        # 只要这些语义标签存在于标签名中，就保留
        if "html" in tag.name:
            continue

        if tag.name not in semantic_tags:
            # 替换非语义标签为其内容，移除多余的换行符
            tag.unwrap()
    
    # 去除多余换行符，清理整个文档
    cleaned_html = "".join(soup.prettify().splitlines())
    return cleaned_html

HTML_test = """
<html0><title>The Greatest Russian Male Figure Skaters of All Time</title><div><div1><div><div><span>COLLECTION</span><span>22 LISTS</span></div><a>Figure Skating</a><span>Lists about the beautiful, technically grueling sports of figure skating and ice dancing and the talented athletes who compete in them all around the world.</span></div><ul><span>The Best Movies About Figure Skating</span><span>The Greatest Male Figure Skaters</span><span>The Greatest Ice Dancers Ever</span><span>The Top Russian Female Figure Skaters</span><span>The Greatest Olympic Figure Skaters</span><span>The Top American Figure Skaters</span><span>Top Female Skaters Working Today</span><span>The Top American Women in History</span><span>The Top Russian Male Figure Skaters</span><span>The Top Russian Figure Skaters</span><span>The Top Skaters Working Today</span><span>The Greatest Women of All Time</span><span>The Top American Male Figure Skaters</span><span>The Top Current Figure Skating Pairs</span><span>The Greatest Ice Dancing Pairs Ever</span><span>The Best Retired Figure Skaters</span><span>The Greatest Men Working Today</span><span>Top Ice Dancing Pairs Working Today</span><span>The Greatest Figure Skating Pairs Ever</span><span>The Top American Ice Dancers</span><span>The Greatest Ice Dancers in the World</span><span>The Top Russian Ice Dancers</span></ul></div1><article><div1><div><p>Russian figure skaters have been making their mark in the sport since the early days of competition. The first time figure skating was held as an Olympic competition was during the 1908 Summer <a>Olympic Games</a> (London), which was also the first and only time a special figures contest was held. With his gold-medal win in this special figures category, Russian figure skater Nikolai Panin became the first Russian to win an Olympic gold medal.</p>\n<p>Russian figures skaters in men\'s\xa0singles competitions have continued to bring glory to Russia up to the present day. The World Figure Skating Championships, for example, were dominated by Russian male skaters in the late \'90s and early 2000s. From 1998 to 2004, it was a Russian who took the gold -\xa0<a>Alexei Yagudin</a>, 1998 to 2000 and 2002; <a>Evgeni Plushenko</a>, 2001, 2003, and 2004.</p>\n<p>This list contains the best skaters in Russian history. These are the men who have inspired newcomers to the sport and drawn audiences to figure skating competitions. They may all have different styles, but they are each worthy of being called the greatest Russian male figure skaters.</p></div><div>Most divisive: <span>Alexei Urmanov</span></div><div>Over 400 Ranker voters have come together to rank this list of The Top Russian Male Figure Skaters</div></div1><ul><li0><div><div><div>1</div><div>225 VOTES</div></div><a>Alexei Yagudin</a></div><li>Photo: <a>YouTube</a></li><div><span>Alexei Yagudin medaled many times in his career at many of figure skating\'s most esteemed events, including the Winter Olympic Games -\xa0<a>he took the gold in 2002</a>.</span>\n<span>225 votes</span><div><div>More\xa0<a>Alexei Yagudin</a></div><ul><li><span>#5<span> of 51 on</span></span><a>The Greatest Male Figure Skaters of All Time</a></li><li><span>#11<span> of 41 on</span></span><a>The Greatest Retired Figure Skaters of All Time</a></li><li><span>#4<span> of 37 on</span></span><a>The Greatest Russian Figure Skaters of All Time</a></li></ul></div></div></li0><li1><div><div><div>2</div><div>216 VOTES</div></div><a>Evgeni Plushenko</a></div><li>Photo: <a>YouTube</a></li><div><span>Evgeni Plushenko counts among his career accomplishments winning the gold at the Russian Figure Skating Championships <a>10 times</a>. </span>\n<span>216 votes</span><div><div>More\xa0<a>Evgeni Plushenko</a></div><ul><li><span>#359<span> of 1,269 on</span></span><a>The 1000+ Best Athletes Of All Time, Ranked</a></li><li><span>#2<span> of 51 on</span></span><a>The Greatest Male Figure Skaters of All Time</a></li><li><span>#2<span> of 37 on</span></span><a>The Greatest Russian Figure Skaters of All Time</a></li></ul></div></div></li1><li3><div><div><div>4</div><div>32 VOTES</div></div><a>Alexandre Fadeev</a></div><li>Photo: <a>YouTube</a></li><div><span>In 1984, <a>1987</a>, 1988, and <a>1999</a>, Alexandre Fadeev (representing the USSR) earned the gold medal at the European Figure Skating Championships.</span>\n<span>32 votes</span></div></li3></ul></article></div></html0>\n<html1><title>Who are the Russian female figure skaters who filled the winners podium-2021? (PHOTOS + VIDEO) - Russia Beyond</title><div1><div0><h1>Who are the Russian female figure skaters who filled the winners podium-2021? (PHOTOS + VIDEO)</h1><a>\n            Lifestyle\n        </a><time>\n             March 30 2021\n        </time><a>\n                    Yekaterina Sinelschikova\n                </a></div0><div2>\n            For the first time in history, Russian figure skaters occupied the entire winners podium at the 2021 World Figure Skating Championships in the ladies’ singles in Stockholm. Previously, only American women skaters had managed the same feat, back in 1991!\n        </div2><div3><h30>Anna Shcherbakova - Gold medalist</h30><div0><p>Anna Shcherbakova of Russia competes in Women\'s Free Skating during the ISU World Figure Skating Championships on March 26, 2021 in Stockholm </p>\nJoel Marklund/Keystone Press Agency/Global Look Press\n            </div0><h31>Elizaveta Tuktamysheva - Silver medalist</h31><p7>A young Tuktamysheva had prevailed at Russian Championships at the end of the noughties, particularly in the ladies singles. She was labelled the child prodigy of figure skating, since she had come to the sport relatively late, at the age of nine, and by the time she was 12, she was already performing extremely difficult elements, having mastered the whole arsenal of triple jumps. It was thanks to these that she earned a reputation as the most “leaping” female figure skater.</p7><p8>“I trained in the old school. I started skating and winning in major league sports very early. This was 2009-2010, when there had been a decline in Russian ladies’ singles skating and you just needed to know how to jump. If you could jump, you would win. That’s the way it turned out ever since I was young. I jumped and won. It was normal. It was how the system worked,” she <a>recounted</a>.</p8><p9>In 2018-2019, she was arguably the most talked about figure skater in the world, after her bewitching performance in a flight attendant’s costume to the accompaniment of Britney Spears’ Toxic in the Grand Prix Series. Accusations that she had “crossed the line” rained down on Tuktamysheva then, to which she just retorted: “I’m just putting the sexiness back into figure skating.”</p9><p13>She is talked about as the main record-setter in Russian figure skating. Trusova was the first female figure skater in history to perform three quads in a single program. In terms of technique, she has already beaten all the adult champions. Recently, a video of her demonstration program was reposted by Hollywood actress Sharon Stone.</p13><p14>The young skater herself admits that only two things in life leave her exhilarated - skating and her dogs (Tina the chihuahua, Lana the poodle and Jack the husky). She has been managed by Evgeni Plushenko since 2020 - and lives in a cottage attached to his academy, so as not to waste time on travel.</p14></div3><div4><a>\n                    sport\n                </a><a>\n                    figure skating\n                </a></div4></div1></html1>\n<div0>\n<span>\n<a>Home</a>\n<a>/ Lifestyle</a>\n<a>/ People</a>\n</span>\n<h1>Figure skating legends inspire youth at Olympic Day event</h1>\n<span>\n          By China Daily | \n          chinadaily.com.cn |\n          Updated: 2024-06-24 15:22\n</span>\n<span>Share</span>\n<span>Share - WeChat</span>\n<span>CLOSE</span>\n<div>\n<div>\n<figcaption>\n   Former Team China figure skater Tong Jian guides students to grasp skating skills on Sunday.\xa0[Photo provided to China Daily] \n </figcaption>\n<p>A figure skating public welfare class, held at the National Speed Skating Oval on Sunday to commemorate Olympic Day, became a platform for inspiration and skill-building for young enthusiasts.</p>\n<p>Former Team China figure skater Tong Jian, a two-time world champion in figure skating pairs alongside his partner Pang Qing, and Russian international coach Andrei Lazukin, took the lead in guiding the eager students through ice skating exercises. Their hands-on approach not only showcased the intricacies of the sport but also nurtured a relaxed and enjoyable learning environment for all participants.</p>\n</div>\n<div>\n<span>1</span>\n<a>2</a>\n<a>3</a>\n<a>4</a>\n<a>Next</a>\xa0\xa0 <a>&gt;&gt;|</a>\n</div>\n<div>\n<a>\n                1/4\n              </a>\n<a>Next</a>\n</div>\n</div>\n</div0>\n<html3>\n<title>Katya Alexandrovskaya: How Did the Figure Skater Die?</title>\n<div0>\n<div0>\n<h1>Katya Alexandrovskaya: How Did the Figure Skater Die?</h1> <div>\n<a> Kriti Mehrotra</a>\n<span>September 16, 2023</span>\n<div>\n<span>Facebook</span><span>Twitter</span><span>Pinterest</span><span>Reddit</span><span>Email</span> </div>\n</div>\n</div0>\n<div>\n<p2>“[Katya] was so tiny, small even for her young age,” her first local coach stated. “I took a closer look at her. This girl was amazing right from the start. She was so swift… and absolutely fearless.” It thus comes as no surprise she began competing at 11 itself, only to then find herself pairing up with Vladislav Lysoy and Aleksandr Epifanov before finding a long-term partner in Harley Windsor. The truth is they’d met in December 2015 for a tryout, but it was evident there was magic between the duo the moment they stepped foot into the rink together thanks to their similar techniques.</p2>\n<p4>Hence came Katya’s release by Russia upon the Australian Association’s request, a move down under, expedited citizenship, and countless training sessions to ensure the pair could debut in 2016. Their first Junior Grand Prix (JGP) in the Czech Republic wasn’t really great, yet they returned with a bang for the Estonia event less than a month later, meaning they earned gold medals. Then came the duo’s 2017 Junior World Championship, 2018 Junior Grand Prix Win, and 2018 Winter Olympics participation, just for them to end up ranking 18 before deciding to change coaches.</p4>\n<p7>Katya was thus medically ordered to step away from her love of skating for good, as a result of which her partnership with Harley also came to a mutual end and she found herself isolated, per the movie. Alas, it all got too much for the 20-year-old and she jumped from the 6th-floor window of her Moscow home on July 18, 2020 — she died by suicide, leaving behind a mere note reading “Lyublyu” (Rough Translation: I love).</p7>\n</div>\n</div0>\n</html3>\n<html4>\n<title>List of Russian champions in figure skating – Pairs - FamousFix List</title>\n<div>\n<div>\n<div>\n<h1>Russian champions in figure skating – Pairs</h1>\r\nThis list has <i>28 members</i>.\r\n</div>\n<div>\r\nFLAG\r\n</div>\n<span>\r\nLike \n</span>\n</div>\n<ul>\n<div0>\n<div>\n<a>Ekaterina Gordeeva</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#1</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>36 </span>\n<span>3 </span>\n<span>2 </span>\n</div>\n<div>Ekaterina "Katia" Alexandrovna Gordeeva (Russian: Екатерина Александровна Гордеева; born 28 May 1971) is a Russian former figure skater. Together with her husband, the late Sergei Grinkov, she was the 1988 and 1994 Olympic Champion and four-time World Champion (1986, 1987, 1989, 1990) in pair skating. After Grinkov\'s death, Gordeeva continued performing as a singles skater.</div>\n<ul>\n<li>\n<a>Figure skaters from Moscow</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>232T</span>\n</li>\n<li>\n<a>Competitors at the 1990 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>351T</span>\n</li>\n<li>\n<a>Russian State University of Physical Education, Sport, Youth and Tourism alumni</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>49T</span>\n</li>\n</ul>\n</div0>\n<div1>\n<div>\n<a>Yelena Berezhnaya</a>\n<span>Pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#2</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n</div>\n<div>Elena Viktorovna Berezhnaya (Russian: Елена Викторовна Бережная, born 11 October 1977) is a Russian former pair skater. With partner Anton Sikharulidze, she is the 1998 and 1999 World champion, 1998 Olympic silver medalist and 2002 Olympic champion.</div>\n<ul>\n<li>\n<a>Competitors at the 1998 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>314T</span>\n</li>\n<li>\n<a>Competitors at the 2001 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>345T</span>\n</li>\n<li>\n<a>Sportspeople from Stavropol Krai</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>47T</span>\n</li>\n</ul>\n</div1>\n<div2>\n<div>\n<a>Sergei Grinkov</a>\n<span>Russian pair skater (1967–1995)</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#3</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n</div>\n<div>Sergei Mikhailovich Grinkov (Russian: Серге́й Миха́йлович Гринько́в; 4 February 1967 — 20 November 1995) was a Russian pair skater. Together with partner and wife Ekaterina Gordeeva, he was the 1988 and 1994 Olympic Champion and a four-time World Champion (1986, 1987, 1989, 1990).</div>\n<ul>\n<li>\n<a>Figure skaters from Moscow</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>232T</span>\n</li>\n<li>\n<a>Competitors at the 1990 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>351T</span>\n</li>\n<li>\n<a>Russian State University of Physical Education, Sport, Youth and Tourism alumni</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>49T</span>\n</li>\n</ul>\n</div2>\n<div3>\n<div>\n<a>Ksenia Stolbova</a>\n<span>Figure skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#4</b> ·\r\n\t\t\t\t\t\t\t</div>\n<div>Ksenia Andreyevna Stolbova (Russian: Ксе́ния Андре́евна Столбо́ва; born 7 February 1992) is a retired Russian pair skater who skated with Andrei Novoselov and Fedor Klimov. She is the 2014 Olympic silver medalist, the 2014 Olympic champion in the team event, the 2014 World silver medalist, a three-time European medalist (2012 bronze, 2014, 2015 silver), the 2015–16 Grand Prix Final champion, the 2013 Winter Universiade champion, a two-time World Junior medalist (2010 bronze, 2011 silver), and a three-time Russian national champion (2014, 2015, 2017). Stolbova announced her retirement from competitions on 12 February 2020.</div>\n<ul>\n<li>\n<a>Figure skaters from Saint Petersburg</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>99T</span>\n</li>\n<li>\n<a>Competitors at the 2013 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>254T</span>\n</li>\n<li>\n<a>Universiade gold medalists for Russia</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>495T</span>\n</li>\n</ul>\n</div3>\n<div4>\n<div>\n<a>Tatiana Volosozhar</a>\n<span>Russian-Ukrainian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#5</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>1 </span>\n</div>\n<div>Tatiana Andreyеvna Volosozhar (Russian: Татья́на Андре́евна Волосожа́р, Ukrainian: Тетяна Андріївна Волосожар; born 22 May 1986) is a Ukrainian-born Russian pair skater. With Maxim Trankov, she is the two-time 2014 Olympic champion in the pairs and in team events, the 2013 World champion, a four-time (2012, 2013, 2014 and 2016) European champion, the 2012 Grand Prix Final champion, and a three-time (2011, 2013 and 2016) Russian national champion. They have also won six events on the Grand Prix series.</div>\n<ul>\n<li>\n<a>Universiade silver medalists for Ukraine</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>146T</span>\n</li>\n<li>\n<a>Season\'s world number one figure skaters</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>32T</span>\n</li>\n<li>\n<a>Season-end world number one figure skaters</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>57T</span>\n</li>\n</ul>\n</div4>\n<div5>\n<div>\n<a>Maxim Trankov</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#6</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>1 </span>\n</div>\n<div>Maxim Leonidovich Trankov (Russian: Макси́м Леони́дович Транько́в, born 7 October 1983) is a Russian pair skater. With Tatiana Volosozhar, he is a two-time  2014 Olympic champion in the pairs and in team events, the 2013 World champion, a four-time (2012, 2013, 2014, 2016) European champion, the 2012 Grand Prix Final champion, and a three-time (2011, 2013, 2016) Russian national champion. They have also won six events on the Grand Prix series.</div>\n<ul>\n<li>\n<a>Competitors at the 2005 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>131T</span>\n</li>\n<li>\n<a>Universiade bronze medalists for Russia</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>263T</span>\n</li>\n<li>\n<a>Season\'s world number one figure skaters</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>31T</span>\n</li>\n</ul>\n</div5>\n<div6>\n<div>\n<a>Tatiana Totmianina</a>\n<span>Pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#7</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>1 </span>\n</div>\n<div>Tatiana Ivanovna Totmianina (Russian: Татьяна Ивановна Тотьмянина; born 2 November 1981) is a Russian former competitive pair skater. With partner Maxim Marinin, she is the 2006 Olympic champion, two-time World champion, and five-time European champion. The pair began skating together in 1996 and retired from competition in 2006.</div>\n<ul>\n<li>\n<a>Sportspeople from Perm, Russia</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>113T</span>\n</li>\n<li>\n<a>Russian female pair skaters</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>107T</span>\n</li>\n<li>\n<a>Medalists at the 2006 Winter Olympics</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>454T</span>\n</li>\n</ul>\n</div6>\n<div7>\n<div>\n<a>Vera Bazarova</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#8</b> ·\r\n\t\t\t\t\t\t\t</div>\n<div>Vera Yevgenyevna Bazarova (Russian: Ве́ра Евге́ньевна База́рова, born 28 January 1993) is a Russian pair skater. With former partner Yuri Larionov, she is the 2012–13 Grand Prix Final silver medalist, a three-time European medalist (silver in 2012; bronze in 2011 and 2014), the 2007 World Junior silver medalist, and the 2012 Russian national champion. They won six senior Grand Prix series medals outside the final, including gold at the 2012 NHK Trophy. After their partnership ended, Bazarova competed with Andrei Deputat.</div>\n<ul>\n<li>\n<a>Competitors at the 2015 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>323T</span>\n</li>\n<li>\n<a>Figure skaters at the 2014 Winter Olympics</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>149T</span>\n</li>\n<li>\n<a>Sportspeople from Yekaterinburg</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>187T</span>\n</li>\n</ul>\n</div7>\n<div8>\n<div>\n<a>Anton Sikharulidze</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#9</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n</div>\n<div>Anton Tarielyevich Sikharulidze (Russian: Антон Тариэльевич Сихарулидзе, born 25 October 1976) is a Russian former pair skater. With Elena Berezhnaya, he is the 1998 and 1999 World champion, 1998 Olympic silver medalist and 2002 Olympic champion.</div>\n<ul>\n<li>\n<a>Competitors at the 1998 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>314T</span>\n</li>\n<li>\n<a>Competitors at the 2001 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>345T</span>\n</li>\n<li>\n<a>Figure skaters from Saint Petersburg</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>92T</span>\n</li>\n</ul>\n</div8>\n<div9>\n<div>\n<a>Maria Petrova</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#10</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>1 </span>\n</div>\n<div>Maria Igorevna Petrova (Russian: Мария Игоревна Петрова; born 29 November 1977) is a Russian pair skater. With partner Alexei Tikhonov, she is the 2000 World champion and a two-time (1999, 2000) European champion</div>\n<ul>\n<li>\n<a>Competitors at the 2001 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>345T</span>\n</li>\n<li>\n<a>Figure skaters from Saint Petersburg</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>99T</span>\n</li>\n<li>\n<a>Competitors at the 1997 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>67T</span>\n</li>\n</ul>\n</div9>\n</ul>\n</div>\n</html4>\n<div>\n<div0>\n<div>\n<h1>Skating colleagues remember Sergei Grinkov, 25 years after his death</h1>\n<bsp-page-byline><div>\n<ul>\n<li>By</li>\n<li><a>Lynn Rutherford</a><span>,</span></li>\n</ul>\n<ul>\n<li>By</li>\n<a>Lynn Rutherford</a>\n</ul>\n</div>\n<div>Published\xa0November 20, 2020 11:54 AM</div>\n</bsp-page-byline>\n</div>\n<ul>\n<span>Facebook</span>\n<span>Twitter</span>\n<span>\n        Email\n    </span>\n</ul>\n</div0>\n<div0>\n<div0>\n<div>ZbUp0WDqkYB1</div>\n<div>November 20, 2020 04:04 PM</div>\n<div>Two-time Olympic pairs figure skating champion Ekaterina "Katia" Gordeeva recalls the death of her husband and skating partner, Sergei Grinkov.</div>\n</div0>\n<div1>\n<p0>On Nov. 20, 1995, two-time Olympic pairs champion <b>Sergei Grinkov</b>, aged 28, died of a heart attack in Lake Placid, New York, while practicing with his wife and partner, <b>Ekaterina “Katia” Gordeeva</b>, for the opening of the Stars on Ice tour just five days away.</p0>\n<p8>Gordeeva and Grinkov, who represented the Soviet Union, have an illustrious competitive legacy, winning four world titles (1986, 1987, 1989, 1990) and 1988 Olympic gold before turning professional in late 1990. The couple regained their Olympic eligibility for the 1993-1994 season and won a second Olympic title, that time competing for Russia.</p8><p13>“It was the quality they brought,” Hamilton added. “They were Olympic champions, and Katia was sort of the <b>Nadia [Comăneci]</b> of the 1988 Games. She was so young (16) and beautiful, or adorable rather than beautiful. She captivated a lot of people. It was really fascinating [because] at that time the Soviet Union was a mystery to a lot of people.”</p13><p15>“Sergei had the look, the line, the quality, and he had great feet in lifts,” Hamilton said. “He was a very strong and big man (about 6 feet, 175 pounds), but he skated with this effortless power. It didn’t seem like it was making any noise over the ice. With Katia, she was always sort of floating above the ice, and they sort of meshed [with] each other. There was no heaviness to their skating, it was beautiful and light.”</p15><p17><b>Elena Bechke</b>, the 1992 Olympic silver medalist with <b>Denis Petrov</b>, grew up in the Soviet Union and competed against Gordeeva and Grinkov while skating with an earlier partner. She and Petrov performed with Stars on Ice for many years, and were present in Lake Placid when Grinkov died.</p17><p23>“They were very light, like (three-time world champion) <b>Patrick Chan</b>,” she said. “That is the difference. (Chan) skates like a feather, flowing on the ice. It’s the same talent Katia and Sergei had. They had natural rhythm; that’s why their pair elements were perfect. They had natural harmony… I saw this, and I emphasized this in the choreography I did for them: the lightness, the connection, how they looked at each other, the bodylines.”</p23><p27>“Celebration of a Life” marked Gordeeva’s debut as a singles skater, in programs choreographed by Zoueva to Tchaikovsky’s “Serenade for Strings” and Mahler’s “Adagietto.” Just 24 when Grinkov died, the skater wrote a bestseller (with E.M. Swift) in 1996 about her life with her first husband, “My Sergei,” and enjoyed a decades-long performing career. But in 1995 her professional future, and earning power, was uncertain.</p27><p31>“All of Katia and Sergei was a love story,” Zoueva said. “They started skating (together) when they were around 11 and 14. First, they were very friendly like brother and sister, then they fell in love, then they were married and had a baby. It was always about their love… Sergei was very, very happy. Always laughing, had lots of friends, loved to skate, loved to perform, loved Katia and loved his daughter. There was a lot of love.”</p31><p33>“There is a memorial plaque with several pictures of Katia and Sergei that sits on the wall steps from my office, which is a daily reminder of Sergei,” Wylie said. “Whenever I go to the USA rink, which is where Sergei passed away, I go to the blue line to the right of entrance and I think about that moment. And whenever I stand where the Zamboni comes on, I remember when Elena Bechke ran in and said, ‘Oh my gosh, Sergei fell down.’ It was one of those moments where your life just changes.”</p33>\n</div1>\n</div0>\n</div>\n<html6><title>Meet Kamila Valieva, Olympic Champion 2022 in figure skating in team classification (PHOTOS) - Russia Beyond</title><div1><div0><h1>Meet Kamila Valieva, Olympic Champion 2022 in figure skating in team classification (PHOTOS)</h1><a>\n            Lifestyle\n        </a><time>\n             Feb 08 2022\n        </time><a>\n                    Victoria Ryabikova\n                </a></div0><div3><p0>After the Russian figure skating team won a gold medal, the awarding ceremony scheduled for February 7 was postponed because of unclear reasons. Soon, <a>Inside The Games</a> and other media quoting anonymous sources reported that the ceremony delay was reportedly caused by a positive doping test of Kamila Valieva.</p0><div1><p>Russia\'s Kamila Valiyeva performs a short program in the women\'s single skating at the 6th stage of the Figure Skating World Cup in Sochi</p>\nAlexander Vilf/Sputnik\n            </div1><div3><p>Kamila Valiyeva after performing her free skating program at the Russian Figure Skating Championship in St. Petersburg. December 25, 2021</p>\nAlexei Danichev/Sputnik\n            </div3><p14>In 2019/2020, her first season together with Tutberidze, Kamila’s program was ‘Girl on a Ball’ (short program), based on the painting of the same name by the Spanish artist Pablo Picasso. Then, the program was noticed by Picasso’s granddaughter Diana and personally <a>congratulated </a>the figure skater. She also <a>won</a> with the program at the Junior Grand Prix in France (where she <a>became</a> the fourth junior skater in history to score over 200 points) and <a>then</a> the World Junior Championships. That same season, Kamila performed a free routine to the music of rock band Muse, ‘Exogenesis: Symphony Pt.3 (Redemption)’.</p14><div4><p>Kamila Valiyeva (Russia) at the exhibition performance of the Grand Prix of Figure Skating in Moscow. 22.11.2020 </p>\nAlexander Wilf/Sputnik\n            </div4><p15>The following season, Kamila tried her hand at adult figure skating with the even more difficult ‘Bolero’ program (free skating), but only in Russia.\xa0 Because of the pandemic, all junior championships were canceled that year, but the Russian Figure Skating Federation made an exception for Kamila, who was allowed to compete as an adult. Her main achievement of the season was gold at the Russian Cup. In the finals, she fell while attempting a quadruple salchow and also missed a jump, but still <a>took</a> first place.\xa0</p15><div5><p>Kamila Valieva of Russia prepares in the Junior Ladies Free Skating during the ISU Grand Prix of Figure Skating Final (Senior &amp; Junior) at Palavela Arena on December 06, 2019 in Turin, Italy</p>\nJoosep Martinson/ISU/Getty Images\n            </div5><p18>With the program ‘In Memoriam’ and with ‘Bolero’, the athlete <a>won</a> gold at the international tournament Skate Canada, won gold at the Russian Championship, becoming the main contender to participate in the Olympic Games in Beijing. In January 2022, Kamila <a>won</a> the European Championship, setting a world record in the short program (90.45). By the way, the record for the free program (185.29) and total points (272.71) are also hers and were set at the Rostelecom Cup in November 2021.\xa0</p18><div6><p>Kamila Valieva of Russia performs during Junior Ladies Free Skating during the ISU Junior Grand Prix of Figure Skating Grand Prix de Courchevel at on August 23, 2019 in Courchevel, France</p>\nMaja Hitij/ISU/Getty Images\n            </div6><p21>Valiyeva is <a>called</a> a unique figure skater, due to her elasticity and jumping skills - she is also strong in ligatures, rotations and component scores. In an interview, Kamila said that the secret to her success is repeating her program every day, as well as her coach.\xa0</p21><div7><p>Figure skater Kamila Valiyeva of Russia performs during an exhibition gala at the 2022 ISU European Figure Skating Championships at the Tondiraba Ice Hall</p>\nSergei Bobylev/TASS\n            </div7></div3><div4><a>\n                    sport\n                </a><a>\n                    olympic games\n                </a><a>\n                    figure skating\n                </a><a>\n                    olympic\n                </a></div4></div1></html6>\n<html7><title>Meet the Russian figure skaters of the 2022 Winter Olympics (PHOTOS+VIDEOS) - Russia Beyond</title><div1><div0><h1>Meet the Russian figure skaters of the 2022 Winter Olympics (PHOTOS+VIDEOS)</h1><a>\n            Lifestyle\n        </a><time>\n             Dec 28 2021\n        </time><a>\n                    Victoria Ryabikova\n                </a></div0><div1>\nAlexander Demyanchuk / TASS; Getty Images; Natalia Fedosenko / TASS\n    </div1><div3><div0><p>Saint-Petersburg. The Russian Figure Skating Championship 2022. Sports complex "Jubilee". Kamila Valieva during the demonstration performances.</p>\nAlexander Demyanchuk / TASS\n            </div0><div1><p>Kamila Valieva performs a short program in women\'s single skating at the final of the Russian Figure Skating Cup in Moscow</p>\nVladimir Song / Sputnik\n            </div1><p3>On December 26, 2021, Kamila won the title of Russian Champion in figure skating among adults and became the first participant for the 2022 Winter Olympics in Beijing.\xa0</p3><div2><p> Alexandra Trusova of Russia skates in the Women\'s Free Skate during the ISU Grand Prix of Figure Skating - Skate America at Orleans Arena on October 24, 2021 in Las Vegas, Nevada</p>\nGetty Images\n            </div2><p5>Alexandra is only 17 and the media is already calling her the <a>‘Quad Queen’.</a> And for good reason – Trusova is the first woman in history to have landed a quadruple Lutz in (ISU JGP 2018) and a quadruple flip (ISU GP Final 2019) in competition. She is also the first woman to land a quadruple toe loop, as well as two quadruple jumps (toe loop and Salchow) in one program in competition (at the ISU World Junior Championships 2018). She has also performed three quadruple jumps (quad Lutz, quad toe, quad toe-triple toe) at the Russian test skates in September 2019 and four quadruple jumps (Lutz, Salchow, two toe loops) at the Japan Open in October 2019. In 2018 and 2019, Alexandra became the world junior champion twice.</p5><p6>Born in Ryazan, the skater has two younger brothers and her father is a master in martial arts. From the age of four, Sasha has been interested in skating.\xa0 At first, her parents gave her roller skates, but when they noticed their daughter almost skating on the ceiling, they decided to train her in figure skating. For five years, she skated in Ryazan and then went to Moscow to skate under Eteri Tutberidze.</p6><div3><p>Alexandra Trusova during demonstration performances at the Russian Figure Skating Championships in St. Petersburg</p>\nAlexander Wilf / Sputnik\n            </div3><div4><p>Stockholm. Anna Shcherbakova (Russia) during a performance in the free program of women\'s single skating at the World Figure Skating Championships</p>\nNatalia Fedosenko / TASS\n            </div4></div3><div6><div>\n                        Read more\n                    </div><ul><a>Meet Anna Shcherbakova, Russia’s newest (and most relentless) Figure Skating World Champion (PHOTOS)</a><a>Who are the Russian female figure skaters who filled the winners podium-2021? (PHOTOS + VIDEO)</a><a>Top 5 YOUNG Russian figure skaters off to a promising start (PHOTOS, VIDEO)</a></ul></div6></div1></html7>\n<html8>\n<title>Famous Russian Figure Skaters</title>\n<div0>\n<h1>Famous Russian Figure Skaters</h1>\n<div1>\n<article0><div><div>\xa01\xa0</div><div>Ekaterina Gordeeva<div>(Russian Figure Skater and Olympic Gold Medalist)</div></div></div><div><span>241</span><span>161</span></div><div><div><b>Birthdate: </b>May 28, 1971</div><div><b>Sun Sign: </b>Gemini</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Height: </b>5\'1" (155 cm)</div></div></article0><article1><div><div>\xa02\xa0</div><div>Sergei Grinkov<div>(Russian Figure Skater and Olympic Gold Medalist)</div></div></div><div><span>706</span><span>164</span></div><div><div><b>Birthdate: </b>February 4, 1967</div><div><b>Sun Sign: </b>Aquarius</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Died: </b>November 20, 1995</div><div><b>Height: </b>6\'0" (183 cm)</div></div></article1><article2><div><div>\xa03\xa0</div><div>Alina Zagitova<div>(Russian Figure Skater and Olympic Gold Medalist)</div></div></div><div><span>188</span><span>4972</span></div><div><div><b>Birthdate: </b>May 18, 2002</div><div><b>Sun Sign: </b>Taurus</div><div><b>Birthplace: </b>Izhevsk, Russia</div><p>Alina Zagitova is a Russian figure skater who has won one gold medal each in <em>Olympic Games, World Figure Skating Championships, European Figure Skating Championships</em>, and <em>Grand Prix of Figure Skating Final</em>; she is the only female figure skater from Russia to have achieved this feat. At the 2019 <em>Glamour Russia Awards</em>, Alina Zagitova was named <em>Sportswoman of the Year.</em></p></div></article2><div><span>Recommended Lists: </span> <a>Russia</a> <a>Russian Female Figure Skaters</a> </div><article3><div><div>\xa04\xa0</div><div><a>Yulia Lipnitskaya</a><div>(Skater)</div></div></div><div><span>99</span><span>151</span></div><div><div><b>Birthdate: </b>June 5, 1998</div><div><b>Sun Sign: </b>Gemini</div><div><b>Birthplace: </b>Yekaterinburg, Russia</div><p>Yulia Lipnitskaya stunned her fans with the news of her retirement at the tender age of 19. Apparently, she had been struggling with injuries and anorexia. The Russian professional figure skater won an <strong>Olympic </strong>gold at 15, thus becoming the youngest Russian gold medalist in history.</p></div></article3><article4><div><div>\xa05\xa0</div><div>Evgenia Medvedeva<div>(Former Competitive Russian Figure Skater)</div></div></div><div><span>40</span><span>8437</span></div><div><div><b>Birthdate: </b>November 19, 1999</div><div><b>Sun Sign: </b>Scorpio</div><div><b>Birthplace: </b>Moscow, Russia</div><p>Evgenia Medvedeva is a Russian figure skater who has won two silver <em>Olympic </em>medals, two <em>World Championship </em>gold medals, and two <em>European Championship</em> gold medals. At the 2017 <em>World Championships</em>, Medvedeva became the first 16-year-old female skater after Michelle Kwan to win back-to-back world titles. Evgenia Medvedeva is also the first skater to achieve two <em>Grand Slams</em> in a row.\xa0</p></div></article4><article5><div><div>\xa06\xa0</div><div>Evgeni Plushenko<div>(Russian Former\xa0Figure Skater\xa0and Olympic Gold Medalist)</div></div></div><div><span>464</span><span>591</span></div><div><div><b>Birthdate: </b>November 3, 1982</div><div><b>Sun Sign: </b>Scorpio</div><div><b>Birthplace: </b>Dzhamku, Russia</div><div><b>Height: </b>5\'10" (178 cm)</div></div></article5><article7><div><div>\xa08\xa0</div><div>Kamila Valieva<div>(Russian Figure Skater and Olympic Gold Medalist)</div></div></div><div><span>22</span><span>444</span></div><div><div><b>Birthdate: </b>April 26, 2006</div><div><b>Sun Sign: </b>Taurus</div><div><b>Birthplace: </b>Kazan, Russia</div><div><b>Height: </b>5\'1" (155 cm)</div></div></article7><article8><div><div>\xa09\xa0</div><div>Elizaveta Tuktamysheva<div>(Russian Figure Skater)</div></div></div><div><span>22</span><span>148</span></div><div><div><b>Birthdate: </b>December 17, 1996</div><div><b>Sun Sign: </b>Sagittarius</div><div><b>Birthplace: </b>Glazov, Russia</div></div></article8><article9><div><div>\xa010\xa0</div><div>Anna Shcherbakova<div>(Russian\xa0Figure Skater, Television Presenter, and Olympic Gold Medalist)</div></div></div><div><span>25</span><span>350</span></div><div><div><b>Birthdate: </b>March 28, 2004</div><div><b>Sun Sign: </b>Aries</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Height: </b>5\'2" (157 cm)</div></div></article9><article12><div><div>\xa013\xa0</div><div>Ilia Kulik<div>(Russian Figure Skater)</div></div></div><div><span>19</span><span>142</span></div><div><div><b>Birthdate: </b>May 23, 1977</div><div><b>Sun Sign: </b>Gemini</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Height: </b>5\'11" (180 cm)</div></div></article12><article13><div><div>\xa014\xa0</div><div>Sergei Ponomarenko<div>(Russian Former Competitive Ice Dancer and Figure Skater)</div></div></div><div><span>13</span><span>137</span></div><div><div><b>Birthdate: </b>October 6, 1960</div><div><b>Sun Sign: </b>Libra</div><div><b>Birthplace: </b>Moscow, Russia</div><p>Sergei Ponomarenko is a Russian retired ice dancer who represented the Unified team of the Soviet Union at the 1992 <em>Olympic Games</em>, <em>World Figure Skating Championships</em>, and <em>European Championships</em>, winning gold medals in all three tournaments. He has also represented the Soviet Union in many <em>World Championships </em>and <em>European Championships,</em> winning two and three gold medals respectively.</p></div></article13><article14><div><div>\xa015\xa0</div><div>Adelina Sotnikova<div>(Russian Figure skater)</div></div></div><div><span>22</span><span>137</span></div><div><div><b>Birthdate: </b>July 1, 1996</div><div><b>Sun Sign: </b>Cancer</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Height: </b>5\'5" (165 cm)</div></div></article14><article15><div><div>\xa016\xa0</div><div>Irina Slutskaya<div>(Russian Former Figure Skater)</div></div></div><div><span>17</span><span>141</span></div><div><div><b>Birthdate: </b>February 9, 1979</div><div><b>Sun Sign: </b>Aquarius</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Height: </b>5\'3" (160 cm)</div></div></article15><article17><div><div>\xa018\xa0</div><div>Alexei Yagudin<div>(Former Russian Competitive\xa0Figure Skater\xa0and Olympic Gold Medalist)</div></div></div><div><span>25</span><span>1400</span></div><div><div><b>Birthdate: </b>March 18, 1980</div><div><b>Sun Sign: </b>Pisces</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><div><b>Height: </b>5\'9" (175 cm)</div></div></article17><article18><div><div>\xa019\xa0</div><div>Elena Ilinykh<div>(Russian Former Competitive\xa0Ice Dancer and Olympic Gold Medalist)</div></div></div><div><span>16</span><span>131</span></div><div><div><b>Birthdate: </b>April 25, 1994</div><div><b>Sun Sign: </b>Taurus</div><div><b>Birthplace: </b>Aktau, Kazakhstan</div><p>Best known for her phenomenal performances with partner Ruslan Zhiganshin, retired Russian ice dancer Elena Ilinykh is a former national champion. Though initially trained in single skating, she later moved on to ice dancing. She also has two <strong>Olympic </strong>medals, including a gold, and three European championship medals in her kitty.</p></div></article18><article19><div><div>\xa020\xa0</div><div>Tatiana Volosozhar<div>(Russian Pair Skater)</div></div></div><div><span>16</span><span>133</span></div><div><div><b>Birthdate: </b>May 22, 1986</div><div><b>Sun Sign: </b>Gemini</div><div><b>Birthplace: </b>Dnipro, Ukraine</div><div><b>Height: </b>5\'3" (160 cm)</div></div></article19><article20><div><div>\xa021\xa0</div><div>Daria Usacheva<div>(Russian Figure Skater)</div></div></div><div><span>19</span><span>124</span></div><div><div><b>Birthdate: </b>May 22, 2006</div><div><b>Sun Sign: </b>Gemini</div><div><b>Birthplace: </b>Khabarovsk, Russia</div><div><b>Height: </b>5\'1" (155 cm)</div></div></article20><article21><div><div>\xa022\xa0</div><div>Victoria Sinitsina<div>(Russian\xa0Ice Dancer and Olympic Gold Medalist)</div></div></div><div><span>18</span><span>25</span></div><div><div><b>Birthdate: </b>April 29, 1995</div><div><b>Sun Sign: </b>Taurus</div><div><b>Birthplace: </b>Moscow, Russia</div><p>Victoria Sinitsina is a Russian ice dancer who won the gold medal at the 2021 <em>World Figure Skating Championships</em> along with her skating partner\xa0Nikita Katsalapov. A successful ice dancer, Sinitsina also won gold medals at the 2020 <em>European Championships </em>and 2012 <em>World Junior Championships.</em></p></div></article21><article22><div><div>\xa023\xa0</div><div>Elena Berezhnaya<div>(Former Russian Pair Skater and 1998 and 1999 World Champion)</div></div></div><div><span>17</span><span>130</span></div><div><div><b>Birthdate: </b>October 11, 1977</div><div><b>Sun Sign: </b>Libra</div><div><b>Birthplace: </b>Nevinnomyssk, Russia</div><div><b>Height: </b>5\'1" (155 cm)</div></div></article22><article23><div><div>\xa024\xa0</div><div>Maxim Trankov<div>(Russian\xa0Pair Skater and Olympic Gold Medalist)</div></div></div><div><span>19</span><span>140</span></div><div><div><b>Birthdate: </b>October 7, 1983</div><div><b>Sun Sign: </b>Libra</div><div><b>Birthplace: </b>Perm, Russia</div></div></article23><article24><div><div>\xa025\xa0</div><div>Maria Sotskova<div>(Retired Russian Figure Skater)</div></div></div><div><span>15</span><span>131</span></div><div><div><b>Birthdate: </b>April 12, 2000</div><div><b>Sun Sign: </b>Aries</div><div><b>Birthplace: </b>Reutov, Russia</div><div><b>Height: </b>5\'8" (173 cm)</div></div></article24><article25><div><div>\xa026\xa0</div><div>Fedor Andreev<div>(Canadian Figure Skater and Ice Dancer)</div></div></div><div><span>14</span><span>131</span></div><div><div><b>Birthdate: </b>March 2, 1982</div><div><b>Sun Sign: </b>Pisces</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Height: </b>6\'1" (185 cm)</div></div></article25><article26><div><div>\xa027\xa0</div><div>Ivan Bukin<div>(Russian\xa0Ice Dancer)</div></div></div><div><span>22</span><span>26</span></div><div><div><b>Birthdate: </b>September 16, 1993</div><div><b>Sun Sign: </b>Virgo</div><div><b>Birthplace: </b>Moscow, Russia</div><p>Ivan Bukin is a Russian ice dancer who has won three bronze medals and a silver medal at <em>European Championships </em>along with Alexandra Stepanova. He also won the 2021 Russian national championship. Overall, Bukin and his skating partner Stepanova have won three <em>Finlandia Trophy</em> titles and seven <em>Grand Prix</em> series medals.</p></div></article26><article27><div><div>\xa028\xa0</div><div>Oksana Grishuk<div>(Russian Former Competitive Ice Dancer and Olympic Gold Medalist)</div></div></div><div><span>12</span><span>130</span></div><div><div><b>Birthdate: </b>March 17, 1971</div><div><b>Sun Sign: </b>Pisces</div><div><b>Birthplace: </b>Odesa, Ukraine</div><p>Best known for her performances with partner Evgeni Platov, competitive ice dancer Oksana Grishuk began figure skating at age 4. She and Platov are three-time European champions and four-time world champions. The duo also has also won two <strong>Olympic </strong>golds. She is now a coach and has appeared on several TV shows.</p></div></article27><article28><div><div>\xa029\xa0</div><div>Mikhail Kolyada<div>(Russian Figure Skater)</div></div></div><div><span>151</span><span>266</span></div><div><div><b>Birthdate: </b>February 18, 1995</div><div><b>Sun Sign: </b>Aquarius</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><div><b>Height: </b>5\'6" (168 cm)</div></div></article28><article29><div><div>\xa030\xa0</div><div>Sofia Samodurova<div>(Russian Retired Figure Skater)</div></div></div><div><span>15</span><span>126</span></div><div><div><b>Birthdate: </b>July 30, 2002</div><div><b>Sun Sign: </b>Leo</div><div><b>Birthplace: </b>Krasnoyarsk, Russia</div><div><b>Height: </b>5\'3" (160 cm)</div></div></article29><article30><div><div>\xa031\xa0</div><div>Maria Petrova<div>(Russian\xa0Pair Skater)</div></div></div><div><span>14</span><span>132</span></div><div><div><b>Birthdate: </b>November 29, 1977</div><div><b>Sun Sign: </b>Sagittarius</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><div><b>Height: </b>5\'0" (152 cm)</div></div></article30><article31><div><div>\xa032\xa0</div><div>Evgenia Tarasova<div>(Russian Pair Skater)</div></div></div><div><span>14</span><span>129</span></div><div><div><b>Birthdate: </b>December 17, 1994</div><div><b>Sun Sign: </b>Sagittarius</div><div><b>Birthplace: </b>Kazan, Russia</div><div><b>Height: </b>5\'3" (160 cm)</div></div></article31><article32><div><div>\xa033\xa0</div><div>Anastasia Mishina<div>(Russian Pair Skater)</div></div></div><div><span>11</span><span>7</span></div><div><div><b>Birthdate: </b>April 24, 2001</div><div><b>Sun Sign: </b>Taurus</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><div><b>Height: </b>5\'2" (157 cm)</div></div></article32><article33><div><div>\xa034\xa0</div><div>Tatiana Totmianina<div>(Russian Former Competitive\xa0Pair Skater and Olympic Gold Medalist)</div></div></div><div><span>12</span><span>130</span></div><div><div><b>Birthdate: </b>November 2, 1981</div><div><b>Sun Sign: </b>Scorpio</div><div><b>Birthplace: </b>Perm, Russia</div><div><b>Height: </b>5\'3" (160 cm)</div></div></article33><article35><div><div>\xa036\xa0</div><div>Tatiana Tarasova<div>(Russian Figure Skating Coach)</div></div></div><div><span>14</span><span>325</span></div><div><div><b>Birthdate: </b>February 13, 1947</div><div><b>Sun Sign: </b>Aquarius</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Height: </b>5\'4" (163 cm)</div></div></article35><article36><div><div>\xa037\xa0</div><div>Ludmila Belousova<div>(Russian\xa0Pair Skater\xa0and Two-Time Olympic Gold Medalist)</div></div></div><div><span>9</span><span>128</span></div><div><div><b>Birthdate: </b>November 22, 1935</div><div><b>Sun Sign: </b>Sagittarius</div><div><b>Birthplace: </b>Ulyanovsk, Russia</div><div><b>Died: </b>September 26, 2017</div><div><b>Height: </b>5\'3" (160 cm)</div></div></article36><article37><div><div>\xa038\xa0</div><div>Alexandra Stepanova<div>(Russian Ice Dancer and Figure Skater)</div></div></div><div><span>11</span><span>13</span></div><div><div><b>Birthdate: </b>August 19, 1995</div><div><b>Sun Sign: </b>Leo</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><p>Alexandra Stepanova is a Russian ice dancer who has won three bronze medals and a silver medal at <em>European Championships </em>with Ivan Bukin. She also won the 2018<em> Grand Prix of Helsinki</em> championship as well as the 2021 Russian national championship. Overall, Stepanova and her skating partner Bukin have won three <em>Finlandia Trophy</em> titles and seven <em>Grand Prix</em> series medals.</p></div></article37><article38><div><div>\xa039\xa0</div><div>Stanislava Konstantinova<div>(Russian Figure Skater)</div></div></div><div><span>14</span><span>122</span></div><div><div><b>Birthdate: </b>July 14, 2000</div><div><b>Sun Sign: </b>Cancer</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><div><b>Height: </b>5\'6" (168 cm)</div></div></article38><article40><div><div>\xa041\xa0</div><div>Alexei Mishin<div>(Russian Figure Skating Coach and Former Pair Skater)</div></div></div><div><span>144</span><span>9</span></div><div><div><b>Birthdate: </b>March 8, 1941</div><div><b>Sun Sign: </b>Pisces</div><div><b>Birthplace: </b>Sevastopol, Russia</div></div></article40><article41><div><div>\xa042\xa0</div><div>Dmitri Aliev<div>(Russian Figure Skater)</div></div></div><div><span>15</span><span>101</span></div><div><div><b>Birthdate: </b>June 1, 1999</div><div><b>Sun Sign: </b>Gemini</div><div><b>Birthplace: </b>Ukhta, Russia</div><div><b>Height: </b>5\'8" (173 cm)</div></div></article41><article43><div><div>\xa044\xa0</div><div>Maya Khromykh<div>(Russian Figure Skater)</div></div></div><div><span>15</span><span>12</span></div><div><div><b>Birthdate: </b>May 25, 2006</div><div><b>Sun Sign: </b>Gemini</div><div><b>Birthplace: </b>Nizhny Tagil, Russia</div><div><b>Height: </b>5\'5" (165 cm)</div></div></article43><article44><div><div>\xa045\xa0</div><div>Alexander Zhulin<div>(Russian Ice Dancer, Figure Skater and Coach)</div></div></div><div><span>24</span><span>9</span></div><div><div><b>Birthdate: </b>July 20, 1963</div><div><b>Sun Sign: </b>Cancer</div><div><b>Birthplace: </b>Moscow, Russia</div><p>Currently a popular ice dancing coach, Alexander Zhulin was also a renowned figure skater. Paired with Maya Usova, he not only won bronze and silver in the Olympics, but also several other competitions including the World and the European Championships. Retired in 1994, he now serves as a coach and choreographer. He has also\xa0appeared\xa0in movies like <em>Muzhchina v moey golove.</em></p></div></article44><article45><div><div>\xa046\xa0</div><div>Anton Sikharulidze<div>(Russian Former Pair Skater)</div></div></div><div><span>8</span><span>3</span></div><div><div><b>Birthdate: </b>October 25, 1976</div><div><b>Sun Sign: </b>Scorpio</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><div><b>Height: </b>6\'0" (183 cm)</div></div></article45><article46><div><div>\xa047\xa0</div><div>Anastasia Tarakanova<div>(Russian Retired Figure Skater)</div></div></div><div><span>14</span><span>22</span></div><div><div><b>Birthdate: </b>April 14, 2004</div><div><b>Sun Sign: </b>Aries</div><div><b>Birthplace: </b>Volgograd, Russia</div><div><b>Height: </b>5\'1" (155 cm)</div></div></article46><article47><div><div>\xa048\xa0</div><div>Kira Ivanova<div>(Russian Figure Skater)</div></div></div><div><span>9</span><span>16</span></div><div><div><b>Birthdate: </b>January 10, 1963</div><div><b>Sun Sign: </b>Capricorn</div><div><b>Birthplace: </b>Moscow, Russia</div><div><b>Died: </b>December 18, 2001</div><div><b>Height: </b>5\'3" (160 cm)</div></div></article47><article48><div><div>\xa049\xa0</div><div>Nikolai Morozov<div>(Russian Former Competitive Ice Dancer, Figure Skating Coach and Choreographer)</div></div></div><div><span>14</span><span>148</span></div><div><div><b>Birthdate: </b>December 17, 1975</div><div><b>Sun Sign: </b>Sagittarius</div><div><b>Birthplace: </b>Moscow, Russian SFSR, Soviet Union</div><p>Nikolai Morozov is a Russian retired ice dancer, choreographer, and figure skating coach. Morozov represented Belarus at the 1998 <em>Winter Olympics</em> and <em>World Championships.</em> He is credited with coaching Japanese figure skater Shizuka Arakawa at the 2006 <em>Olympics </em>where she won the gold medal. Nikolai Morozov also coached Miki Ando to a couple of world titles.</p></div></article48><article49><div><div>\xa050\xa0</div><div>Tamara Moskvina<div>(Russian Pair Skating Coach and Former Competitive Skater)</div></div></div><div><span>8</span><span>2</span></div><div><div><b>Birthdate: </b>June 26, 1941</div><div><b>Sun Sign: </b>Cancer</div><div><b>Birthplace: </b>Saint Petersburg, Russia</div><div><b>Height: </b>4\'10" (147 cm)</div></div></article49>\n</div1>\n</div0>\n</html8>\n<html9>\n<title>‘Beautiful’ Russian pairs figure skaters strike silver at Olympics — RT Sport News</title>\n<div><h1>\n        ‘Beautiful’ Russian pairs figure skaters strike silver at Olympics\n    </h1><p0>Russian pairs figure skaters Evgenia Tarasova and Vladimir Morozov have been hailed for a<em> "beautiful"</em> showing at the Winter Olympics as the duo who are part of coach\xa0Eteri Tutberidze\'s group came within a\xa0whisker of gold at the Beijing Games.</p0>\n</div>\n</html9>\n<html10><title>Russian figure skater Sofya Akatyeva on quads, Beijing 2022 and her future</title>\n<section0><section0><section0><p>Russia\'s newest <a>figure skating</a> rising star, <strong>Sofya Akatyeva</strong>, was thrust into the limelight after becoming the first female skate to land a <a>triple Axel alongside two quadruple jumps</a> in a single program.</p>\n<p>Akatyeva achieved the feat last week at the <strong>Junior Cup of Russia</strong> stop in Kazan, receiving a score <strong>higher than</strong> that of European champion <strong>Alena Kostornaia</strong>\'s in the <a>senior event</a>. She landed two <strong>quadruple toe loops</strong> (one in combination) in addition to the <strong>triple Axel</strong>.</p>\n<p>Speaking exclusively to Olympic Channel a few days after her historic skate, she frankly stated: <strong>"I love to jump quads"</strong>. The <strong>Eteri Tutberidze</strong>-trained athlete, who <strong>aged 13</strong> remains ineligible for senior international competition until the 2023/24 season, has her eyes set on big things.</p>\n<p>There is one more stop on the Cup of Russia circuit in Moscow, with the best performers qualifying for the senior and junior Russian Nationals.</p>\n<em>The interview below has been translated from Russian and edited for clarity and length.</em></section0>\n<p0><strong>Sofya Akatyeva (SA):</strong> Yesterday in the free program, the first half I skated quite well and in the second half I made very rough mistakes. I think this is because I did not work enough in training. That\'s for sure. I will work more in the second half at the next competition, I hope to skate well.</p0>\n<h2>Akatyeva on herself and figure skating</h2>\n<p3><strong>SA:</strong> In my training group I have <a>Alina Zagitova</a> and Anna Shcherbakova, their results motivate me to work. Their skating is so amazing, Anya (Shcherbakova) does such difficult jumps. It motivates me very much.</p3>\n</section0><div><h2>Related content</h2><h3>Packed women’s field highlights figure skating at Rostelecom Cup</h3><h3>Back injury sidelines Evgenia Medvedeva on eve of Rostelecom Cup</h3><h3>Is Sofya Akatyeva Russia\'s next figure skating superstar?</h3><h3>Zagitova, Fernández thrill in Moscow exhibition as fans return to figure skating</h3></div></section0></html10>\n<html11><title>Famous Figure Skaters from Russia | List of Top Russian Figure Skaters</title><div><div1><div><div><span>COLLECTION</span><span>58 LISTS</span></div><a>To Russia, With Love</a></div><ul><span>Must-See in Russia</span><span>Must-See in Moscow</span><span>Beers</span><span>Tennis Players</span><span>Actors</span><span>Soccer Players</span><span>Pop Artists</span><span>Authors</span><span>Pianists</span><span>Boxers</span><span>Vladamir Lenin Quotes</span><span>TV Channels</span><span>Bands</span><span>Books About Stalin</span><span>Movies</span><span>Politicians</span><span>Female Athletes</span><span>Basketball Players</span><span> Violinists</span><span>Artists</span><span>Hockey Players</span><span>Photographers</span><span>Gymnasts</span><span>Philosophers</span><span>Olympic Athletes</span><span>Guitar Players</span><span>Film Directors</span><span>Writers</span><span>Mathematicians</span><span>Conductors</span><span>Playwrights</span><span>Psychologists</span><span>Film Actors</span><span>Athletes</span><span>Journalists</span><span>Volleyball Players</span><span>Astronauts</span><span>Famous People</span><span>Architects</span><span>Russian Olympic Athletes</span><span>Best Russians</span><span>Scientists</span><span>Chemists</span><span>Physicists</span><span>Figure Skaters</span><span>Voice Actors</span><span>Physicians</span><span>Painters</span><span>Engineers</span><span>Guitarists</span><span>Composers</span><span>TV Producers</span><span>Poets</span><span>Track and Field Athletes</span><span>Cross-Country Skiers</span><span>Screenwriters</span><span>Animators</span><span>Novelists</span></ul></div1><article><div0><a>People</a><div><span>Famous Figure Skaters from Russia</span><div><a>Reference</a><div><span>Updated June 15, 2019</span><span> 18 items</span></div></div></div></div0><div1><p>List of notable or famous figure skaters from Russia, with bios and photos, including the <a>top figure skaters</a> <a>born in Russia</a> and even some popular figure skaters who immigrated to Russia. If you\'re trying to find out the names of famous Russian figure skaters then this list is the perfect resource for you. These figure skaters are among the most prominent in their field, and information about each well-known figure skater from Russia is included when available. </p><p>The list you\'re viewing is made up of people like Jana Khokhlova and Maxim Shabalin. </p><p>This historic figure skaters from Russia list can help answer the questions "Who are some Russian figure skaters of note?" and "Who are the <a>most famous figure skaters</a> from Russia?" These prominent figure skaters of Russia may or may not be currently alive, but what they all have in common is that they\'re all respected Russian figure skaters.</p>Use this list of renowned Russian figure skaters to discover some new figure skaters that you aren\'t familiar with. Don\'t forget to share this list by clicking one of the social media icons at the top or bottom of the page.\xa0</div1><ul><li0><li>Photo: <ul><a>Metaweb (FB)</a><a>CC-BY-SA</a></ul></li><a>Alena Leonova</a><span>Alena Igorevna Leonova (Russian: Алёна Игоревна Леонова; born 23 November 1990) is a Russian figure skater. She is the 2012 World silver medalist, the 2011 Grand Prix Final bronze medalist, the 2009 World Junior champion, and a three-time (2010–2012) Russian national medalist. She is also the 2014–15 ISU Challenger Series runner-up.</span></li0><li1><li>Photo: <ul><a>Metaweb (FB)</a><a>GNU Free Documentation License</a></ul></li><a>Alexander Enbert</a><span>Alexander Yuryevich Enbert (Russian: Александр Юрьевич Энберт; born 17 April 1989) is a Russian pair skater. With Natalya Zabiyako, he is the 2019 World bronze medalist, 2018 European bronze medalist, 2018 Grand Prix of Helsinki champion, 2018 NHK Trophy champion, 2016 Rostelecom Cup silver medalist, and three-time Russian national medalist.\nFrom 2010 to 2014, he competed with Katarina Gerboldt. The pair placed fourth at the 2011 European Championships.</span></li1><li2><li>Photo: <ul><a>Metaweb (FB)</a><li>Public domain</li></ul></li><a>Alexander Smirnov</a><span>Alexander Viktorovich Smirnov (Russian: Александр Викторович Смирнов, born 11 October 1984) is a Russian pair skater. Smirnov teamed up with Yuko Kavaguti in May 2006. They are two-time European champions (2010, 2015), two-time World bronze medalists (2009, 2010), two-time ISU Grand Prix Final bronze medalists (11–12 and 15–16), and three-time Russian national champions (2008–2010). In 2015, they became the first pair in history to complete two quadruple throw jumps in one program and the first to land a quadruple throw loop</span></li2><li3><li>Photo: <ul><a>Metaweb (FB)</a><a>CC-BY-SA-2.5</a></ul></li><a>Andrei Lutai</a><span>Andrei Vladimirovich Lutai (Russian: Андрей Владимирович Лутай; born 24 July 1986) is a Russian former competitive figure skater. He is the 2006 Karl Schäfer Memorial champion, 2006 International Cup of Nice champion, and a three-time Russian national medalist (2007 &amp; 2008 silver; 2009 bronze). He placed as high as 5th at the European Championships (2007) and 10th at the World Championships (2009).</span></li3><li5><li>Photo: <ul><a>Metaweb (FB)</a><a>CC-BY-SA-2.5</a></ul></li><a>Igor Lukanin</a><span>Igor Lukanin (born 3 February 1976) is a former competitive ice dancer who competed internationally for Azerbaijan with Kristin Fraser.\nHe previously competed for Germany with Ksenia Smetanenko. He competed with Jenny Dahlen for two years, representing Azerbaijan. He teamed up with Kristin Fraser in 2000. They are four-time Azerbaijani national champions and have competed at the 2002 Winter Olympics and the 2006 Winter Olympics.\nLukanin was married to Daria Timoshenko from 2000 to 2005. He married Kristin Fraser on December 31, 2010.\nLukanin was the flag bearer for Azerbaijan at the 2006 Winter Olympics.</span></li5></ul></article></div></html11>\n<html12>\n<title>List of Russian pair skaters - FamousFix List</title>\n<div>\n<div0>\n<div>\n<h1>Russian pair skaters</h1>\n<div>The list "Russian pair skaters" has been viewed 262 times.</div>This list has <i>2 sub-lists</i> and <i>141 members</i>. See also <a>Russian figure skaters</a>, <a>Pair skaters</a>\n</div>\n<div>\r\nFLAG\r\n</div>\n<div>204 views · <span>\r\nLike \n</span></div>\n</div0>\n<div1>\n<a>Russian female pair skaters</a>\r\n\t\t\t\t\t\t\t\t116 T\r\n\t\t\t\t\t\t\t</div1>\n<div2>\n<a>Russian male pair skaters</a>\r\n\t\t\t\t\t\t\t\t102 T\r\n\t\t\t\t\t\t\t</div2>\n<ul>\n<div0>\n<div>\n<a>Ekaterina Gordeeva</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#1</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>36 </span>\n<span>3 </span>\n<span>2 </span>\n</div>\n<div>Ekaterina "Katia" Alexandrovna Gordeeva (Russian: Екатерина Александровна Гордеева; born 28 May 1971) is a Russian former figure skater. Together with her husband, the late Sergei Grinkov, she was the 1988 and 1994 Olympic Champion and four-time World Champion (1986, 1987, 1989, 1990) in pair skating. After Grinkov\'s death, Gordeeva continued performing as a singles skater.</div>\n<ul>\n<li>\n<a>Figure skaters from Moscow</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>232T</span>\n</li>\n<li>\n<a>Competitors at the 1990 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>351T</span>\n</li>\n<li>\n<a>Russian State University of Physical Education, Sport, Youth and Tourism alumni</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>49T</span>\n</li>\n</ul>\n</div0>\n<div1>\n<div>\n<a>Irina Rodnina</a>\n<span>Russian politician and figure skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#2</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>49 </span>\n</div>\n<div>Irina Konstantinovna Rodnina (Russian: Ирина Константиновна Роднина, born 12 September 1949) is a Russian politician and figure skater, who is the only pair skater to win 10 successive World Championships (1969–78) and three successive Olympic gold medals (1972, 1976, 1980). She was elected to the State Duma in the 2007 legislative election as a member of President Vladimir Putin\'s United Russia party. As a figure skater, she initially competed with Alexei Ulanov and later teamed up with Alexander Zaitsev. She is the first pair skater to win the Olympic title with two different partners, followed only by Artur Dmitriev.</div>\n<ul>\n<li>\n<a>Eighth convocation members of the State Duma (Russian Federation)</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>453T</span>\n</li>\n<li>\n<a>Recipients of the Order of the Red Banner of Labour</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>1,697T</span>\n</li>\n<li>\n<a>Figure skaters from Moscow</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>272T</span>\n</li>\n</ul>\n</div1>\n<div2>\n<div>\n<a>Yelena Berezhnaya</a>\n<span>Pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#3</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n</div>\n<div>Elena Viktorovna Berezhnaya (Russian: Елена Викторовна Бережная, born 11 October 1977) is a Russian former pair skater. With partner Anton Sikharulidze, she is the 1998 and 1999 World champion, 1998 Olympic silver medalist and 2002 Olympic champion.</div>\n<ul>\n<li>\n<a>Competitors at the 1998 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>314T</span>\n</li>\n<li>\n<a>Competitors at the 2001 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>345T</span>\n</li>\n<li>\n<a>Sportspeople from Stavropol Krai</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>47T</span>\n</li>\n</ul>\n</div2>\n<div4>\n<div>\n<a>Sergei Grinkov</a>\n<span>Russian pair skater (1967–1995)</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#5</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n</div>\n<div>Sergei Mikhailovich Grinkov (Russian: Серге́й Миха́йлович Гринько́в; 4 February 1967 — 20 November 1995) was a Russian pair skater. Together with partner and wife Ekaterina Gordeeva, he was the 1988 and 1994 Olympic Champion and a four-time World Champion (1986, 1987, 1989, 1990).</div>\n<ul>\n<li>\n<a>Figure skaters from Moscow</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>232T</span>\n</li>\n<li>\n<a>Competitors at the 1990 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>351T</span>\n</li>\n<li>\n<a>Russian State University of Physical Education, Sport, Youth and Tourism alumni</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>49T</span>\n</li>\n</ul>\n</div4>\n<div5>\n<div>\n<a>Vasilisa Davankova</a>\n<span>Russian Ice dancer</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#6</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n</div>\n<div>Vasilisa Alexandrovna Davankova (Russian: Василиса Александровна Даванкова; born 2 May 1998) is a Russian YouTuber and a former Ice dancer and pair skater. With former partner Andrei Deputat, she was the 2012 World Junior bronze medalist, the 2012–13 JGP Final silver medalist, and the 2012 Russian Junior champion. She then briefly teamed up with Alexander Enbert but ended their partnership in 2015.</div>\n<ul>\n<li>\n<a>Figure skaters from Moscow</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>241T</span>\n</li>\n<li>\n<a>Competitors at the 2017 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>220T</span>\n</li>\n<li>\n<a>Russian YouTubers</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>64T</span>\n</li>\n</ul>\n</div5>\n<div6>\n<div>\n<a>Ksenia Stolbova</a>\n<span>Figure skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#7</b> ·\r\n\t\t\t\t\t\t\t</div>\n<div>Ksenia Andreyevna Stolbova (Russian: Ксе́ния Андре́евна Столбо́ва; born 7 February 1992) is a retired Russian pair skater who skated with Andrei Novoselov and Fedor Klimov. She is the 2014 Olympic silver medalist, the 2014 Olympic champion in the team event, the 2014 World silver medalist, a three-time European medalist (2012 bronze, 2014, 2015 silver), the 2015–16 Grand Prix Final champion, the 2013 Winter Universiade champion, a two-time World Junior medalist (2010 bronze, 2011 silver), and a three-time Russian national champion (2014, 2015, 2017). Stolbova announced her retirement from competitions on 12 February 2020.</div>\n<ul>\n<li>\n<a>Figure skaters from Saint Petersburg</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>99T</span>\n</li>\n<li>\n<a>Competitors at the 2013 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>254T</span>\n</li>\n<li>\n<a>Universiade gold medalists for Russia</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>495T</span>\n</li>\n</ul>\n</div6>\n<div7>\n<div>\n<a>Tatiana Volosozhar</a>\n<span>Russian-Ukrainian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#8</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>1 </span>\n</div>\n<div>Tatiana Andreyеvna Volosozhar (Russian: Татья́на Андре́евна Волосожа́р, Ukrainian: Тетяна Андріївна Волосожар; born 22 May 1986) is a Ukrainian-born Russian pair skater. With Maxim Trankov, she is the two-time 2014 Olympic champion in the pairs and in team events, the 2013 World champion, a four-time (2012, 2013, 2014 and 2016) European champion, the 2012 Grand Prix Final champion, and a three-time (2011, 2013 and 2016) Russian national champion. They have also won six events on the Grand Prix series.</div>\n<ul>\n<li>\n<a>Universiade silver medalists for Ukraine</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>146T</span>\n</li>\n<li>\n<a>Season\'s world number one figure skaters</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>32T</span>\n</li>\n<li>\n<a>Season-end world number one figure skaters</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>57T</span>\n</li>\n</ul>\n</div7>\n<div8>\n<div>\n<a>Maxim Trankov</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#9</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>1 </span>\n</div>\n<div>Maxim Leonidovich Trankov (Russian: Макси́м Леони́дович Транько́в, born 7 October 1983) is a Russian pair skater. With Tatiana Volosozhar, he is a two-time  2014 Olympic champion in the pairs and in team events, the 2013 World champion, a four-time (2012, 2013, 2014, 2016) European champion, the 2012 Grand Prix Final champion, and a three-time (2011, 2013, 2016) Russian national champion. They have also won six events on the Grand Prix series.</div>\n<ul>\n<li>\n<a>Competitors at the 2005 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>131T</span>\n</li>\n<li>\n<a>Universiade bronze medalists for Russia</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>263T</span>\n</li>\n<li>\n<a>Season\'s world number one figure skaters</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>31T</span>\n</li>\n</ul>\n</div8>\n<div9>\n<div>\n<a>Yelena Bechke</a>\n<span>Russian pair skater</span>\n</div>\n<span>\n<span>\n\xa0<span>0</span>\n</span>\n<span>\n\xa0<span>0</span>\n</span>\n</span>\n<div>\r\n\t\t\t\t\t\t\t\t\trank <b>#10</b> ·\r\n\t\t\t\t\t\t\t\t\t<span>WDW</span>\n<span>1 </span>\n</div>\n<div>Elena Yurievna Bechke (Russian: Елена Юрьевна Бечке; born 7 January 1966) is a Russian former pair skater. She competed for the Soviet Union, the Commonwealth of Independent States, and the Unified Team. With partner and then-husband Denis Petrov, she is the 1992 Olympic silver medalist, the 1989 World bronze medalist, and the 1991 and 1992 European silver medalist.</div>\n<ul>\n<li>\n<a>Figure skaters from Saint Petersburg</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>116T</span>\n</li>\n<li>\n<a>Competitors at the 1990 Goodwill Games</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>355T</span>\n</li>\n<li>\n<a>Competitors at the 1987 Winter Universiade</a>\r\n\t\t\t\t\t\t\t\t\t\t\t\t· <span>32T</span>\n</li>\n</ul>\n</div9>\n</ul>\n</div>\n</html12>\n<div0>\n<div><strong0>Ekaterina Gordeeva is a professional figure skater from Russia, currently living in Los Angeles, with ethereal beauty and soulful athleticism, a two-time <em>Olympic Champion</em>\xa0and a four-time <em>World Champion\xa0</em>in pair skating. Ekaterina and her daughter Daria have also appeared in a few documentaries and movies.</strong0>\n<p0>Her beloved lover and husband, <strong>Sergei Grinkov</strong>, was an unbeatable pair with his wife in the Olympics pair-skating.</p0>\n<strong1>\xa0EKATERINA GORDEEVA Olympic skating champion Ekaterina Gordeeva poses for a portrait (Credit: Photo by Susan Sterner/AP/Shutterstock )</strong1>\n<p3>Ekaterina started skating when she was only four and finally retired <em>37 years</em> later. Her Father was a professional dancer and directed her toward figure skating.</p3>\n<p5>The champion figure skater has married <strong>thrice</strong> and has a daughter from each marriage. She welcomed her first child with her first husband almost three years before he died.</p5>\n<span9>Ekaterina Gordeeva | Professional Figure Skating Career</span9>\n<span11>The two-time Olympic Champion started her figure skating career early. At four, she joined the Children and Youth Sports School in a significant Russian club <em>CSKA Moscow.</em> </span11>\n<span12>She was so small, so she wore multiple socks to fit her skates. Initially, she was paired with her first husband, <strong>Sergei Grinkov</strong>.</span12>\n<strong3>Ekaterina Gordeeva With Her Dance Partner</strong3>\n<p14>However, she disagreed, and instead of changing her partner, she changed her coach. The duo has been partners since Katina was <em>11 years old</em>\xa0and Sergei was <em>15.</em></p14>\n<span14>Later, in <em>1985</em> and <em>1986,</em> the pair won annual figure skating competitions like the <em>World</em> <em>Junior Figure Skating Championships</em> and the <em>World Figure Skating Championships</em>. Moreover, they held the title in <em>1987,\xa01989,</em> and <em>1990.</em> </span14>\n<strong4>The Four-Time World Champion Figure Skating</strong4>\n<span20>The figure skater started her solo career in <em>1996</em> after Sergei passed away suddenly on November 20, 1995, due to a heart attack. </span20>\n<span29>Ekaterina married her skating partner and first love, <strong>Sergei Grinkov</strong>. The pair met when Gordeeva was <em>11</em>\xa0and Grinkov was <em>15</em> through their passion for figure skating.</span29>\n<span36>Gordeeva met her second husband, <strong>Ilia Kulik,</strong> through figure skating too. They started their romance during the <em>1999-2000</em> season of Stars on Ice and even performed a romantic duet. </span36>\n<strong5>The Figure Skater With Ex-Husband Ilia Kulik And Daughter Elizaveta Kulik</strong5>\n<div>Ekaterina Gordeeva is married to Canadian figure skater <strong>David Pelletier</strong>. The couple got married on <strong>July 25,</strong><em><strong>\xa02020</strong>.</em></div>\n<span44>The exceptional figure skater has built a whopping net worth after almost <em>22 years</em> of a professional career. </span44>\n<h31>Who was Ekaterina Gordeeva’s partner at Battle of the Blades?</h31>\n<p32>Yes, Ekaterina Gordeeva was paired with hockey player Valeri Bure in the figure skating reality show “<em>Battle of the Blades” </em>in 2010.</p32>\n</div>\n<article2>\n<footer>\n<cite>Nancy Miller</cite>\n<time>\nFebruary 20, 2021 At 11:55 am</time>\n</footer>\n<p>This is an interesting article with some great facts and photos. HOWEVER, the very first photo (in the dark red box) is not a picture of Katia. I don’t know how a photo of a different woman got into this article, but it definitely is some other woman. I have been following Katia’s career for many years and I know her distinctive features. The woman in the red box is very attractive, and about the same age, but she is NOT Ekaterina Gordeeva.</p>\n<a>Reply</a>\n</article2>\n</div0>\n<html14>\n<title>15 Most Famous Russian Figure Skaters of All Time | Stadium Talk</title>\n<article>\n<header><h1>15 Most Famous Russian Figure Skaters of All Time</h1><div>Russian figure skaters have a reputation for over-the-top training practices, but their results are hard to argue with.</div></header><section0><h2>The Most Famous Russian Figure Skaters of All Time</h2><p>Of all the Olympic sports, Russia is known for dominating one in particular: figure skating. Russian figure skaters are trained to operate like athletic machines before they\'re old enough to attend kindergarten.\xa0</p><p>While their methods are questionable, Russia\'s churned out many memorable figure skating champions. Which ones <em>were</em> the most memorable? Here are the most popular to date.</p></section0><section1><h2>Sergei Ponomarenko</h2><p><strong>Date of birth:</strong> Oct. 6, 1960</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> Olympic Champion, three-time World Champion</p><p><strong>Bottom Line:</strong> Sergei Ponomarenko and his skating partner and wife Marina Klimova made waves during the 1980s and \'90s as ice dancers. They won four European titles in a row, and they had one of the longest careers at the time, spanning over a decade. Ponomarenko was one of the oldest skating champions, delaying retirement until he was in his early 30s.</p><p>Following retirement, he and Klimova were both inducted into the World Figure Skating Hall of Fame, and Ponomarenko remains an ISU technical specialist for Russia. Both of them now coach in San Jose, California, and one of their sons, Anthony Ponomarenko, is now a competitive ice dancer.\xa0</p></section1><section2><h2>Ilia Kulik</h2><p><strong>Date of birth:</strong> May 23, 1977</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> First figure skater to win Olympic gold with a quadruple jump</p><p><strong>Bottom Line:</strong> Ilia Kulik was 20 when he won a gold medal at the 1998 Winter Olympics in Nagano, Japan. He was only the second man in the sport\'s history to win gold on his first try. He started skating at 5 and was initially more on the clumsy side. But his coach noted Kulik\'s refusal to give up and encouraged him to continue lessons.</p><p>Kulik went on to win two Russian national championships, followed by a stunning Olympic debut with one of the most challenging programs ever successfully skated at that time. It included a quadruple toe loop and eight triples. Later in life, he became a distinguished choreographer, touring with both "Champions on Ice" and "Stars on Ice."\xa0</p></section2><section3><h2>Adelina Sotnikova</h2><p><strong>Date of birth:</strong> July 1, 1996</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> Four-time Russian National Champion</p><p><strong>Bottom Line:</strong> Adelina Sotnikova isn\'t the most famous Russian figure skater, but she had a loyal fanbase throughout the 2010s. She started skating when she was just 4 at a rink near her home in Moscow. Just eight years later, she won gold at the senior-level Russian Nationals.</p><p>She was too young to compete on the Junior Grand Prix circuit that season, but she continued working her way up, eventually winning Olympic gold in 2014. She had a string of lingering injuries, eventually retiring due to chronic pain.\xa0</p><strong>RELATED: Meet Ilia Malinin, the \'Quad God\' of Figure Skating</strong></section3><section4><h2>Alexei Yagudin</h2><p><strong>Date of birth:</strong> March 18, 1980</p><p><strong>Birthplace:</strong> Saint Petersburg, Russia</p><p><strong>Biggest achievement:</strong> Only skater to have achieved a Golden Slam</p><p><strong>Bottom Line:</strong> One of the most memorable Russian figure skaters of all time, Alexei Yagudin is the only skater to have won a Golden Slam – a victory in every major figure skating championship in the same season. This includes the Olympics, World Championships, European Championships, Grand Prix assignments and Grand Prix final. He was primed for success from the start, landing all his double jumps by age 10 and five triples before the age of 12.\xa0</p><p>He was awarded the Order of Merit by the Russian Federation in 2003 and joined the figure skating Hall of Fame in 2017. Since then, he\'s worked as an actor, TV host and sports commentator.</p></section4><section5><h2>Irina Slutskaya</h2><p><strong>Date of birth:</strong> Feb. 9, 1979</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> Two-time World Champion</p><p><strong>Bottom Line:</strong> Irina Slutskaya had a rocky road in figure skating, but her athleticism and innovation set her performances apart from the rest. After initial success in the sport, she hit a plateau in her skating career, failing to qualify for both the European and World championships. Instead of throwing in the towel, she continued training.\xa0</p><p>It\'s a good thing she did because she went on to beat Michelle Kwan at the 2000 Grand Prix Final, plus won a silver medal at the 2002 Winter Olympics in Salt Lake City, Utah. She actually tied for gold, but Sara Hughes of the U.S. took the gold in a tiebreaker free skate.</p><p>Slutskaya was known for her power on the ice, making history as the first woman to land a triple-Lutz-triple-loop combination. She was also known for her signature double Biellmann spin, which she made up herself.\xa0</p></section5><section6><h2>Alexandra Trusova</h2><p><strong>Date of birth:</strong> June 23, 2004</p><p><strong>Birthplace:</strong> Ryazan, Russia</p><p><strong>Biggest achievement:</strong> First female to land two quads in a free skate</p><p><strong>Bottom Line:</strong> Alexandra Trusova is one of Russia\'s favorite figure skaters at the moment. She was the Beijing 2022 Olympic silver medalist, and quads are her claim to fame. She\'s the first female to land the quad Lutz, flip and toe loop jumps in competition and was the first to land two, then three quads in a free skate.\xa0</p><p>She has three Grand Prix Champion titles under her belt, and she\'s expected to continue on her quad-packed career path at the next Olympic games.</p></section6><section8><h2>Anna Shcherbakova</h2><p><strong>Date of birth:</strong> March 28, 2004</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> First woman to land a quad Lutz in competition</p><p><strong>Bottom Line:</strong> Anna Scherbakova is still young, and she\'s one of the most promising Russian figure skaters in competition. She was the second figure skater to perform two quadruple Lutz jumps in one program, following American skater Nathan Chen. She\'s also the first to land a quad flip in combination with a triple.\xa0</p><p>She gets her inspiration from retired Japanese figure skater Mao Asada, and she loves caring for her pets in her limited free time, including a dog named Sandy, a street cat named Mafia and a Maine coon cat named Shiny.</p><p><strong>RELATED: 32 Best Male Figure Skaters of All Time, Ranked</strong>\xa0</p></section8><section9><h2>Evgeni Plushenko</h2><p><strong>Date of birth:</strong> Nov. 3, 1982</p><p><strong>Birthplace:</strong> Dzhamku, Russia</p><p><strong>Biggest achievement:</strong> Earned four Olympic medals</p><p><strong>Bottom Line:</strong> If you haven\'t heard the name Plushenko, you\'re probably not a figure skating fan. Plushenko was one of the biggest names in figure skating in the early 2000s, dominating the competition between 2003 and 2006. He was trying quads before it was popular, but his artistry was what made him so memorable.\xa0</p><p>Despite several injuries, including a back injury resulting in surgery, he continued competing all the way up until 2014. He was crowned European champion for the seventh time in 2012 and won a fourth Olympic medal at the Sochi winter games in 2014.</p></section9><section10><h2>Kamila Valieva</h2><p><strong>Date of birth:</strong> April 26, 2006</p><p><strong>Birthplace:</strong> Kazan, Russia</p><p><strong>Biggest achievement:</strong> First woman to land a quad at the Olympics\xa0</p><p><strong>Bottom Line:</strong> Kamila Valieva is one of five Russian figure skaters coached by Eteri Tutberidze at the same rink. That means she trains alongside Alexandra Trusova and Anna Shcherbakova. She started skating when she was only 3 and trains off the ice in ballet. She\'s best known for her artistry, inspired by her love of painting. Many of her programs are inspired by famous works of art.\xa0</p><p>In 2022, she became the center of an unfortunate doping scandal. She tested positive for a banned heart medication that\'s known to help improve cardiovascular endurance. She was denied her gold medal in the 2022 Russian Figure Skating Championships due to the positive drug test and is currently suspended.</p><p>Make no mistake, however. She was only 15 at the time of the scandal, and her exclusion from the sport is tragic. The adults, including her coaches and parents, are the ones to blame, not the teenager.\xa0</p></section10><section11><h2>Yulia Lipnitskaya</h2><p><strong>Date of birth:</strong> June 5, 1998</p><p><strong>Birthplace:</strong> Yekaterinburg, Russia</p><p><strong>Biggest achievement:</strong> 2014 European champion</p><p><strong>Bottom Line:</strong> Ready to be impressed? Yulia Lipnitskaya didn\'t fall a single time during the 2011-2012 season. Skating through multiple competitions without a single fall is almost unheard of. The Russian skater adores competition, and her intense training was almost entirely self-motivated.</p><p>She even competed with a twisted ankle and ended up winning first in the short program and third in the free skate. She was the youngest woman figure skater to win Olympic gold under modern rules, at only 15 years, 249 days, and she was the youngest gold medalist at the European Championships in the ladies\' singles category. Sadly, she retired in 2017 due to injuries and a battle with an eating disorder.</p><strong>RELATED: Why Don’t We Ever See Black Figure Skaters?</strong></section11><section12><h2>Alina Zagitova</h2><figcaption>\\</figcaption><p><strong>Date of birth:</strong> May 18, 2002</p><p><strong>Birthplace:\xa0</strong>Izhevsk, Russia</p><p><strong>Biggest achievement:\xa0</strong>2018 Olympic Champion, World Champion</p><p><strong>Bottom Line:</strong> Alina Zagitova was ranked as the best female skater in the world in the 2018-2019 season, with countless titles and medals under her name. She is the only Russian female figure skater to win gold at the Olympics, World Figure Skating Champtionships, European Championships and Grand Prix Final, and she\'s the first Muslim athlete to win an Olympic gold, a world title and a Super Slam.\xa0</p><p>She also broke the world record under the old system once as well as four times under the new system. She got an early start, beginning formal lessons when she was 4 at the rink her father worked at as a hockey coach. She was named female athlete of the year by the Ministry of Sport of Russia in 2018 and was listed as the top athlete in the sports category of Forbes Russia\'s Top 30 Under 30 list.\xa0</p></section12><section13><h2>Evgenia Medvedeva</h2><p><strong>Date of birth:</strong> Nov. 19, 1999</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> Only one of four women to achieve a Grand Slam</p><p><strong>Bottom Line:</strong> Evgenia Medvedeva is the first ladies\' singles skater to win Senior Worlds the year after winning Junior Worlds. In 2017, she also became the first female skater to win back-to-back world titles since Michelle Kwan did so in 2000 and 2001. And she\'s the first skater in all disciplines to complete two Grand Slams in a row.\xa0</p><p>Not impressed yet? Medvedeva set the world record score 13 times and is the first woman to pass the 80-point short program score, the 160-point free skating score, and both the 230 and 240-point total score. Now, she\'s studying at Moscow State University.\xa0</p><p>Fun fact: Medvedeva has lepidopterophobia, the fear of moths and butterflies. She also has two French bulldogs named Jerry and Tofu.</p></section13><section14><h2>Sergei Grinkov</h2><p><strong>Date of birth:</strong> Feb. 4, 1967</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> Two-time Olympic Champion, four-time World Champion in pair skating</p><p><strong>Bottom Line:</strong> Sergei Grinkov was a Russian pairs skater who hit the ice at the age of 5. He began training at the Children and Youth Sports School in Moscow, but he wasn\'t a strong solo skater. His coach opted to give him a shot in pairs skating, and he teamed up with 11-year-old skater Ekaterina Gordeeva when he was 14. They won the Junior World Championship in 1985, followed by a string of gold medals at the European Championships, World Championships and Olympic Games.\xa0</p><p>Tragically, Grinkov collapsed during practice in Lake Placid, New York, in 1995, dying of a massive heart attack. He was only 28, with a 4-year-old daughter whom he shared with partner and wife Ekaterina Gordeeva.\xa0</p></section14><section15><h2>Ekaterina Gordeeva</h2><p><strong>Date of birth:</strong> May 28, 1971</p><p><strong>Birthplace:</strong> Moscow, Russia</p><p><strong>Biggest achievement:</strong> Two-time Olympic Champion, four-time World Champion in pair skating</p><p><strong>Bottom Line:</strong> Sergei Grinkov may not be around anymore, but his partner, Ekaterina Gordeeva, is still going strong. She started skating when she was only 4 and initially had to wear several pairs of socks because the smallest available pair of skates were still too big. When she teamed up with Grinkov, their chemistry was instant. Their coach once tried to get her to team up with a new partner after Grinkov was repeatedly late, but she was adamant that he was the only one for her. Instead of switching partners, they switched coaches.\xa0</p><p>The pair went on to win almost every competition they entered. Out of 31 competitions, they finished first 24 times and never placed lower than second. After Grinkov passed away, Gordeeva remarried another famous Russian figure skater on this list, Ilia Kulik. She has two daughters, both of whom skate.\xa0</p></section15>\n</article>\n</html14>\n<html15><title>The Greatest Russian Female Figure Skaters of All Time, Ranked</title><div><div1><div><div><span>COLLECTION</span><span>22 LISTS</span></div><a>Figure Skating</a><span>Lists about the beautiful, technically grueling sports of figure skating and ice dancing and the talented athletes who compete in them all around the world.</span></div><ul><span>The Best Movies About Figure Skating</span><span>The Greatest Male Figure Skaters</span><span>The Greatest Ice Dancers Ever</span><span>The Top Russian Female Figure Skaters</span><span>The Greatest Olympic Figure Skaters</span><span>The Top American Figure Skaters</span><span>Top Female Skaters Working Today</span><span>The Top American Women in History</span><span>The Top Russian Male Figure Skaters</span><span>The Top Russian Figure Skaters</span><span>The Top Skaters Working Today</span><span>The Greatest Women of All Time</span><span>The Top American Male Figure Skaters</span><span>The Top Current Figure Skating Pairs</span><span>The Greatest Ice Dancing Pairs Ever</span><span>The Best Retired Figure Skaters</span><span>The Greatest Men Working Today</span><span>Top Ice Dancing Pairs Working Today</span><span>The Greatest Figure Skating Pairs Ever</span><span>The Top American Ice Dancers</span><span>The Greatest Ice Dancers in the World</span><span>The Top Russian Ice Dancers</span></ul></div1><article><div1><span1>It\'s easy to underestimate how popular figure skating has been in Russia since the first skating rink opened in St. Petersburg around 1865? The first World Figure Skating Championships, in fact, were held in the city in 1896. Although only male skaters were allowed to participate, female figures skaters, especially Russian ones, were soon competing and winning medals for their countries in all types of figure skating events, including the Olympics. And as far as Olympic medals go, Russia has one of the highest medal counts in the sport for the category.</span1><p0>Of course, Russian female figure skaters aren’t known and celebrated only in Russia. Most fans of figure skating across the world are familiar with the names of some of the greatest ladies in Russian figure skating history. From Irina Slutskaya, who has medaled at the Olympics, World Figure Skating Championships, and <a>Russian Figure Skating Championships</a>, to Viktoria Volchkova, who medaled at the Russian Figure Skating Championships seven times, these amazing Russian women have had a worldwide impact on the sport and made Russia proud.</p0><p1>This list contains all of the greats in Russian ladies’ single skating history, whether they are an all-time ice dance legend or a young girl in the sport.\xa0Although many have left the sport, their greatest performances will be watched and studied by aspiring skaters and sports fans for years to come. These women have left a lasting legacy, giving them the honor of being known as the greatest female figure skaters from Russia.</p1></div1><div2><ul><li><div><div><div>1</div><div>242 VOTES</div></div><a>Ekaterina Gordeeva</a></div><li>Photo: <a>YouTube</a></li><div><span>Although Ekaterina Gordeeva is remembered for her pair work, she also competed in ladies’ single pair skating.</span><span>242 votes</span><div><div>More\xa0<a>Ekaterina Gordeeva</a></div><ul><li><span>#7<span> of 43 on</span></span><a>The Greatest Female Figure Skaters of All Time</a></li><li><span>#1<span> of 30 on</span></span><a>The Greatest Figure Skating Pairs of All Time</a></li><li><span>#1<span> of 37 on</span></span><a>The Greatest Russian Figure Skaters of All Time</a></li></ul></div></div></li><li><div><div><div>2</div><div>369 VOTES</div></div><a>Alina Zagitova</a></div><li>Photo: <a>YouTube</a></li><span>369 votes</span></li><li><div><div><div>3</div><div>276 VOTES</div></div><h2>Julia Lipnitskaia</h2></div><li>Photo: <a>YouTube</a></li><div><span>Julia Lipnitskaia placed first at the European Figure Skating Championships in 2014.</span><span>276 votes</span></div></li><li><div><div><div>4</div><div>219 VOTES</div></div><a>Irina Slutskaya</a></div><li>Photo: <a>YouTube</a></li><div><span>Irina Slutskaya competed (and medaled) at both the 2002 and 2006 Winter Olympic Games.</span><span>219 votes</span></div></li><li><div><div><div>5</div><div>165 VOTES</div></div><h2>Elizaveta Tuktamysheva</h2></div><li>Photo: <a>YouTube</a></li><div><span>Elizaveta Tuktamysheva is a medalist many times over; she won, for example, the gold medal at the Trophée Eric Bompard in 2011.</span><span>165 votes</span></div></li><li><div><div><div>6</div><div>112 VOTES</div></div><a>Maria Butyrskaya</a></div><li>Photo: <a>YouTube</a></li><div><span>Maria Butyrskaya won the gold medal at the 2002 European Figure Skating Championships at the age of 29, giving her the distinction of <a>being the oldest</a> ladies’ single figure skater to have done so.</span><span>112 votes</span></div></li></ul><button>load more</button></div2></article></div></html15>\n<html16>\n<title>The Most Famous Russian Figure Skater, Ranked - StrawPoll</title>\n<div>\n<div0>\n<div0>\n<h1>\n\t\t\t\tThe Most Famous Russian Figure Skater, Ranked\n\t\t\t</h1>\n<div2>\n\t\t\t\tFigure skating, with its blend of athleticism and artistry, has always captured the hearts of audiences across the globe. Narrowing down the most notable among Russian figure skaters can provide a meaningful snapshot of the sport\'s evolution and influence within this powerhouse nation. It helps to highlight key figures who have shaped the legacy and pushed the boundaries of what is achievable on ice.\nBy participating in the ranking process, users contribute to a collective recognition of talent and achievement, offering a unique perspective on who stands out in the world of figure skating. This dynamic ranking adjusts with each vote, reflecting the current consensus on the most impactful skaters. It\'s an engaging way to see how preferences and perceptions change over time and to appreciate the stars who have defined this sport.\n\t\t\t</div2>\n<section>\n<h2>Who Is the Most Famous Russian Figure Skater?</h2>\n<ol>\n<div0>\n<div>\n<div>\n\t\t\t\t\t\t1\n\t\t\t\t\t</div>\n<div>\n<span>48</span>\n<div>points</div>\n</div>\n</div>\n<div>\n<button>Source\n</button>\n<h3>\n\t\t\t\t\t\t\tEvgeni Plushenko\n\t\t\t\t\t\t</h3>\n<div>A retired figure skater and two-time Olympic gold medalist, known for his athleticism and artistry on the ice.</div>\n<ul>\n<li>Olympic medals: 4</li>\n<li>World Championships titles: 3</li>\n</ul>\n</div>\n</div0>\n<div1>\n<div>\n<div>\n\t\t\t\t\t\t2\n\t\t\t\t\t</div>\n<div>\n<span>15</span>\n<div>points</div>\n</div>\n</div>\n<div>\n<button>Source\n</button>\n<h3>\n\t\t\t\t\t\t\tAlina Zagitova\n\t\t\t\t\t\t</h3>\n<div>An Olympic gold medalist, Zagitova is known for her technical precision and became the first Russian female skater to win the Olympic gold in PyeongChang 2018.</div>\n<ul>\n<li>Olympic medals: 1</li>\n<li>World Championships titles: 1</li>\n</ul>\n<div>\n<div>\n\t\t\t\t\t\t\tIn other topics\n\t\t\t\t\t\t</div>\n<a>Position 3 in the ranking of the most beautiful figure skater</a>\n</div>\n</div>\n</div1>\n<div2>\n<div>\n<div>\n\t\t\t\t\t\t3\n\t\t\t\t\t</div>\n<div>\n<span>7</span>\n<div>points</div>\n</div>\n</div>\n<div>\n<button>Source\n</button>\n<h3>\n\t\t\t\t\t\t\tIrina Slutskaya\n\t\t\t\t\t\t</h3>\n<div>A retired figure skater known for her technical skills, Slutskaya is a two-time World Champion and Olympic medalist.</div>\n<ul>\n<li>Olympic medals: 2</li>\n<li>World Championships titles: 2</li>\n</ul>\n</div>\n</div2>\n<div5>\n<div>\n<div>\n\t\t\t\t\t\t6\n\t\t\t\t\t</div>\n<div>\n<span>0</span>\n<div>points</div>\n</div>\n</div>\n<div>\n<button>Source\n</button>\n<h3>\n\t\t\t\t\t\t\tAnastasia Mishina and Aleksandr Galliamov\n\t\t\t\t\t\t</h3>\n<div>A pairs skating team, Mishina and Galliamov are World Champions and known for their technical skills and expressive performances.</div>\n<li>World Championships titles: 1</li>\n</div>\n</div5>\n<div7>\n<div>\n<div>\n\t\t\t\t\t\t8\n\t\t\t\t\t</div>\n<div>\n<span>0</span>\n<div>points</div>\n</div>\n</div>\n<div>\n<button>Source\n</button>\n<h3>\n\t\t\t\t\t\t\tTatiana Navka\n\t\t\t\t\t\t</h3>\n<div>A former competitive ice dancer, Navka is an Olympic gold medalist and two-time World Champion.</div>\n<ul>\n<li>Olympic medals: 1</li>\n<li>World Championships titles: 2</li>\n</ul>\n</div>\n</div7>\n<div8>\n<div>\n<div>\n\t\t\t\t\t\t9\n\t\t\t\t\t</div>\n<div>\n<span>0</span>\n<div>points</div>\n</div>\n</div>\n<div>\n<button>Source\n</button>\n<h3>\n\t\t\t\t\t\t\tEkaterina Gordeeva and Sergei Grinkov\n\t\t\t\t\t\t</h3>\n<div>A legendary pairs skating team, Gordeeva and Grinkov won two Olympic gold medals and were known for their deep connection on the ice.</div>\n<li>Olympic medals: 2</li>\n<div>\n<div>\n\t\t\t\t\t\t\tIn other topics\n\t\t\t\t\t\t</div>\n<a>Position 10 in the ranking of the most famous ice dancer</a>\n</div>\n</div>\n</div8>\n<div9>\n<div>\n<div>\n\t\t\t\t\t\t10\n\t\t\t\t\t</div>\n<div>\n<span>0</span>\n<div>points</div>\n</div>\n</div>\n<div>\n<button>Source\n</button>\n<h3>\n\t\t\t\t\t\t\tAdelina Sotnikova\n\t\t\t\t\t\t</h3>\n<div>Sotnikova is known for being the first Russian woman to win Olympic gold in ladies\' singles figure skating, achieving this feat at the Sochi 2014 Winter Olympics.</div>\n<ul>\n<li>Olympic medals: 1</li>\n<li>World Championships titles: 0</li>\n</ul>\n</div>\n</div9>\n</ol>\n</section>\n</div0>\n<div1>\n<div>\n<h4>About this ranking</h4>\n<p>This is a community-based ranking of the most famous Russian figure skater. We do our best to provide fair voting, but it is not intended to be exhaustive. So if you notice something or skater is missing, feel free to help improve the ranking!</p>\n</div>\n<div>\n<h4>Statistics</h4>\n<ul>\n<span>2651 views</span>\n<span>75 votes</span>\n<span>10 ranked items</span>\n</ul>\n</div>\n<h4>Movers &amp; Shakers</h4>\n<div>\n<h4>Voting Rules</h4>\n<p>\n\t\t\t\tA participant may cast an up or down vote for each skater once every 24 hours. The rank of each skater is then calculated from the weighted sum of all up and down votes.\n\t\t\t</p>\n</div>\n<div>\n<h4>Categories</h4>\n<ul>\n<a>Athletes</a>\n<a>Famous Personalities</a>\n<a>Voting</a>\n</ul>\n</div>\n</div1>\n</div0>\n<div0>\n<h2>More about the Most Famous Russian Figure Skater</h2>\n<div1>\n\t\t\t\t\t\tRank #1 for the most famous Russian figure skater: Evgeni Plushenko (<a>Source</a>)\n\t\t\t\t\t</div1>\n<div2>Figure skating holds a special place in Russian sports culture. The country has produced many renowned skaters who have left a mark on the sport. These athletes often start training at a young age, dedicating countless hours to perfecting their craft. Their journey usually begins in local skating clubs, where coaches identify and nurture talent.Russian figure skaters are known for their technical skills and artistic expression. They perform complex jumps, spins, and footwork with precision. Their routines often tell a story, captivating audiences worldwide. This blend of athleticism and artistry sets them apart.The path to success is not easy. Skaters face intense competition and rigorous training schedules. They must maintain peak physical condition and mental focus. Injuries are common, but resilience defines these athletes. Support from coaches, family, and fans plays a crucial role in their careers.International competitions are the ultimate stage for Russian skaters. They compete in events like the World Championships and the Winter Olympics. These platforms provide an opportunity to showcase their talent and represent their country. Winning medals brings national pride and personal fulfillment.Coaching is another key element in the success of Russian figure skaters. Experienced coaches guide skaters through their careers, offering technical advice and emotional support. Many coaches are former skaters themselves, bringing a wealth of knowledge and experience to the rink.Choreographers also contribute to the success of these athletes. They design routines that highlight the skater\'s strengths and personality. Music selection and costume design are integral parts of the performance, creating a complete package that appeals to judges and audiences alike.Training facilities in Russia are among the best in the world. State-of-the-art rinks and equipment provide skaters with the resources they need to excel. Government support for sports development ensures that young talent receives the necessary training and opportunities.The legacy of Russian figure skating is rich with memorable performances and historic achievements. Each generation of skaters builds on the success of those who came before them. They inspire future athletes to pursue their dreams and continue the tradition of excellence.Media coverage and public interest in figure skating remain high in Russia. Skaters often become national celebrities, appearing in interviews and on television. Their success stories inspire young people to take up the sport and work hard to achieve their goals.In conclusion, Russian figure skaters are celebrated for their skill, artistry, and dedication. Their journey involves hard work, support, and a passion for the sport. They continue to set high standards and inspire future generations, ensuring that the legacy of Russian figure skating endures.</div2>\n</div0>\n</div>\n</html16>\n<figcaption0><span>Kamila Valieva channeled her inner Addams at the Russian Figure Skating Championship in Krasnoyarsk, Russia.</span><span>Alexander Vilf / AP</span></figcaption0>\n<div1>\n<div0>\n<div><a>News</a><a>TV News</a></div>\n<h1>Watch pro figure skater recreate Jenna Ortega’s iconic ‘Wednesday’ dance</h1>\n<p>16-year-old Kamila Valieva was performing at the 2023 Russian Figure Skating Championships\r\n</p>\n<div><span>By</span> <a>Ali Shutler</a></div>\n<time>29th December 2022</time> </div0>\n<figcaption>Kamila Valieva at the 2023 Russian Figure Staking Championships. CREDIT: YouTube</figcaption>\n</div1>\n<html19>\n<title>Aleksandr Gorshkov, Half of First Ice Dance Olympic Gold Medal Pair, Dies - The Moscow Times</title>\n<article>\n<a>Aleksandr Gorshkov, Half of First Ice Dance Olympic Gold Medal Pair, Dies</a>\n<figcaption>\n<span>\n\t\t\t\t\t\t\t\tSoviet ice dancers Lyudmila Pakhomova and Alexander Gorshkov, 1976.\t\t\t\t\t\t\t</span>\n<span>\n\t\t\t\t\t\t\t\tYuri Zhiltukhin and V.Nikolayev / TASS \t\t\t\t\t\t\t</span>\n</figcaption>\n<div0>\n<p>Russian skater Aleksandr Gorshkov, the first Olympic ice dancing champion and president of the Russian Figure Skating Federation, has died at the age of 76, the federation said Thursday.\xa0</p>\n<p>"Aleksandr Georgyevich gave his whole life to figure skating, his contribution to the development of Russian and world skating is invaluable," federation director general Alexander Kogan\xa0told the official TASS news agency.\xa0</p>\n<p>"The whole family of Russian skaters ... is in mourning with his family. This is a huge loss for us," he added.\xa0</p>\n<p>Gorshkov with Lyudmila Pakhomova, who was also his wife, won six world titles in seven years starting in 1969. When ice dance became an Olympic event the pair won the first gold at Innsbruck in 1976.\xa0</p>\n<p>They also won the European championship six times.\xa0</p>\n<p>Pakhomova died of leukemia in 1986 aged 39.</p>\n<p>Gorshkov had headed the Russian Figure Skating Federation since 2010.\xa0</p> </div0>\n</article>\n</html19>\n<html20>\n<title>Yekaterina Gordeeva and Sergey Grinkov | Olympic Gold, Pairs Skating, Ice Dancing | Britannica</title>\n<div0>\n<div1>\n<div>\n<dl>\n<dt>Born: </dt>\n<dd>May 28, 1971, Moscow, Russia, U.S.S.R.</dd>\n</dl>\n<em>(Show\xa0more)</em>\n</div>\n<div>\n<dl>\n<dt>Born: </dt>\n<dd>February 4, 1967, Moscow</dd>\n</dl>\n<em>(Show\xa0more)</em>\n</div>\n<div>\n<dl>\n<dt>Died: </dt>\n<dd>November 20, 1995, Lake Placid, New York, U.S.</dd>\n</dl>\n<em>(Show\xa0more)</em>\n</div>\n</div1>\n<section><p0><strong>Yekaterina Gordeeva and Sergey Grinkov</strong> (respectively,  born May 28, 1971, Moscow, Russia, U.S.S.R.; born February 4, 1967, Moscow—died November 20, 1995, <a>Lake Placid</a>, <a>New York</a>, U.S.) were a Russian-born <a>figure-skating</a> pair who gained <a>worldwide</a> acclaim with four world championship titles and two <a>Olympic</a> gold medals.</p0><p1><a>Gordeeva</a> and Grinkov were teamed at age 11 and 15, respectively, and initially appeared to many critics as an awkward couple owing to a large disparity in height (Grinkov was about a foot [30 cm] taller than Gordeeva). In 1985 they proved the critics wrong by capturing the junior world championship pairs title. The following year they went on to claim the first of four world championships (1986–87 and 1989–90) and were described as lithesome and creative, mature and strong. Their career peaked when they won their first Olympic gold medal with a unanimous vote at the <a>1988 Winter Games</a> in Calgary, Alberta, Canada, but it took a surprising turn in 1990 when, claiming their competitive drive had diminished, they retired as world champions.</p1><p2>During what turned out to be a <a>hiatus</a>, the pair married in April 1991. Thanks to rule changes in amateur skating, they were able to return to Olympic competition in <a>1994 at Lillehammer</a>, Norway, where they won a second gold medal despite a few minor errors in their routine. Afterward they immigrated to the <a>United States</a> to skate professionally. Their partnership ended abruptly in November 1995 when Grinkov died of a <a>heart attack</a> while practicing a routine. Gordeeva subsequently pursued a singles skating career. She entered professional skating competitions and appeared on television in skating-themed specials. She also toured with <a>Stars on Ice</a>, the professional skating company cofounded by Olympic gold medalist <a>Scott Hamilton</a>. She coauthored (with E.M. Swift) <em>My Sergei: A Love Story</em> (1996).</p2></section> </div0>\n</html20>\n<div><section1><h2>Biography</h2><section><p><strong>Alexandra Trusova</strong> is a Russian figure skater who is the Beijing 2022 Olympic silver medallist (representing ROC), 2021 World Figure Skating Championships bronze medallist (representing FSR), and 2020 European Championships bronze medallist.</p>\n<p>The 17-year-old is best known for her quadruple-jumping abilities. She was the first female to land each of the quad Lutz, flip and toe jumps in competition, and was also the first to land two – and then three – quads in a free skate.</p>\n<p>The three-time Russian national medallist has three Grand Prix gold medals to her name, as well, and trains under <strong>Eteri Tutberidze</strong>, having briefly worked with Olympic gold medallist <strong>Evgeni Plushenko</strong>.</p>\n<p>After years of hype around her and her "quad squad" teammates, Trusova made her Olympic debut at the Olympic Winter Games <strong>Beijing 2022</strong>, competing in the women\'s event after being passed over for the team event.</p>\n<p>In the short program, Trusova fell on a triple Axel attempt, finishing in a disappointing fourth place. However, she picked herself up in the free skate. Opting against the triple Axel, she instead packed five quadruple jumps into her free routine, landing three of them cleanly to score a personal best 177.13 points in the segment for a total of 251.73 and win silver.</p><p>(As of 2 March 2022, the women\'s singles results from Beijing 2022 are provisional.)</p></section></section1><section2><h2>Featured</h2> <div><article><a>Figure Skating</a><h3>8 things we learned from the figure skating season ahead of Beijing 2022</h3></article><article><a>Figure Skating</a><h3>World figure skating championships 2021: Preview, TV &amp; streaming details, who’s competing, and more</h3></article><article><a>Figure Skating</a><h3>Evgeni Plushenko exclusive: The coach of Kostornaia and Trusova says, we are ‘a very powerful team’</h3></article></div></section2></div>
"""

# 调用函数
cleaned_html = remove_nonsemantic_tags(HTML_test)
# cleaned_html = simplify_html(BeautifulSoup(cleaned_html, 'html.parser'))
print(cleaned_html)
# cleaned_html = remove_nonsemantic_tags(simplify_html(BeautifulSoup(HTML_test, 'html.parser')))

# # 输出结果
# print(cleaned_html)

# 解析并生成文档树
document_tree = build_document_tree(cleaned_html)

print_tree(document_tree)

<html0> <title>  The Greatest Russian Male Figure Skaters of All Time </title> COLLECTION 22 LISTS Figure Skating Lists about the beautiful, technically grueling sports of figure skating and ice dancing and the talented athletes who compete in them all around the world. <ul>  The Best Movies About Figure Skating  The Greatest Male Figure Skaters  The Greatest Ice Dancers Ever  The Top Russian Female Figure Skaters  The Greatest Olympic Figure Skaters  The Top American Figure Skaters  Top Female Skaters Working Today  The Top American Women in History  The Top Russian Male Figure Skaters  The Top Russian Figure Skaters  The Top Skaters Working Today  The Greatest Women of All Time  The Top American Male Figure Skaters  The Top Current Figure Skating Pairs  The Greatest Ice Dancing Pairs Ever  The Best Retired Figure Skaters  The Greatest Men Working Today  Top Ice Dancing Pairs Working Today  The Greatest Figure Skating Pairs Ever  The Top American Ice Dancers  The Greatest Ice Dancers 

/tmp/ipykernel_2399415/3164862471.py:112: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  direct_text = ''.join(list_item.find_all(text=True, recursive=False)).strip()


NameError: name 'parent' is not defined